In [1]:
#RUN IN SAOD or SAGEO (sageo has a problem with pandas min function)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import geopandas as gpd
import scipy.io
from functools import reduce
from pyproj import Proj, transform #can run this from sageo
import rasterio
import rasterstats
from netCDF4 import Dataset
import shapely
import rioxarray
import xarray
from functools import reduce
import glob
import re
import math
import folium
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from matplotlib.transforms import Affine2D
from matplotlib.lines import Line2D
from shapely.wkt import loads, dumps
from shapely.ops import polygonize, split
from fiona.crs import from_epsg

from typing import Callable
from scipy.spatial.distance import pdist
import skgstat
import xdem

# from osgeo import ogr
from shapely.geometry import MultiLineString, Point, LineString, Polygon
from shapely import wkt
import sys

pd.set_option('display.max_columns', 500)
shapely.speedups.enable()

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

date_string='Spring2022_v6'

density=0.900
vertically_averaged_velocity = 1

In [2]:
#read in fluxgate lines
flux_lines= gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Kochtitzky_fluxgates.gpkg')
flux_lines_ice_sheet = flux_lines[flux_lines['Glacier_type']=='Ice sheet']
flux_lines_periphery = flux_lines[flux_lines['Glacier_type']=='Periphery']

# put ice sheet attributes with the flux gates
GRE_outlines = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Mouginot_GIS_basins/Kochtitzky_edits_Mouginot_GIS_basins_3995_v3.gpkg')
flux_lines_ice_sheet=gpd.sjoin(flux_lines_ice_sheet,GRE_outlines,how='left', op='intersects')[['Glacier_type','NAME','geometry']]

#clip periphery to RGI
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')   
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['Name']=RGI['Name2']
#Get only the points on the glaciers (RGI outlines)
flux_lines_periphery = gpd.overlay(flux_lines_periphery, RGI, how='intersection')[['Glacier_type','RGIId','RGIid_2', 'Name2','geometry']]

flux_lines=flux_lines_ice_sheet.append(flux_lines_periphery,sort=False)
flux_lines.reset_index(inplace=True)
flux_lines.loc[flux_lines['NAME'].isna(),'NAME']=flux_lines['RGIid_2']

flux_lines.to_file('Greenland/Grenland_flux_lines_'+date_string+'.gpkg',driver='GPKG')

In [3]:
#Make points every distance along your fluxgates or lines
distance=25 #change this distance based on how frequently you want points
export_flux_gate_points = pd.DataFrame()
inProj = Proj(init='epsg:3995')
outProj = Proj(init='epsg:4326')
counter=0
for i in range(0,len(flux_lines)): #iterate through all your fluxgate lines and create points every distance along them
    x1,y1 = flux_lines.loc[i].geometry.centroid.x,flux_lines.loc[i].geometry.centroid.y
    x2,y2 = transform(inProj,outProj,x1,y1)
    projection = ccrs.Orthographic(central_longitude=x2, central_latitude=y2)
    flux_lines_proj = flux_lines.to_crs(projection)
    easting = []
    northing = []
    name = []
    region = []
    for x in range(0,int(flux_lines_proj.loc[i].geometry.length),distance):
        x,y = flux_lines_proj.loc[i].geometry.interpolate(x).xy   
        easting.append(x[0])
        northing.append(y[0])
        name.append(flux_lines_proj.loc[i].NAME)
#         creation.append(flux_lines_proj.loc[i].creation) #save other attributes
#         region.append(flux_lines_proj.loc[i].Region) #save other attribute        
    #read the data to a pandas dataframe
    lists = {'easting':easting,'northing':northing,'name':name}
    df = pd.DataFrame(lists)
    #read the data into a GeoDataFrame
    flux_gate_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.easting, df.northing))
    flux_gate_points.crs = projection#{'init' :'epsg:3995'} #define projection
    flux_gate_points = flux_gate_points.to_crs('epsg:3995')
    if counter == 0:
        export_flux_gate_points=flux_gate_points
        counter=counter+1
    else:
        export_flux_gate_points=export_flux_gate_points.append(flux_gate_points)
        
export_flux_gate_points.loc[:,'easting']=export_flux_gate_points.geometry.x
export_flux_gate_points.loc[:,'northing']=export_flux_gate_points.geometry.y
export_flux_gate_points.to_file('Greenland/Grenland_flux_gate_points_25m_'+date_string+'.gpkg',driver='GPKG')#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https

In [4]:
flux_gate_points = gpd.read_file('Greenland/Grenland_flux_gate_points_25m_'+date_string+'.gpkg',driver='GPKG')

#now point query all the tiff data for each year by region - Greenland
gre_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/GRE*_polar.tif'))
for i in range(0,len(gre_datasets)):
    data_name= re.search('240_(.*)_polar.tif',gre_datasets[i]).group(1)
    flux_gate_points[str(data_name)]=rasterstats.point_query(flux_gate_points.geometry,gre_datasets[i],interpolate='nearest')    

#Pull in measures data
gre_measures_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Greenland_measures/*polar.tif'))
for i in range(0,len(gre_measures_datasets)):
    data_name= re.search('00_(.*)_v02.1_polar.tif',gre_measures_datasets[i]).group(1)
    data_name = str('GM_'+data_name)
    flux_gate_points[str(data_name)]=rasterstats.point_query(flux_gate_points.geometry,gre_measures_datasets[i],interpolate='nearest')    

#Pull in special velocity data for northern Greenland
flux_gate_points['Northern_velocity']=rasterstats.point_query(flux_gate_points.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_W_northern_velocity_2016to2020_composite.tif',interpolate='nearest')
flux_gate_points['Northern_velocity_x']=rasterstats.point_query(flux_gate_points.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_W_northern_velocity_2016to2020_composite_x.tif',interpolate='nearest')
flux_gate_points['Northern_velocity_y']=rasterstats.point_query(flux_gate_points.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_W_northern_velocity_2016to2020_composite_y.tif',interpolate='nearest')
# Greenland_flux_gates['Northern_velocity2']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_E_northern_velocity_2016to2020_composite.tif',interpolate='nearest')
flux_gate_points['Northern_velocity_x2']=rasterstats.point_query(flux_gate_points.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_E_northern_velocity_2016to2020_composite_x.tif',interpolate='nearest')
flux_gate_points['Northern_velocity_y2']=rasterstats.point_query(flux_gate_points.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_E_northern_velocity_2016to2020_composite_y.tif',interpolate='nearest')

flux_gate_points['Northern_velocity'].fillna(np.sqrt(flux_gate_points['Northern_velocity_x2']**2+flux_gate_points['Northern_velocity_y2']**2),inplace=True)
flux_gate_points['Northern_velocity_x'].fillna(flux_gate_points['Northern_velocity_x2'],inplace=True)
flux_gate_points['Northern_velocity_y'].fillna(flux_gate_points['Northern_velocity_y2'],inplace=True)
flux_gate_points.drop(columns=['Northern_velocity_x2','Northern_velocity_y2'],inplace=True)

flux_gate_points.to_file('Greenland/GRE_flux_gates_velocity_'+date_string+'_2020.gpkg', driver="GPKG")#change file name based on distance


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: Da

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: Da

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: Da

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: Da

In [5]:
Greenland_flux_gates = gpd.read_file('Greenland/GRE_flux_gates_velocity_'+date_string+'_2020.gpkg')

#read in thickness change for each point
Greenland_flux_gates['dhdt_early']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt/aa_Hugonnet_thickness_early_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['dhdt_err_early']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt_err/aa_Hugonnet_thickness_err_early_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['dhdt_late']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt/aa_Hugonnet_thickness_late_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['dhdt_err_late']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt_err/aa_Hugonnet_thickness_err_late_mosaic.vrt',interpolate='nearest')

Greenland_flux_gates['dhdt_Khan_early']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Greenland_Calving/Geodetic_mass_balance_Data/dhdt_2000-2010_GrIS.tif',interpolate='nearest')
Greenland_flux_gates['dhdt_Khan_err_early']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Greenland_Calving/Geodetic_mass_balance_Data/dhdt_2000-2010_GrIS_err.tif',interpolate='nearest')
Greenland_flux_gates['dhdt_Khan_late']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Greenland_Calving/Geodetic_mass_balance_Data/dhdt_2011-2019_GrIS.tif',interpolate='nearest')
Greenland_flux_gates['dhdt_Khan_err_late']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Greenland_Calving/Geodetic_mass_balance_Data/dhdt_2011-2019_GrIS_err.tif',interpolate='nearest')

Greenland_flux_gates['dhdt_early'].fillna(Greenland_flux_gates['dhdt_Khan_early'],inplace=True)
Greenland_flux_gates['dhdt_err_early'].fillna(Greenland_flux_gates['dhdt_Khan_err_early'],inplace=True)
Greenland_flux_gates['dhdt_late'].fillna(Greenland_flux_gates['dhdt_Khan_late'],inplace=True)
Greenland_flux_gates['dhdt_err_late'].fillna(Greenland_flux_gates['dhdt_Khan_err_late'],inplace=True)

Greenland_flux_gates.to_file('Greenland/GRE_flux_gates_mass_balance_'+date_string+'2020.gpkg', driver="GPKG")

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [6]:
#Read in GlaThiDa
# GlaThiDa= gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Arctic_GlaThiDa_EPSG3995.gpkg')
GlaThiDa = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Thickness_Data/Greenland_OIB_within_10km_of_fluxgates_or_near_termini_3995.gpkg')
#Add GlaThiDa data to the dataset
GlaThiDa['geometry']=GlaThiDa.buffer(100)

#get thickness data
Greenland_flux_gates = gpd.read_file('Greenland/GRE_flux_gates_mass_balance_'+date_string+'2020.gpkg')
# Greenland_flux_gates['Farinotti_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
# Greenland_flux_gates['model1_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
# Greenland_flux_gates['model2_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['bed_machine']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Greenland_Calving/Thickness_Data/BedMachine_v4_Morlighem/BedMachineGreenland-2021-01-26_thickness_polar.tif',interpolate='nearest')
Greenland_flux_gates['bed_machine_err']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Greenland_Calving/Thickness_Data/BedMachine_v4_Morlighem/BedMachineGreenland-2021-01-26_err_polar.tif',interpolate='nearest')

Greenland_flux_gates['Milan_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['Milan_thickness_err']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for Greenland
Greenland_flux_gates_with_glathida = gpd.sjoin(Greenland_flux_gates,GlaThiDa,how='left', op='intersects')
Greenland_flux_gates_with_glathida = Greenland_flux_gates_with_glathida[['easting','northing','THICK','FRAME']]#loose all the attributes we don't care about
Greenland_flux_gates_with_glathida = Greenland_flux_gates_with_glathida.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Greenland_flux_gates=Greenland_flux_gates.merge(Greenland_flux_gates_with_glathida,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

#get the date for OIB data
for i in range(0,len(Greenland_flux_gates)):
    if ~np.isnan(Greenland_flux_gates['FRAME'].loc[i]):
        Greenland_flux_gates.loc[i,'thickness_date'] = int(str(Greenland_flux_gates['FRAME'].loc[i])[0:4])
        
Greenland_flux_gates.to_file('Greenland/GRE_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

In [7]:
Greenland_flux_gates

,easting,northing,name,1999_v_err,1999_v,1999_vx,1999_vy,2000_count,2000_v_err,2000_v,2000_vx,2000_vy,2001_count,2001_v_err,2001_v,2001_vx,2001_vy,2002_count,2002_v_err,2002_v,2002_vx,2002_vy,2003_count,2003_v_err,2003_v,2003_vx,2003_vy,2004_count,2004_v_err,2004_v,2004_vx,2004_vy,2005_count,2005_v_err,2005_v,2005_vx,2005_vy,2006_count,2006_v_err,2006_v,2006_vx,2006_vy,2007_count,2007_v_err,2007_v,2007_vx,2007_vy,2008_count,2008_v_err,2008_v,2008_vx,2008_vy,2009_count,2009_v_err,2009_v,2009_vx,2009_vy,2010_count,2010_v_err,2010_v,2010_vx,2010_vy,2011_count,2011_v_err,2011_v,2011_vx,2011_vy,2012_count,2012_v_err,2012_v,2012_vx,2012_vy,2013_count,2013_v_err,2013_v,2013_vx,2013_vy,2014_count,2014_v_err,2014_v,2014_vx,2014_vy,2015_count,2015_v_err,2015_v,2015_vx,2015_vy,2016_count,2016_v_err,2016_v,2016_vx,2016_vy,2017_count,2017_v_err,2017_v,2017_vx,2017_vy,2018_count,2018_v_err,2018_v,2018_vx,2018_vy,GM_2014_2015_ex,GM_2014_2015_ey,GM_2014_2015_vv,GM_2014_2015_vx,GM_2014_2015_vy,GM_2015_2016_ex,GM_2015_2016_ey,GM_2015_2016_vv,GM_2015_2016_vx,GM_2015_2016_vy,GM_2016_2017_ex,GM_2016_2017_ey,GM_2016_2017_vv,GM_2016_2017_vx,GM_2016_2017_vy,GM_2017_2018_ex,GM_2017_2018_ey,GM_2017_2018_vv,GM_2017_2018_vx,GM_2017_2018_vy,GM_2000_2001_ex,GM_2000_2001_ey,GM_2000_2001_vv,GM_2000_2001_vx,GM_2000_2001_vy,GM_2005_2006_ex,GM_2005_2006_ey,GM_2005_2006_vv,GM_2005_2006_vx,GM_2005_2006_vy,GM_2006_2007_ex,GM_2006_2007_ey,GM_2006_2007_vv,GM_2006_2007_vx,GM_2006_2007_vy,GM_2007_2008_ex,GM_2007_2008_ey,GM_2007_2008_vv,GM_2007_2008_vx,GM_2007_2008_vy,GM_2008_2009_ex,GM_2008_2009_ey,GM_2008_2009_vv,GM_2008_2009_vx,GM_2008_2009_vy,GM_2009_2010_ex,GM_2009_2010_ey,GM_2009_2010_vv,GM_2009_2010_vx,GM_2009_2010_vy,GM_2012_2013_ex,GM_2012_2013_ey,GM_2012_2013_vv,GM_2012_2013_vx,GM_2012_2013_vy,Northern_velocity,Northern_velocity_x,Northern_velocity_y,dhdt_early,dhdt_err_early,dhdt_late,dhdt_err_late,dhdt_Khan_early,dhdt_Khan_err_early,dhdt_Khan_late,dhdt_Khan_err_late,geometry,bed_machine,bed_machine_err,Milan_thickness,Milan_thickness_err,THICK,FRAME,thickness_date
0,-2.265026e+06,-2.249917e+06,KIATTUUT-QOOQQUP,37.712971,54.197815,40.786144,35.691650,38.0,16.005251,41.818005,18.209877,37.644997,33.0,24.575344,38.583450,22.767639,31.149916,17.0,40.238838,63.705154,40.778126,48.943756,13.0,51.147881,59.771122,34.409821,48.872807,6.0,49.824165,21.743340,-1.806418,-21.668171,8.0,36.435009,4.474655,-1.892625,4.054690,30.0,12.040308,24.288162,16.161638,18.130533,15.0,3.829997,36.018932,24.385000,26.509155,16.0,21.219667,26.936899,23.208921,13.672691,22.0,17.382555,43.181889,29.664412,31.379904,18.0,10.492643,39.952831,27.870415,28.626364,17.0,20.961485,43.356236,23.118868,36.678074,43.0,15.024530,34.251736,24.663858,23.767107,148.0,6.592764,48.498539,34.058296,34.527393,320.0,5.131060,48.524254,31.260185,37.113392,347.0,4.823144,52.865669,34.316158,40.214180,472.0,4.399333,51.660496,27.014637,44.034264,719.0,3.698101,46.241920,30.414520,34.832058,437.0,4.876183,53.967621,29.347544,45.290459,27.820324,42.334141,57.909901,-3.971312,-57.773571,3.955592,7.916884,2.709903,2.483119,1.085217,2.693179,3.316055,9.378020,9.195504,1.841190,1.207007,1.316377,7.917215,-5.292315,5.888437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.916837,20.070873,82.466812,-24.058023,-78.879570,3.752824,14.222678,2.852514,-2.480064,1.409296,12.296901,31.441433,104.674690,-63.80978,-82.976524,NaN,NaN,NaN,-1.665886,1.616307,0.001830,1.058147,-0.067273,0.047273,-0.253889,0.125000,POINT (-2265025.504 -2249916.831),123.246475,63,NaN,NaN,NaN,NaN,NaN
1,-2.265025e+06,-2.249943e+06,KIATTUUT-QOOQQUP,53.425064,118.249367,-57.212147,-103.487595,12.0,30.983711,144.950394,-88.004921,-115.177048,4.0,74.935165,135.290207,-75.588867,-112.204117,3.0,92.003967,110.159904,-53.978611,-96.028717,1.0,222.026443,116.911949,-105.201828,-50.999790,4.0,120.965302,105.780807,-11.675596,-105.134483,NaN,NaN,NaN,NaN,NaN,11.0,26.989134,120.776901,-68.446838,-99.509247,5.0,51.158020,133.7

In [8]:
#Read in all the files with thickness data
Arctic_discharge = gpd.read_file('Greenland/GRE_flux_gates_thick_'+date_string+'2020.gpkg')

Arctic_discharge['length']=25 #length of point spacing from fluxlines
#convert the dataframe to all numbers (some columns are objects not float 64)
for i in range(0, 85):
    Arctic_discharge.iloc[:,i] = pd.to_numeric(Arctic_discharge.iloc[:,i], errors='ignore')

In [9]:
GM_errx = ['GM_2000_2001_ex', 'GM_2005_2006_ex', 'GM_2006_2007_ex', 'GM_2007_2008_ex', 'GM_2008_2009_ex', 'GM_2009_2010_ex', 'GM_2012_2013_ex', 'GM_2014_2015_ex', 'GM_2015_2016_ex', 'GM_2016_2017_ex', 'GM_2017_2018_ex']
GM_erry = ['GM_2000_2001_ey', 'GM_2005_2006_ey', 'GM_2006_2007_ey', 'GM_2007_2008_ey', 'GM_2008_2009_ey', 'GM_2009_2010_ey', 'GM_2012_2013_ey', 'GM_2014_2015_ey', 'GM_2015_2016_ey', 'GM_2016_2017_ey', 'GM_2017_2018_ey']
GM_v = ['GM_2000_2001_vv', 'GM_2005_2006_vv', 'GM_2006_2007_vv', 'GM_2007_2008_vv', 'GM_2008_2009_vv', 'GM_2009_2010_vv', 'GM_2012_2013_vv', 'GM_2014_2015_vv', 'GM_2015_2016_vv', 'GM_2016_2017_vv', 'GM_2017_2018_vv']
GM_vx = ['GM_2000_2001_vx', 'GM_2005_2006_vx', 'GM_2006_2007_vx', 'GM_2007_2008_vx', 'GM_2008_2009_vx', 'GM_2009_2010_vx', 'GM_2012_2013_vx', 'GM_2014_2015_vx', 'GM_2015_2016_vx', 'GM_2016_2017_vx', 'GM_2017_2018_vx']
GM_vy = ['GM_2000_2001_vy', 'GM_2005_2006_vy', 'GM_2006_2007_vy', 'GM_2007_2008_vy', 'GM_2008_2009_vy', 'GM_2009_2010_vy', 'GM_2012_2013_vy', 'GM_2014_2015_vy', 'GM_2015_2016_vy', 'GM_2016_2017_vy', 'GM_2017_2018_vy']

IL_v_err = ['2001_v_err', '2006_v_err', '2007_v_err', '2008_v_err', '2009_v_err', '2010_v_err', '2013_v_err', '2015_v_err', '2016_v_err', '2017_v_err', '2018_v_err']
IL_v = ['2001_v', '2006_v', '2007_v', '2008_v', '2009_v', '2010_v', '2013_v', '2015_v', '2016_v', '2017_v', '2018_v']
IL_vx = ['2001_vx', '2006_vx', '2007_vx', '2008_vx', '2009_vx', '2010_vx', '2013_vx', '2015_vx', '2016_vx', '2017_vx', '2018_vx']
IL_vy = ['2001_vy', '2006_vy', '2007_vy', '2008_vy', '2009_vy', '2010_vy', '2013_vy', '2015_vy', '2016_vy', '2017_vy', '2018_vy']


# %matplotlib notebook
for n in range(0,len(GM_errx)):
    GM_verr_field = 'GM_' + str(IL_v[n]) + '_err'
    GM_err_ratio_field = 'GM_' + str(IL_v[n]) + '_err_ratio'
    Arctic_discharge[GM_verr_field] = np.sqrt(Arctic_discharge[GM_errx[n]]**2+Arctic_discharge[GM_erry[n]]**2)
    Arctic_discharge[GM_err_ratio_field] = Arctic_discharge[GM_verr_field]/Arctic_discharge[GM_v[n]]
    
    for i in range(0,len(Arctic_discharge)):
        if (Arctic_discharge[GM_err_ratio_field][i]<Arctic_discharge[IL_v_err[n]][i]):
            Arctic_discharge.loc[i,IL_v_err[n]] = Arctic_discharge[GM_verr_field][i]
            Arctic_discharge.loc[i,IL_v[n]] = Arctic_discharge[GM_v[n]][i]
            Arctic_discharge.loc[i,IL_vx[n]] = Arctic_discharge[GM_vx[n]][i]
            Arctic_discharge.loc[i,IL_vy[n]] = Arctic_discharge[GM_vy[n]][i]

v_vals = ['1999_v','2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v','2010_v','2011_v','2012_v','2013_v','2014_v','2015_v','2016_v','2017_v','2018_v']
err_vals = ['1999_v_err','2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err','2010_v_err','2011_v_err','2012_v_err','2013_v_err','2014_v_err','2015_v_err','2016_v_err','2017_v_err','2018_v_err']

for i in range(0,len(v_vals)):
    ratio_vals=str(err_vals[i])+'_ratio'
    Arctic_discharge[ratio_vals]=Arctic_discharge[err_vals[i]]/Arctic_discharge[v_vals[i]]
    
Arctic_discharge['2000_2010_min_vel_ratio'] = Arctic_discharge[['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']].min(axis=1,skipna=True)
Arctic_discharge['2010_2018_min_vel_ratio'] = Arctic_discharge[['2010_v_err_ratio', '2011_v_err_ratio', '2012_v_err_ratio', '2013_v_err_ratio', '2014_v_err_ratio', '2015_v_err_ratio', '2016_v_err_ratio', '2017_v_err_ratio', '2018_v_err_ratio']].min(axis=1,skipna=True)

In [10]:
#Figure out how many good obs we have, so we can throw out bad obs
Arctic_discharge['num_bad_obs_snr_too_high_early'] = 0
Arctic_discharge['num_good_obs_early'] = 0
Arctic_discharge['num_nan_obs_early'] = 0

for i in range(0,len(Arctic_discharge)):
    for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
        if math.isnan(Arctic_discharge[err_ratio_field][i]):
            Arctic_discharge.loc[i,'num_nan_obs_early'] = Arctic_discharge['num_nan_obs_early'][i]+1
        elif Arctic_discharge[err_ratio_field][i] >= 1:
            Arctic_discharge.loc[i,'num_bad_obs_snr_too_high_early'] = Arctic_discharge['num_bad_obs_snr_too_high_early'][i]+1
        elif Arctic_discharge[err_ratio_field][i] < 1:
            Arctic_discharge.loc[i,'num_good_obs_early'] = Arctic_discharge['num_good_obs_early'][i]+1
            
#put a nan in places where the errors are high and we have good data elsewhere.
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['num_good_obs_early'][i]>=1:
        for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
            if Arctic_discharge[err_ratio_field][i]>=1:
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan #get the velocity field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan # get the error field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the x field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the y field and set to nan
                
Arctic_discharge['num_bad_obs_snr_too_high_late'] = 0
Arctic_discharge['num_good_obs_late'] = 0
Arctic_discharge['num_nan_obs_late'] = 0

for i in range(0,len(Arctic_discharge)):
    for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
        if math.isnan(Arctic_discharge[err_ratio_field][i]):
            Arctic_discharge.loc[i,'num_nan_obs_late'] = Arctic_discharge['num_nan_obs_late'][i]+1
        elif Arctic_discharge[err_ratio_field][i] >= 1:
            Arctic_discharge.loc[i,'num_bad_obs_snr_too_high_late'] = Arctic_discharge['num_bad_obs_snr_too_high_late'][i]+1
        elif Arctic_discharge[err_ratio_field][i] < 1:
            Arctic_discharge.loc[i,'num_good_obs_late'] = Arctic_discharge['num_good_obs_late'][i]+1
            
#put a nan in places where the errors are high and we have good data elsewhere.
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['num_good_obs_late'][i]>=1:
        for err_ratio_field in ['2010_v_err_ratio', '2011_v_err_ratio', '2012_v_err_ratio', '2013_v_err_ratio', '2014_v_err_ratio', '2015_v_err_ratio', '2016_v_err_ratio', '2017_v_err_ratio', '2018_v_err_ratio']:
            if Arctic_discharge[err_ratio_field][i]>=1:
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan #get the velocity field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan # get the error field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the x field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the y field and set to nan

In [11]:
#Create velocity products
#figure out the direction of flow (use mean)
Arctic_discharge.loc[:,'2000_2010_mean_x'] = Arctic_discharge[['2000_vx','2001_vx','2002_vx','2003_vx','2004_vx','2005_vx','2006_vx','2007_vx','2008_vx','2009_vx']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2000_2010_mean_y'] = Arctic_discharge[['2000_vy','2001_vy','2002_vy','2003_vy','2004_vy','2005_vy','2006_vy','2007_vy','2008_vy','2009_vy']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_x'] = Arctic_discharge[['2010_vx','2011_vx','2012_vx','2013_vx','2014_vx','2015_vx','2016_vx','2017_vx','2018_vx']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_y'] = Arctic_discharge[['2010_vy','2011_vy','2012_vy','2013_vy','2014_vy','2015_vy','2016_vy','2017_vy','2018_vy']].mean(axis=1,skipna=True)

#Take a mean including other years
Arctic_discharge.loc[:,'2000_2010_mean'] = Arctic_discharge[['2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2000_2010_mean_err'] = Arctic_discharge[['2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err']].mean(axis=1,skipna=True)

Arctic_discharge['2010_2018_mean'] = Arctic_discharge[['2010_v','2011_v','2012_v','2013_v','2014_v','2015_v','2016_v','2017_v','2018_v']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_err'] = Arctic_discharge[['2010_v_err','2011_v_err','2012_v_err','2013_v_err','2014_v_err','2015_v_err','2016_v_err','2017_v_err','2018_v_err']].mean(axis=1,skipna=True)


# fill gaps in the velocity record
# fill 2000-2010 holes with newer data
Arctic_discharge.loc[:,'2010_2018_mean'] = Arctic_discharge['2010_2018_mean'].interpolate(method='nearest',limit=20) #fill no data gaps with nearby values, limit to prevent data filling in Jan Mayen where there is lots of nans
Arctic_discharge.loc[:,'2010_2018_mean_err'] = Arctic_discharge['2010_2018_mean_err'].interpolate(method='nearest',limit=20) #fill no data gaps with nearby values, limit to prevent data filling in Jan Mayen where there is lots of nans

#Fill the uncertainty from custom velocity maps with 0.4 m/yr (bedrock movement in these maps)
northern_vel_idx = Arctic_discharge['2010_2018_mean'].isna()&Arctic_discharge['Northern_velocity'].notna()
#insert northern_velocities where there are holes in ITS_LIVE
Arctic_discharge.loc[northern_vel_idx,'2010_2018_mean']=Arctic_discharge[northern_vel_idx]['Northern_velocity']
#calculate errors
Arctic_discharge.loc[northern_vel_idx,'2010_2018_mean_err'] = 0.4


print('number of nans in 2000-2010 velocity layer that are filled with 2010-2020',Arctic_discharge['2000_2010_mean'].isna().sum(),'out of ', len(Arctic_discharge))

Arctic_discharge['2000_2010_mean'].fillna(Arctic_discharge['2010_2018_mean'], inplace=True)
Arctic_discharge['2000_2010_mean_err'].fillna(Arctic_discharge['2000_2010_mean_err'], inplace=True)

print(np.mean(Arctic_discharge['2000_2010_mean_err']))
#if the error is bigger than the velocity, replace with the velocity value
i= Arctic_discharge['2000_2010_mean_err']>Arctic_discharge['2000_2010_mean']
Arctic_discharge.loc[i,'2000_2010_mean_err']=Arctic_discharge['2000_2010_mean']
i=Arctic_discharge['2010_2018_mean_err']>Arctic_discharge['2010_2018_mean']
Arctic_discharge.loc[i,'2010_2018_mean_err']=Arctic_discharge['2010_2018_mean']
print(np.mean(Arctic_discharge['2000_2010_mean_err']))

Arctic_discharge['2000_2010_mean'].fillna(value = 0, inplace = True)
Arctic_discharge['2000_2010_mean_err'].fillna(value = 0, inplace = True)
Arctic_discharge['2010_2018_mean'].fillna(value = 0, inplace = True)
Arctic_discharge['2010_2018_mean_err'].fillna(value = 0, inplace = True)

number of nans in 2000-2010 velocity layer that are filled with 2010-2020 2555 out of  122135
32.18659501039511
27.84208879767385


In [12]:
#figure out glacier thickness
Arctic_discharge['best_thickness']=Arctic_discharge['THICK']
print('Number of data points filled with OIB data',Arctic_discharge['best_thickness'].count())

#insert uncertainties
observation_idx=Arctic_discharge['THICK'].notna()
Arctic_discharge['thickness_uncertainty']=np.nan
Arctic_discharge.loc[observation_idx,'thickness_uncertainty']=10

Arctic_discharge['best_thickness'].fillna(Arctic_discharge['bed_machine'],inplace=True)
Arctic_discharge['thickness_uncertainty'].fillna(Arctic_discharge['bed_machine_err'],inplace=True)
#assign thickness dates from obs and give everything else a date of 2000
Arctic_discharge['thickness_date'].fillna(2007,inplace=True)

Arctic_discharge['best_thickness'].fillna(Arctic_discharge['Milan_thickness'],inplace=True)
Arctic_discharge['thickness_uncertainty'].fillna(Arctic_discharge['Milan_thickness_err'],inplace=True)
Arctic_discharge['thickness_date'].fillna(2017,inplace=True)

#fill Millan data for periphery glaciers
Arctic_discharge.loc[(Arctic_discharge['bed_machine']<Arctic_discharge['Milan_thickness'])&~observation_idx,'best_thickness']=Arctic_discharge['Milan_thickness']
Arctic_discharge.loc[(Arctic_discharge['bed_machine']<Arctic_discharge['Milan_thickness'])&~observation_idx,'thickness_uncertainty']=Arctic_discharge['Milan_thickness_err']
Arctic_discharge.loc[(Arctic_discharge['bed_machine']<Arctic_discharge['Milan_thickness'])&~observation_idx,'thickness_date']=2017

#fill missing GlaThiDa dates with 2000
Arctic_discharge.loc[Arctic_discharge['THICK'].astype(np.float)>0,'thickness_date'].fillna(2000)

#calculate data you need for interpolated thickness modeling
#Calculate your own U shaped valley for all glaciers and fill in holes in observations
Arctic_discharge['position']=Arctic_discharge.index #create positions
Arctic_discharge_position_counter = Arctic_discharge[['name','position']].groupby(['name']).min().reset_index()
Arctic_discharge_position_counter = Arctic_discharge_position_counter.rename(columns={"position": "min_position"})
Arctic_discharge=Arctic_discharge.merge(Arctic_discharge_position_counter,how='left',left_on='name',right_on='name')

Arctic_discharge_position_counter = Arctic_discharge[['name','position']].groupby(['name']).max().reset_index()
Arctic_discharge_position_counter = Arctic_discharge_position_counter.rename(columns={"position": "max_position"})
Arctic_discharge=Arctic_discharge.merge(Arctic_discharge_position_counter,how='left',left_on='name',right_on='name')

Arctic_discharge['distance_across']=(Arctic_discharge['position']-Arctic_discharge['min_position'])*25#25 meters for the spacing you defined above
Arctic_discharge['position']=(Arctic_discharge['position']-Arctic_discharge['min_position'])

#create a mask to find areas near the glacier margin
Arctic_discharge['velocity_glacier_edge_mask']=0
for i in range(0,len(Arctic_discharge)):
    if (Arctic_discharge['position'][i]<20)|(Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i]-Arctic_discharge['position'][i]<20): #if you are within 500 m (20*25 m points), then we create a mask for velocity orientation mapping later
        Arctic_discharge.loc[i,'velocity_glacier_edge_mask']=1


Number of data points filled with OIB data 11543


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [13]:
#set up data for interpolated thickness model
glacier_best_thickness=[]
name = []
centerline_distance_across = []

for i in range(0,len(Arctic_discharge)):
    #          dividing by 5 gives you middle 20% of the glacier                             This subtraction is giving you the number of position away you are from the middle, so if you are in the middle 20% these calculations will happen
    if (((Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i]))/5 >= np.abs(Arctic_discharge['position'][i]-((Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i])/2))): #change 2.5 to include more or less points
        name.append(Arctic_discharge['name'][i])
        glacier_best_thickness.append(Arctic_discharge['best_thickness'][i])
        centerline_distance_across.append(Arctic_discharge['distance_across'][i])

centerline_thickness = pd.DataFrame(data={'name':name,'centerline_depth':glacier_best_thickness,'centerline_distance_across':centerline_distance_across})
centerline_thickness=centerline_thickness.groupby(['name']).mean().reset_index()
centerline_thickness['do_we_have_an_observation_in_middle_fifth']=np.nan
centerline_thickness.loc[centerline_thickness['centerline_depth']>0,'do_we_have_an_observation_in_middle_fifth']=1 #add a one if there is a centerline depth greater than 0 because this means this glacier has a centerline observation
Arctic_discharge=Arctic_discharge.merge(centerline_thickness,how='left',left_on='name',right_on='name')

Arctic_discharge['velocity_glacier_edge_mask']=0
i=(Arctic_discharge['position']<20)|(Arctic_discharge['max_position']-Arctic_discharge['min_position']-Arctic_discharge['position']<20) #if you are within 500 m (20*25 m points), then we create a mask for velocity orientation mapping later
Arctic_discharge.loc[i,'velocity_glacier_edge_mask']=1


#calculate U shaped valley from centerline thickness
centerline_depth=Arctic_discharge['centerline_depth']
distance1=Arctic_discharge['centerline_distance_across']
distance2=np.abs(Arctic_discharge['centerline_distance_across']-Arctic_discharge['distance_across'])
Arctic_discharge.loc[:,'interpolated_thickness']=((10-centerline_depth)/distance1**2)*(distance2**2)+centerline_depth

#figure out where interpolated data will go
# interp_idx = Arctic_discharge['best_thickness'].isnull()&(Arctic_discharge['interpolated_thickness']>0)
#fill with interpolated data where possible
# Arctic_discharge['best_thickness'].fillna(Arctic_discharge['interpolated_thickness'], inplace=True)
#only fill some glaciers that should have u-shaped valley modeling
Arctic_discharge.loc[Arctic_discharge['name']=='RGI60-05.10315_8','best_thickness']=Arctic_discharge['interpolated_thickness']
Arctic_discharge.loc[Arctic_discharge['name']=='RGI60-05.10315_4','best_thickness']=Arctic_discharge['interpolated_thickness']
Arctic_discharge.loc[Arctic_discharge['name']=='RGI60-05.10315_3','best_thickness']=Arctic_discharge['interpolated_thickness']
Arctic_discharge.loc[Arctic_discharge['name']=='RGI60-05.08054','best_thickness']=Arctic_discharge['interpolated_thickness']

In [14]:
#First interpolate 2000 dhdht, then fill it with dhdt_late data
Arctic_discharge.loc[:,'dhdt_early'] = Arctic_discharge['dhdt_early'].interpolate(method='nearest',limit=20)
Arctic_discharge.loc[:,'dhdt_err_early'] = Arctic_discharge['dhdt_err_early'].interpolate(method='nearest',limit=20)
#interpolate dhdt to fill data gaps
Arctic_discharge.loc[:,'dhdt_late'] = Arctic_discharge['dhdt_late'].interpolate(method='nearest',limit=30)
Arctic_discharge.loc[:,'dhdt_err_late'] = Arctic_discharge['dhdt_err_late'].interpolate(method='nearest',limit=30)
#fill gaps with late data
Arctic_discharge.loc[:,'dhdt_early'].fillna(Arctic_discharge['dhdt_late'],inplace=True)
Arctic_discharge.loc[:,'dhdt_err_early'].fillna(Arctic_discharge['dhdt_err_late'],inplace=True)

Arctic_discharge.loc[Arctic_discharge['best_thickness']<Arctic_discharge['thickness_uncertainty'],'thickness_uncertainty']=Arctic_discharge['best_thickness']

i=Arctic_discharge['thickness_date']<2010
Arctic_discharge.loc[i,'best_thickness_early'] = ((2005-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_early'])+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_late'] = (((2010-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_early'])+(5*Arctic_discharge['dhdt_late']))+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_early_err']=np.sqrt(((2005-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_err_early'])**2+(Arctic_discharge['thickness_uncertainty'])**2)
Arctic_discharge.loc[i,'best_thickness_late_err']=np.sqrt((((2010-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_err_early'])**2+(5*Arctic_discharge['dhdt_err_late']))+(Arctic_discharge['thickness_uncertainty'])**2)

i=Arctic_discharge['thickness_date']>=2010
Arctic_discharge.loc[i,'best_thickness_early'] = (-5*Arctic_discharge['dhdt_early'])+(-(Arctic_discharge['thickness_date']-2010)*Arctic_discharge['dhdt_late'])+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_late'] = ((2015-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_late'])+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_early_err']=np.sqrt((-5*Arctic_discharge['dhdt_err_early'])**2+(-(Arctic_discharge['thickness_date']-2010)*Arctic_discharge['dhdt_err_late'])**2+(Arctic_discharge['thickness_uncertainty'])**2)
Arctic_discharge.loc[i,'best_thickness_late_err']=np.sqrt(((2015-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_err_late'])**2+(Arctic_discharge['thickness_uncertainty'])**2)


#add column for data sources
Arctic_discharge['thickness_data_source']=np.nan
Arctic_discharge.loc[(Arctic_discharge['THICK']>0)&np.isnan(Arctic_discharge['thickness_data_source']),'thickness_data_source']=1
print('just OIB',Arctic_discharge['thickness_data_source'].count())
Arctic_discharge.loc[(Arctic_discharge['bed_machine']==Arctic_discharge['best_thickness'])&np.isnan(Arctic_discharge['thickness_data_source']),'thickness_data_source']=2
print('plus bed machine', Arctic_discharge['thickness_data_source'].count())
Arctic_discharge.loc[(Arctic_discharge['Milan_thickness']==Arctic_discharge['best_thickness'])&np.isnan(Arctic_discharge['thickness_data_source']),'thickness_data_source']=3
print('plus Millan', Arctic_discharge['thickness_data_source'].count())
Arctic_discharge['thickness_data_source'].fillna(0,inplace=True)#fill the rest with 5 for Millan

#measured ice thickness corrected for melt
Arctic_discharge['best_measured_ice_thickness_early'] = np.nan
Arctic_discharge['best_measured_ice_thickness_late'] = np.nan
Arctic_discharge.loc[Arctic_discharge['thickness_data_source']==1,'best_measured_ice_thickness_early']=Arctic_discharge['best_thickness_early']
Arctic_discharge.loc[Arctic_discharge['thickness_data_source']==1,'best_measured_ice_thickness_late']=Arctic_discharge['best_thickness_late']
#Millan ice thickness corrected for melt
Arctic_discharge.loc[:,'Milan_ice_thickness_early'] = (-5*Arctic_discharge['dhdt_early'])+(-(2017-2010)*Arctic_discharge['dhdt_late'])+Arctic_discharge['Milan_thickness']
Arctic_discharge.loc[:,'Milan_ice_thickness_late'] = ((2015-2017)*Arctic_discharge['dhdt_late'])+Arctic_discharge['Milan_thickness']
#Farinotti ice thickness corrected for melt
Arctic_discharge.loc[:,'BedMachine_ice_thickness_early'] = ((2005-2007)*Arctic_discharge['dhdt_early'])+Arctic_discharge['bed_machine']
Arctic_discharge.loc[:,'BedMachine_ice_thickness_late'] = (((2010-2007)*Arctic_discharge['dhdt_early'])+(5*Arctic_discharge['dhdt_late']))+Arctic_discharge['bed_machine']

#we dont want negative thicknesses
Arctic_discharge.loc[Arctic_discharge['best_thickness_early']<0,'best_thickness_early']=0
Arctic_discharge.loc[Arctic_discharge['best_thickness_early_err']<0,'best_thickness_early_err']=0
Arctic_discharge.loc[Arctic_discharge['best_thickness_late']<0,'best_thickness_late']=0
Arctic_discharge.loc[Arctic_discharge['best_thickness_late_err']<0,'best_thickness_late_err']=0
Arctic_discharge.loc[Arctic_discharge['Milan_ice_thickness_early']<0,'Milan_ice_thickness_early']=0
Arctic_discharge.loc[Arctic_discharge['Milan_ice_thickness_late']<0,'Milan_ice_thickness_late']=0
Arctic_discharge.loc[Arctic_discharge['BedMachine_ice_thickness_early']<0,'BedMachine_ice_thickness_early']=0
Arctic_discharge.loc[Arctic_discharge['BedMachine_ice_thickness_late']<0,'BedMachine_ice_thickness_late']=0

Arctic_discharge['Difference_between_late_and_early_thickness'] = Arctic_discharge['best_thickness_late']-Arctic_discharge['best_thickness_early']

just OIB 11543
plus bed machine 81219
plus Millan 120979


In [15]:
Arctic_discharge = Arctic_discharge.sort_values(by=['name','distance_across'])

Arctic_discharge['bearing_to_next_point'] = np.nan
Arctic_discharge['bearing_of_velocity_early'] = np.nan
Arctic_discharge['bearing_of_velocity_late'] = np.nan
Arctic_discharge['velocity_offsett_angle_early'] = np.nan
Arctic_discharge['velocity_offsett_angle_late'] = np.nan
Arctic_discharge['claculated_fluxgate_width_early_min'] = np.nan
Arctic_discharge['claculated_fluxgate_width_late_min'] = np.nan
Arctic_discharge['claculated_fluxgate_width_early_max'] = np.nan
Arctic_discharge['claculated_fluxgate_width_late_max'] = np.nan
Arctic_discharge['length_early_min'] = 25
Arctic_discharge['length_late_min'] = 25
Arctic_discharge['length_early_max'] = 25
Arctic_discharge['length_late_max'] = 25

#calculate the angle that flow is hitting the fluxgate
for i in range(0,len(Arctic_discharge)-1):
    if (Arctic_discharge['name'][i]!=Arctic_discharge['name'][i+1]):
        Arctic_discharge.loc[i,'bearing_to_next_point'] = Arctic_discharge['bearing_to_next_point'][i-1]
    elif Arctic_discharge['name'][i]==Arctic_discharge['name'][i+1]:
        hypot=np.sqrt((Arctic_discharge['easting'][i+1]-Arctic_discharge['easting'][i])**2+(Arctic_discharge['northing'][i+1]-Arctic_discharge['northing'][i])**2)
        Arctic_discharge.loc[i,'bearing_to_next_point'] = math.atan2((Arctic_discharge['easting'][i+1]-Arctic_discharge['easting'][i])/hypot,(Arctic_discharge['northing'][i+1]-Arctic_discharge['northing'][i])/hypot)*180/math.pi #% 360
        
    Arctic_discharge.loc[i,'bearing_of_velocity_early'] = math.atan2(Arctic_discharge['2000_2010_mean_x'][i]/Arctic_discharge['2000_2010_mean'][i],Arctic_discharge['2000_2010_mean_y'][i]/Arctic_discharge['2000_2010_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
    Arctic_discharge.loc[i,'bearing_of_velocity_late'] = math.atan2(Arctic_discharge['2010_2018_mean_x'][i]/Arctic_discharge['2010_2018_mean'][i],Arctic_discharge['2010_2018_mean_y'][i]/Arctic_discharge['2010_2018_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
    
    Arctic_discharge.loc[i,'velocity_offsett_angle_early'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_early'][i]) % 360)
    if Arctic_discharge['velocity_offsett_angle_early'][i]>180:
        Arctic_discharge.loc[i,'velocity_offsett_angle_early']=360-Arctic_discharge['velocity_offsett_angle_early'][i]
    if Arctic_discharge['velocity_offsett_angle_early'][i]>90:
        Arctic_discharge.loc[i,'claculated_fluxgate_width_early_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]+10))*25)
        Arctic_discharge.loc[i,'claculated_fluxgate_width_early_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]-10))*25)
    if Arctic_discharge['velocity_offsett_angle_early'][i]<90:
        Arctic_discharge.loc[i,'claculated_fluxgate_width_early_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]-10))*25)
        Arctic_discharge.loc[i,'claculated_fluxgate_width_early_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]+10))*25)
    
    Arctic_discharge.loc[i,'velocity_offsett_angle_late'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_late'][i]) % 360)
    if Arctic_discharge.loc[i,'velocity_offsett_angle_late']>180:
        Arctic_discharge.loc[i,'velocity_offsett_angle_late']=360-Arctic_discharge['velocity_offsett_angle_late'][i]
    if Arctic_discharge['velocity_offsett_angle_late'][i]>90:
        Arctic_discharge.loc[i,'claculated_fluxgate_width_late_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]+10))*25)
        Arctic_discharge.loc[i,'claculated_fluxgate_width_late_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]-10))*25)
    if Arctic_discharge['velocity_offsett_angle_late'][i]<90:
        Arctic_discharge.loc[i,'claculated_fluxgate_width_late_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]-10))*25)
        Arctic_discharge.loc[i,'claculated_fluxgate_width_late_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]+10))*25)

    
    #if you have slow velocities, replace fluxgate with 25 m width
    if (Arctic_discharge['2000_2010_mean'][i] > 25)&(45<Arctic_discharge['velocity_offsett_angle_early'][i]<135):
        Arctic_discharge.loc[i,'length_early_min'] = Arctic_discharge['claculated_fluxgate_width_early_min'][i]
        Arctic_discharge.loc[i,'length_early_max'] = Arctic_discharge['claculated_fluxgate_width_early_max'][i]
    elif (Arctic_discharge['2000_2010_mean'][i] < 25)|(Arctic_discharge['velocity_offsett_angle_early'][i]<45)|(Arctic_discharge['velocity_offsett_angle_early'][i]>135)|(Arctic_discharge['velocity_glacier_edge_mask'][i] == 1):#if the velocity is either slow, high angle, or near the edge, replace with max uncertainty
        Arctic_discharge.loc[i,'length_early_min'] = np.abs(math.sin(math.radians(45))*25)#gate width at 45°
        Arctic_discharge.loc[i,'length_early_max'] = 25        
        
    if (Arctic_discharge['2010_2018_mean'][i] > 25)&(45<Arctic_discharge['velocity_offsett_angle_late'][i]<135):
        Arctic_discharge.loc[i,'length_late_min'] = Arctic_discharge['claculated_fluxgate_width_late_min'][i]
        Arctic_discharge.loc[i,'length_late_max'] = Arctic_discharge['claculated_fluxgate_width_late_max'][i]
    elif (Arctic_discharge['2010_2018_mean'][i] < 25)|(Arctic_discharge['velocity_offsett_angle_late'][i]<45)|(Arctic_discharge['velocity_offsett_angle_late'][i]>135)|(Arctic_discharge['velocity_glacier_edge_mask'][i] == 1):#if the velocity is either slow, high angle, or near the edge, replace with max uncertainty
        Arctic_discharge.loc[i,'length_late_min'] = np.abs(math.sin(math.radians(45))*25)#gate width at 45°
        Arctic_discharge.loc[i,'length_late_max'] = 25

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars


In [16]:
#calculate the flux from discharge
Arctic_discharge.loc[:,'length_early_avg'] = (Arctic_discharge['length_early_min']+Arctic_discharge['length_early_max'])/2
Arctic_discharge.loc[:,'length_late_avg'] = (Arctic_discharge['length_late_min']+Arctic_discharge['length_late_max'])/2
Arctic_discharge.loc[:,'length_early_avg_err'] = (Arctic_discharge['length_early_max']-Arctic_discharge['length_early_min'])/2
Arctic_discharge.loc[:,'length_late_avg_err'] = (Arctic_discharge['length_late_max']-Arctic_discharge['length_late_min'])/2

Arctic_discharge['2000_2010_mean_flux_gt'] = np.nan
Arctic_discharge['2000_2010_mean_flux_err_gt'] = np.nan
Arctic_discharge['2010_2020_mean_flux_gt'] = np.nan
Arctic_discharge['2010_2020_mean_flux_err_gt'] = np.nan

#if thickness is less than 0, reset to 0
Arctic_discharge.loc[Arctic_discharge['best_thickness']<0,'best_thickness'] = 0
# Arctic_discharge.loc[Arctic_discharge['best_thickness_early']<0,'best_thickness_early'] = 0
# Arctic_discharge.loc[Arctic_discharge['best_thickness_late']<0,'best_thickness_late'] = 0


#If the glacier has a average thickness of less than 30 m, make the thickness 30±20 m
stored_rgi_early = []
stored_rgi_late = []
for i in range(0,len(Arctic_discharge)):
    if (Arctic_discharge['best_thickness'][Arctic_discharge['name']==Arctic_discharge['name'][i]].mean()<30)&(Arctic_discharge['best_thickness'][i]<30):
        Arctic_discharge.loc[i,'best_thickness'] = 30
        Arctic_discharge.loc[i,'thickness_uncertainty'] = 20
#         stored_rgi.append(Arctic_discharge['RGIId'][i])

#TURN THIS ON WHEN WE HAVE EARLY AND LATE THICKNESS WORKING
    if (Arctic_discharge['best_thickness_early'][Arctic_discharge['name']==Arctic_discharge['name'][i]].mean()<30)&(Arctic_discharge['best_thickness_early'][i]<30):
        Arctic_discharge.loc[i,'best_thickness_early'] = 30
        Arctic_discharge.loc[i,'thickness_uncertainty'] = 20
        stored_rgi_early.append(Arctic_discharge['name'][i])
    if (Arctic_discharge['best_thickness_late'][Arctic_discharge['name']==Arctic_discharge['name'][i]].mean()<30)&(Arctic_discharge['best_thickness_late'][i]<30):
        Arctic_discharge.loc[i,'best_thickness_late'] = 30
        Arctic_discharge.loc[i,'thickness_uncertainty'] = 20
        stored_rgi_late.append(Arctic_discharge['name'][i])
        print(Arctic_discharge['name'][i],'----', Arctic_discharge['best_thickness'][Arctic_discharge['name']==Arctic_discharge['name'][i]].mean())
print('number of glaciers with thickness less than 30 m early:  ',len(set(stored_rgi_early)))
print('number of glaciers with thickness less than 30 m late:  ',len(set(stored_rgi_late)))

i=Arctic_discharge['best_thickness_early'] > 0
#early discharge
#.                                                                             >depth average velocity adjustment                                                      
Arctic_discharge.loc[i,'2000_2010_mean_flux_gt'] = Arctic_discharge['2000_2010_mean']*vertically_averaged_velocity*Arctic_discharge['length_early_avg']*Arctic_discharge['best_thickness_early']/1000000000*density #0.9 converts to Gt
Arctic_discharge.loc[i,'2000_2010_mean_flux_err_gt'] = np.sqrt((Arctic_discharge['2000_2010_mean_err']/Arctic_discharge['2000_2010_mean'])**2
                                                      +(Arctic_discharge['length_early_avg_err']/Arctic_discharge['length_early_avg'])**2
                                                      +(Arctic_discharge['thickness_uncertainty']/Arctic_discharge['best_thickness_early'])**2)*Arctic_discharge['2000_2010_mean_flux_gt']*density #0.9 converts to Gt
                                                                                                                                            #    ^multiply the percent error by total flux to get the error in km3 a-1   
i=Arctic_discharge['best_thickness_late'] > 0
    #late discharge
#.                                                                             >depth average velocity adjustment                                                      
Arctic_discharge.loc[i,'2010_2020_mean_flux_gt'] = Arctic_discharge['2010_2018_mean']*vertically_averaged_velocity*Arctic_discharge['length_early_avg']*Arctic_discharge['best_thickness_late']/1000000000*density #0.9 converts to Gt
Arctic_discharge.loc[i,'2010_2020_mean_flux_err_gt'] = np.sqrt((Arctic_discharge['2010_2018_mean_err']/Arctic_discharge['2010_2018_mean'])**2
                                                      +(Arctic_discharge['length_late_avg_err']/Arctic_discharge['length_late_avg'])**2
                                                      +(Arctic_discharge['thickness_uncertainty']/Arctic_discharge['best_thickness_late'])**2)*Arctic_discharge['2010_2020_mean_flux_gt']*density #0.9 converts to Gt

i=Arctic_discharge['best_thickness_early'] <= 0
Arctic_discharge.loc[i,'2000_2010_mean_flux_gt'] = 0
Arctic_discharge.loc[i,'2000_2010_mean_flux_err_gt'] = 0
i=Arctic_discharge['best_thickness_late'] <= 0
Arctic_discharge.loc[i,'2010_2020_mean_flux_gt'] = 0
Arctic_discharge.loc[i,'2010_2020_mean_flux_err_gt'] = 0
        
Arctic_discharge.loc[:,'2000_2010_mean_flux'] = Arctic_discharge['2000_2010_mean_flux_gt'].astype(float)
Arctic_discharge.loc[:,'2000_2010_mean_flux_err'] = Arctic_discharge['2000_2010_mean_flux_err_gt'].astype(float)
Arctic_discharge.loc[:,'2010_2020_mean_flux'] = Arctic_discharge['2010_2020_mean_flux_gt'].astype(float)
Arctic_discharge.loc[:,'2010_2020_mean_flux_err'] = Arctic_discharge['2010_2020_mean_flux_err_gt'].astype(float)
Arctic_discharge.loc[:,'thickness_uncertainty'] = Arctic_discharge['thickness_uncertainty'].astype(float)

ILORLIIT-SERMINNGUAQ ---- 1.9
ILORLIIT-SERMINNGUAQ ---- 2.8666666666666667
ILORLIIT-SERMINNGUAQ ---- 3.8333333333333335
ILORLIIT-SERMINNGUAQ ---- 4.8
ILORLIIT-SERMINNGUAQ ---- 5.766666666666667
ILORLIIT-SERMINNGUAQ ---- 6.733333333333333
ILORLIIT-SERMINNGUAQ ---- 7.7
ILORLIIT-SERMINNGUAQ ---- 8.666666666666666
ILORLIIT-SERMINNGUAQ ---- 9.633333333333333
ILORLIIT-SERMINNGUAQ ---- 10.6
ILORLIIT-SERMINNGUAQ ---- 11.566666666666666
ILORLIIT-SERMINNGUAQ ---- 12.533333333333333
ILORLIIT-SERMINNGUAQ ---- 13.5
ILORLIIT-SERMINNGUAQ ---- 14.466666666666667
ILORLIIT-SERMINNGUAQ ---- 15.433333333333334
ILORLIIT-SERMINNGUAQ ---- 16.4
ILORLIIT-SERMINNGUAQ ---- 17.366666666666667
ILORLIIT-SERMINNGUAQ ---- 18.333333333333332
ILORLIIT-SERMINNGUAQ ---- 19.3
ILORLIIT-SERMINNGUAQ ---- 20.266666666666666
ILORLIIT-SERMINNGUAQ ---- 21.233333333333334
ILORLIIT-SERMINNGUAQ ---- 22.2
ILORLIIT-SERMINNGUAQ ---- 23.166666666666668
ILORLIIT-SERMINNGUAQ ---- 24.133333333333333
ILORLIIT-SERMINNGUAQ ---- 25.1
ILORLIIT

QALERALLIT_SERMIAT ---- 8.404514970450565
QALERALLIT_SERMIAT ---- 8.51922318031048
QALERALLIT_SERMIAT ---- 8.633931390170392
QALERALLIT_SERMIAT ---- 8.748639600030307
QALERALLIT_SERMIAT ---- 8.863347809890222
QALERALLIT_SERMIAT ---- 8.978056019750134
QALERALLIT_SERMIAT ---- 9.092764229610049
QALERALLIT_SERMIAT ---- 9.204677713328394
QALERALLIT_SERMIAT ---- 9.320480215138403
QALERALLIT_SERMIAT ---- 9.43628271694841
QALERALLIT_SERMIAT ---- 9.55208521875842
QALERALLIT_SERMIAT ---- 9.667887720568427
QALERALLIT_SERMIAT ---- 9.783690222378436
QALERALLIT_SERMIAT ---- 9.899492724188443
QALERALLIT_SERMIAT ---- 10.01529522599845
QALERALLIT_SERMIAT ---- 10.12801637320683
QALERALLIT_SERMIAT ---- 10.240737520415207
QALERALLIT_SERMIAT ---- 10.357247451256061
QALERALLIT_SERMIAT ---- 10.473757382096915
QALERALLIT_SERMIAT ---- 10.590267312937769
QALERALLIT_SERMIAT ---- 10.706777243778623
QALERALLIT_SERMIAT ---- 10.823287174619477
QALERALLIT_SERMIAT ---- 10.936415836728852
QALERALLIT_SERMIAT ---- 11.049

SERMILIK ---- 24.01010636151847
SERMILIK ---- 24.084390931210276
SERMILIK ---- 24.158675500902078
SERMILIK ---- 24.232960070593883
SERMILIK ---- 24.307244640285685
SERMILIK ---- 24.379187632415253
SERMILIK ---- 24.451130624544824
SERMILIK ---- 24.52307361667439
SERMILIK ---- 24.59501660880396
SERMILIK ---- 24.666959600933527
SERMILIK ---- 24.738902593063095
SERMILIK ---- 24.807763309802038
SERMILIK ---- 24.876624026540984
SERMILIK ---- 24.945484743279927
SERMILIK ---- 25.01434546001887
SERMILIK ---- 25.08689686403436
SERMILIK ---- 25.159448268049854
SERMILIK ---- 25.231999672065346
SERMILIK ---- 25.302147816803497
SERMILIK ---- 25.372295961541646
SERMILIK ---- 25.442444106279794
SERMILIK ---- 25.512592251017942
SERMILIK ---- 25.58274039575609
SERMILIK ---- 25.65288854049424
SERMILIK ---- 25.728451098425914
SERMILIK ---- 25.802299111576403
SERMILIK ---- 25.876147124726895
SERMILIK ---- 25.949995137877384
SERMILIK ---- 26.023843151027872
SERMILIK ---- 26.097691164178364
SERMILIK ---- 26.

LEIDY-MARIE-SERMIARSUPALUK ---- 5.283049889330594
LEIDY-MARIE-SERMIARSUPALUK ---- 5.419842342160782
LEIDY-MARIE-SERMIARSUPALUK ---- 5.556634794990972
LEIDY-MARIE-SERMIARSUPALUK ---- 5.69342724782116
LEIDY-MARIE-SERMIARSUPALUK ---- 5.830219700651349
LEIDY-MARIE-SERMIARSUPALUK ---- 5.967012153481537
LEIDY-MARIE-SERMIARSUPALUK ---- 6.103804606311726
LEIDY-MARIE-SERMIARSUPALUK ---- 6.240597059141915
LEIDY-MARIE-SERMIARSUPALUK ---- 6.377389511972104
LEIDY-MARIE-SERMIARSUPALUK ---- 6.514181964802292
LEIDY-MARIE-SERMIARSUPALUK ---- 6.650974417632481
LEIDY-MARIE-SERMIARSUPALUK ---- 6.792483419742224
LEIDY-MARIE-SERMIARSUPALUK ---- 6.9339924218519675
LEIDY-MARIE-SERMIARSUPALUK ---- 7.075501423961711
LEIDY-MARIE-SERMIARSUPALUK ---- 7.217010426071455
LEIDY-MARIE-SERMIARSUPALUK ---- 7.358519428181198
LEIDY-MARIE-SERMIARSUPALUK ---- 7.500028430290942
LEIDY-MARIE-SERMIARSUPALUK ---- 7.6415374324006855
LEIDY-MARIE-SERMIARSUPALUK ---- 7.783046326547299
LEIDY-MARIE-SERMIARSUPALUK ---- 7.924555220693912

LEIDY-MARIE-SERMIARSUPALUK ---- 28.05188679245283
LEIDY-MARIE-SERMIARSUPALUK ---- 28.18867924528302
LEIDY-MARIE-SERMIARSUPALUK ---- 28.32547169811321
LEIDY-MARIE-SERMIARSUPALUK ---- 28.462264150943398
LEIDY-MARIE-SERMIARSUPALUK ---- 28.599056603773583
LEIDY-MARIE-SERMIARSUPALUK ---- 28.735849056603772
LEIDY-MARIE-SERMIARSUPALUK ---- 28.87264150943396
LEIDY-MARIE-SERMIARSUPALUK ---- 29.00943396226415
LEIDY-MARIE-SERMIARSUPALUK ---- 29.150943396226417
LEIDY-MARIE-SERMIARSUPALUK ---- 29.29245283018868
LEIDY-MARIE-SERMIARSUPALUK ---- 29.433962264150942
LEIDY-MARIE-SERMIARSUPALUK ---- 29.57547169811321
LEIDY-MARIE-SERMIARSUPALUK ---- 29.71698113207547
LEIDY-MARIE-SERMIARSUPALUK ---- 29.858490566037737
LEIDY-MARIE-SERMIARSUPALUK ---- 30.0
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675327
SHARP ---- 38.68373795675

...

RGI60-05.04906 ---- 27.76273148148148
RGI60-05.04906 ---- 28.00173611111111
RGI60-05.04906 ---- 28.274016203703702
RGI60-05.04906 ---- 28.519675925925927
RGI60-05.04906 ---- 29.07349537037037
RGI60-05.04906 ---- 29.627314814814813
RGI60-05.04906 ---- 30.453993055555557
RGI60-05.04584 ---- 19.2415625
RGI60-05.04584 ---- 19.6365625
RGI60-05.04584 ---- 20.0315625
RGI60-05.04584 ---- 20.6315625
RGI60-05.04584 ---- 21.2315625
RGI60-05.04584 ---- 21.8315625
RGI60-05.04584 ---- 22.4315625
RGI60-05.04584 ---- 23.0315625
RGI60-05.04584 ---- 23.6315625
RGI60-05.04584 ---- 24.2315625
RGI60-05.04584 ---- 24.8315625
RGI60-05.04584 ---- 25.4315625
RGI60-05.04584 ---- 26.0315625
RGI60-05.04584 ---- 26.6115625
RGI60-05.04584 ---- 27.1915625
RGI60-05.04584 ---- 27.7715625
RGI60-05.04584 ---- 28.3515625
RGI60-05.04584 ---- 28.9315625
RGI60-05.04584 ---- 29.5115625
RGI60-05.04584 ---- 30.1115625
RGI60-05.04584 ---- 30.1115625
RGI60-05.04526 ---- 29.036658653846153
RGI60-05.04526 ---- 29.528245192307693
R

RGI60-05.10738 ---- 21.323863636363637
RGI60-05.10738 ---- 21.808948863636363
RGI60-05.10738 ---- 22.29403409090909
RGI60-05.10738 ---- 22.44637784090909
RGI60-05.10738 ---- 22.59872159090909
RGI60-05.10738 ---- 22.93252840909091
RGI60-05.10738 ---- 23.266335227272727
RGI60-05.10738 ---- 23.737926136363637
RGI60-05.10738 ---- 24.3125
RGI60-05.10738 ---- 24.894176136363637
RGI60-05.10738 ---- 25.475852272727273
RGI60-05.10738 ---- 26.05752840909091
RGI60-05.10738 ---- 26.601207386363637
RGI60-05.10738 ---- 26.936789772727273
RGI60-05.10738 ---- 27.593039772727273
RGI60-05.10738 ---- 28.249289772727273
RGI60-05.10738 ---- 29.124644886363637
RGI60-05.10738 ---- 30.0
RGI60-05.10629 ---- 2.16
RGI60-05.10629 ---- 3.32
RGI60-05.10629 ---- 4.48
RGI60-05.10629 ---- 5.64
RGI60-05.10629 ---- 6.8
RGI60-05.10629 ---- 7.96
RGI60-05.10629 ---- 9.12
RGI60-05.10629 ---- 10.28
RGI60-05.10629 ---- 11.44
RGI60-05.10629 ---- 12.6
RGI60-05.10629 ---- 13.76
RGI60-05.10629 ---- 14.92
RGI60-05.10629 ---- 16.08

RGI60-05.10938 ---- 23.70967741935484
RGI60-05.10938 ---- 24.193548387096776
RGI60-05.10938 ---- 24.677419354838708
RGI60-05.10938 ---- 25.161290322580644
RGI60-05.10938 ---- 25.64516129032258
RGI60-05.10938 ---- 26.129032258064516
RGI60-05.10938 ---- 26.612903225806452
RGI60-05.10938 ---- 27.096774193548388
RGI60-05.10938 ---- 27.580645161290324
RGI60-05.10938 ---- 28.06451612903226
RGI60-05.10938 ---- 28.548387096774192
RGI60-05.10938 ---- 29.032258064516128
RGI60-05.10938 ---- 29.516129032258064
RGI60-05.10938 ---- 30.0
RGI60-05.10945 ---- 0.45454545454545453
RGI60-05.10945 ---- 0.9090909090909091
RGI60-05.10945 ---- 1.3636363636363635
RGI60-05.10945 ---- 1.8181818181818181
RGI60-05.10945 ---- 2.272727272727273
RGI60-05.10945 ---- 2.727272727272727
RGI60-05.10945 ---- 3.1818181818181817
RGI60-05.10945 ---- 3.6363636363636362
RGI60-05.10945 ---- 4.090909090909091
RGI60-05.10945 ---- 4.545454545454546
RGI60-05.10945 ---- 5.0
RGI60-05.10945 ---- 5.454545454545454
RGI60-05.10945 ---- 5.

RGI60-05.12860 ---- 17.731848822699654
RGI60-05.12860 ---- 17.955509609646267
RGI60-05.12860 ---- 18.17917039659288
RGI60-05.12860 ---- 18.402831183539497
RGI60-05.12860 ---- 18.64563857184516
RGI60-05.12860 ---- 18.888445960150825
RGI60-05.12860 ---- 19.152674780951607
RGI60-05.12860 ---- 19.416903601752388
RGI60-05.12860 ---- 19.68113242255317
RGI60-05.12860 ---- 19.94536124335395
RGI60-05.12860 ---- 20.20959006415473
RGI60-05.12860 ---- 20.473818884955513
RGI60-05.12860 ---- 20.738603803846573
RGI60-05.12860 ---- 21.00338872273763
RGI60-05.12860 ---- 21.26817364162869
RGI60-05.12860 ---- 21.53295856051975
RGI60-05.12860 ---- 21.79774347941081
RGI60-05.12860 ---- 22.062528398301865
RGI60-05.12860 ---- 22.325160132514107
RGI60-05.12860 ---- 22.587791866726345
RGI60-05.12860 ---- 22.850423600938583
RGI60-05.12860 ---- 23.113055335150825
RGI60-05.12860 ---- 23.375687069363064
RGI60-05.12860 ---- 23.638318803575302
RGI60-05.12860 ---- 23.900950537787544
RGI60-05.12860 ---- 24.18480724758

RGI60-05.06401 ---- 21.856370192307693
RGI60-05.06401 ---- 22.517427884615383
RGI60-05.06401 ---- 23.981971153846153
RGI60-05.06401 ---- 25.446514423076923
RGI60-05.06401 ---- 27.75420673076923
RGI60-05.06401 ---- 30.06189903846154
RGI60-05.12738 ---- 1.4047719594594594
RGI60-05.12738 ---- 2.197635135135135
RGI60-05.12738 ---- 2.990498310810811
RGI60-05.12738 ---- 3.78125
RGI60-05.12738 ---- 4.5720016891891895
RGI60-05.12738 ---- 5.357263513513513
RGI60-05.12738 ---- 6.142525337837838
RGI60-05.12738 ---- 6.927787162162162
RGI60-05.12738 ---- 7.711570945945946
RGI60-05.12738 ---- 8.49535472972973
RGI60-05.12738 ---- 9.279138513513514
RGI60-05.12738 ---- 10.062922297297296
RGI60-05.12738 ---- 10.84670608108108
RGI60-05.12738 ---- 11.630489864864865
RGI60-05.12738 ---- 12.41427364864865
RGI60-05.12738 ---- 13.198057432432432
RGI60-05.12738 ---- 13.981841216216216
RGI60-05.12738 ---- 14.765625
RGI60-05.12738 ---- 15.549408783783784
RGI60-05.12738 ---- 16.33319256756757
RGI60-05.12738 ---- 

In [17]:
print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
print(Arctic_discharge['2010_2020_mean_flux_err_gt'].sum())

441.2801998510875
43.15989722146002


In [18]:
# write out file for observations for every point
Arctic_discharge.to_file('Greenland/Greenland_discharge_points_epsg3995_'+date_string+'.gpkg', driver="GPKG")#change file name based on distance

In [16]:
Arctic_discharge = gpd.read_file('Greenland/Greenland_discharge_points_epsg3995_'+date_string+'.gpkg', driver="GPKG")#change file name based on distance

In [78]:
print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
print(Arctic_discharge['2010_2020_mean_flux_err_gt'].sum())

449.6154925149414
44.80578125266209


In [19]:
#differentiate between ice sheet and mountain glacier
Arctic_discharge['Glacier_type']=np.nan
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['name'][i][0:3]=='RGI':
        Arctic_discharge.loc[i,'Glacier_type']='mountain'
    else:
        Arctic_discharge.loc[i,'Glacier_type']='ice sheet'


In [20]:
#ERROR ANALYSIS from ROMAIN
# GeoPandas load 10GB+ in memory through fiona, for some unknown reason, converting to CSV and reading with pandas

# To convert the shapefile in command-line
# ogr2ogr -f CSV Will_frontal_ablation.csv Arctic_discharge_points_epsg3995_Spring2021_v9.gpkg
# Arctic_discharge = gpd.read_file('/home/atom/ongoing/work_frontal_ablation/Arctic_discharge_points_epsg3995_Fall2021_v10.gpkg', driver="GPKG")

# df = pd.read_csv('/home/atom/ongoing/work_frontal_ablation/Will_frontal_ablation.csv')

for section in [0,1,2]:
    if section == 0:
        print('-----------------------All Greenland-----------------------')
        #If you want all Greenland together
        Arctic_discharge['region']='Greenland'
        regionsornot='all_Greenland_'

    if section == 1:
        print('-----------------------Greenland ice sheet and periphery-----------------------')
        #If you want periphery and ice sheet
        Arctic_discharge['region']=Arctic_discharge['Glacier_type']
        regionsornot='seperated_Greenland_'

    if section == 2:
        print('-----------------------Greenland regions-----------------------')
        #if you want regions of Greenland (NE, SW, etc.)
        greenland_aois = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Greenland_regions.gpkg')
        Arctic_discharge = gpd.sjoin(Arctic_discharge,greenland_aois)
        Arctic_discharge = Arctic_discharge.drop(columns='index_right')
        regionsornot='regional_Greenland_'
        Arctic_discharge['region']=Arctic_discharge['Greenland_region_name']


    # Choose "early" or "late"
    for period in ['early','late']:
        if period == 'early':
            period_date = '2000_2010'
        else:
            period_date = '2010_2018'

        print('Running for period: '+period)

        # 0/ Perform the bias correction for ice thickness

        print('Correcting ice thickness bias per ice thickness range...')
        for model in ['BedMachine_','Milan_']:
            if model == 'Milan_':
                print(model,'----------------------------')
                best_meas_icethickness = Arctic_discharge['best_measured_ice_thickness_'+period].values
                best_modelled_icethickness = Arctic_discharge[model+'ice_thickness_'+period].values
                it_diff = best_meas_icethickness - best_modelled_icethickness

                print('Median bias before:' +str(np.nanmean(it_diff)))

                df_bin = xdem.spatialstats.nd_binning(it_diff, list_var=[best_modelled_icethickness], list_var_names=['ice_thickness'], list_var_bins=18)
                y = df_bin.nanmedian.values
                x = pd.IntervalIndex(df_bin.ice_thickness).mid.values
                p = np.polyfit(x, y, deg=1)

                # Plot (optional)
                # import matplotlib.pyplot as plt
                # x0 = np.linspace(-50, 850)
                # xdem.spatialstats.plot_1d_binning(df_bin, var_name='ice_thickness', statistic_name='nanmedian',
                #                                   label_var='Modelled ice thickness (m)', label_statistic='Bias with measured ice thickness (m)')
                # ax = plt.gca()
                # ax.plot(x0, np.polyval(p, x0), color='tab:gray', linestyle='dashed', label='Linear fit')
                # ax.legend()

                # Derive the debiased modelled ice thickness
                Arctic_discharge[model+'ice_thickness_'+period+'_debiased'] = Arctic_discharge[model+'ice_thickness_'+period] + np.polyval(p, Arctic_discharge[model+'ice_thickness_'+period])

                print('Median bias after:' +str(np.nanmean(best_meas_icethickness - Arctic_discharge[model+'ice_thickness_'+period+'_debiased'])))

                # Replace the Milan ice thickness by their debiased version, 3 is Milan data
                Arctic_discharge.loc[Arctic_discharge['thickness_data_source']==3, 'best_thickness_'+period] = Arctic_discharge[model+'ice_thickness_'+period+'_debiased']

                #make sure that velocity and thickness errors are not bigger than the signal
                Arctic_discharge.loc[Arctic_discharge['best_thickness_'+period+'_err']>Arctic_discharge['best_thickness_'+period], 'best_thickness_'+period+'_err'] = Arctic_discharge['best_thickness_'+period]
                Arctic_discharge.loc[Arctic_discharge[period_date+'_mean']<Arctic_discharge[period_date+'_mean_err'], period_date+'_mean_err'] = Arctic_discharge[period_date+'_mean']

            else:
                print('skipping ',model)
        # 1/ Propagation flux gate -> glacier

        # Get variables of interest: velocities, ice thickness, flux gate segment widths
        Hn = Arctic_discharge['best_thickness_'+period].values 
        Hn_err = Arctic_discharge['best_thickness_'+period+'_err'].values
        vn = Arctic_discharge[period_date+'_mean'].values * vertically_averaged_velocity # vertically averaged velocity
        vn_err = Arctic_discharge[period_date+'_mean_err'].values * vertically_averaged_velocity # same for the uncertainty
        d= Arctic_discharge['length_'+period+'_avg'].values
        d_err = Arctic_discharge['length_'+period+'_avg_err'].values

        # Define the NMAD as a robust measure of dispersion
        def nmad(data: np.ndarray, nfact: float = 1.4826) -> float:
            """
            Calculate the normalized median absolute deviation (NMAD) of an array.

            :param data: input data
            :param nfact: normalization factor for the data; default is 1.4826

            :returns nmad: (normalized) median absolute deviation of data.
            """
            data_arr = np.asarray(data)
            return nfact * np.nanmedian(np.abs(data_arr - np.nanmedian(data_arr)))

        # Remove aberrantly large uncertainty values for velocity errors
        vn_err[np.abs(vn_err - np.nanmedian(vn_err)) > 5 * nmad(vn_err)] = np.nan
        # Replace no data that might exist in the uncertainty from the average uncertainty
        vn_err[~np.isfinite(vn_err)] = np.nanmean(vn_err)

        # Correct velocity error given the spread estimated from the yearly comparison to Sentinel-1 and Greenland velocities
        # (theoretical error: 6 m a-1; empirical: 16 m a-1)
    #     vn_err *= 16./6.

        # Remove aberrantly large uncertainty values for ice thickness errors
        Hn_err[np.abs(Hn_err - np.nanmedian(Hn_err)) > 5 * nmad(Hn_err)] = np.nan
        # Replace no data that might exist in the uncertainty from the average uncertainty
        Hn_err[~np.isfinite(Hn_err)] = np.nanmean(Hn_err)
        # After bias correction of the Milan model, ice thickness error have the same spread as yielded by the comparison
        # to GlaThiDa and other measurements
        # (theoretical error: 58 m a-1; empirical: 60 m a-1)
        Hn_err *= 1.

        # Calculate ice volume discharge
        Vn = Hn * vn * d * density / 10**9
        print(period,'__',np.nansum(Vn))

        Vn_err = Hn * vn * d * np.sqrt(Hn_err**2/Hn**2 + vn_err**2/vn**2 + d_err**2/d**2) * density / 10**9

        # Remove aberrantly large uncertainty values for volume discharge uncertainty
        Vn_err[np.abs(Vn_err - np.nanmedian(Vn_err)) > 5 * nmad(Vn_err)] = np.nan
        # Replace no data that might exist in the uncertainty from the average uncertainty
        Vn_err[~np.isfinite(Vn_err)] = np.nanmean(Vn_err)

        # (ALL GOOD) Check that calculations match Will's for ice discharge
        # Product of the three variables: volume discharge at the flux gate
        # Vn_check = df['2000_2010_mean_flux'].values
        # Vn_err_check = df['2000_2010_mean_flux_err'].values
        # ratio_Vn = Vn_check/Vn
        # ratio_Vn_err = Vn_err_check/Vn_err

        # Parameters estimated for ghe variograms (nb_model x (correlation range, partial sill)) for velocity and ice thickness:
        # ranges in meters, partial sills are in percentage of the variance
        params_velocity =((691 ,63), (18000, 26),
                          (1040000, 11))
        params_icethickness = ((2100, 28), (47000, 50),
                               (1354000, 22))

        def sum_vgm(h, params, error1, error2):
            """Sum of variograms models"""
            v = 0
            for k in range(len(params)):
                v += skgstat.models.spherical(h, params[k][0], params[k][1] * error1 * error2)

            return v

        def product_vgms(h, mean_vn_err, mean_Hn_err, mean_d, mean_vn, mean_Hn):
            """Quadratic product of independent velocity and ice thickness variograms"""

            v = (sum_vgm(h, params_icethickness, mean_Hn_err, mean_Hn_err) * mean_d ** 2 * mean_vn ** 2 + \
                 sum_vgm(h, params_velocity, mean_vn_err, mean_vn_err) * mean_d ** 2 * mean_Hn ** 2) /\
                ((mean_Hn_err**2 * mean_vn**2 * mean_d**2 + mean_vn_err**2 * mean_Hn**2 * mean_d**2) * 100)
            return v

        # Function to integrate the variogram from a spatial support to another: a double sum of covariances
        def double_sum_covar(coords: np.ndarray, areas: np.ndarray, errors: np.ndarray,
                             vgm_func: Callable):
            """
            Double sum of covariance for euclidean coordinates
            :param coords: Spatial support (typically, pixel) coordinates
            :param areas:  Area of supports
            :param errors: Standard errors of supports
            :param vgm_func: Variogram function
            :return:
            """

            n = len(coords)
            pds = pdist(coords)
            var = 0
            for i in range(n):
                for j in range(n):

                    # For index calculation of the pairwise distance, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
                    if i == j:
                        d = 0
                    elif i < j:
                        ind = n * i + j - ((i + 2) * (i + 1)) // 2
                        d = pds[ind]
                    else:
                        ind = n * j + i - ((j + 2) * (j + 1)) // 2
                        d = pds[ind]

                    var += errors[i] * errors[j] * (1 - vgm_func(d)) * areas[i] * areas[j]

            total_area = sum(areas)
            se_dsc = np.sqrt(var / total_area ** 2)

            return se_dsc

        # Looping for every glacier
        unique_rgiid = list(np.unique(Arctic_discharge['name'].values))

        list_df = []
        print('working on individual glaciers')
        for rgiid in unique_rgiid:

            # Select only flux gates of a given RGIId = glacier
            index_glacier = (Arctic_discharge['name'] == rgiid)

    #         print('Working on glacier: '+rgiid + ' with '+ str(np.count_nonzero(index_glacier))+ ' flux gates...')

            # Variogram function: derive an average correlation of volume discharge using the mean ice thickness and velocity of
            # the given glacier
            mean_vn = np.nanmean(vn[index_glacier])
            mean_Hn = np.nanmean(Hn[index_glacier])
            mean_d = np.nanmean(d[index_glacier])

            mean_vn_err = np.nanmean(vn_err[index_glacier])
            mean_Hn_err = np.nanmean(Hn_err[index_glacier])
            mean_d_err = np.nanmean(d_err[index_glacier])

            Vn_err_gla = Vn_err[index_glacier]
            Vn_gla = Vn[index_glacier]

            def vgm_func_gla(h):
                v = product_vgms(h, mean_vn_err, mean_Hn_err, mean_d, mean_vn, mean_Hn)
                return v

            # Sum of volume discharges
            V_tot = np.nansum(Vn_gla)

            # Uncertainty on the sum, considering everything independent
            V_tot_err_uncorr = np.sqrt(np.nansum(Vn_err_gla**2))

            # Uncertainty on the sum, considering everything correlated
            V_tot_err_fullycorr = np.nansum(Vn_err_gla)

            # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
            coords = np.array((Arctic_discharge['easting'].values[index_glacier], Arctic_discharge['northing'].values[index_glacier])).T
            V_tot_err_corr = double_sum_covar(coords=coords, areas=np.ones(len(coords)), errors=Vn_err_gla,
                                         vgm_func=vgm_func_gla) * len(coords)

            # Store results in a dataframe
            df_res = pd.DataFrame()
            df_res = df_res.assign(region = [Arctic_discharge['region'].values[index_glacier][0]] ,rgiid = [rgiid], V_tot = [V_tot],
                                   V_tot_err_uncorr = [V_tot_err_uncorr], V_tot_err_corr = [V_tot_err_corr],
                                   V_tot_err_fullycorr = [V_tot_err_fullycorr],
                                   mean_easting = [np.nanmean(Arctic_discharge['easting'].values[index_glacier])],
                                   mean_northing = [np.nanmean(Arctic_discharge['northing'].values[index_glacier])],
                                   nb_flux_gates = [np.count_nonzero(index_glacier)],
                                   mean_vn = [mean_vn], mean_Hn = [mean_Hn], mean_d = [mean_d],
                                   mean_vn_err = [mean_vn_err], mean_Hn_err = [mean_Hn_err], mean_d_err = [mean_d_err])
            list_df.append(df_res)

        # Concatenate all results
        if period=='early':
            df_all_early = pd.concat(list_df)
            list_regions = list(np.unique(df_all_early.region.values))
            df_all_early.to_pickle('df_all_early_'+date_string+'.pkl')
        if period=='late':
            df_all_late = pd.concat(list_df)
            list_regions = list(np.unique(df_all_late.region.values))
            df_all_late.to_pickle('df_all_late_'+date_string+'.pkl')

        # df_all.to_csv('/home/atom/ongoing/work_frontal_ablation/df_perglacier_final_'+period+'.csv')

        # 2/ Propagation glacier -> region



        list_df_reg = []
        for region in list_regions:

            print('Working on region: '+region)

            # Isolate region of interest
            if period=='early':
                df_reg = df_all_early[df_all_early.region == region]
            if period=='late':
                df_reg = df_all_late[df_all_late.region == region]
            # Sum volume discharges
            V_tot_reg = np.sum(df_reg.V_tot.values)

            # Uncertainty on the sum, considering everything independent
            V_tot_err_uncorr_reg = np.sqrt(np.nansum(df_reg.V_tot_err_uncorr.values ** 2))

            # Uncertainty on the sum, considering everything correlated
            V_tot_err_fullycorr_reg = np.nansum(df_reg.V_tot_err_fullycorr)

            # Get the regional weighted mean of velocities, thicknesses, flux gate width, and related uncertainties
            mean_vn_reg = np.nansum(df_reg.mean_vn.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
            mean_Hn_reg = np.nansum(df_reg.mean_Hn.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
            mean_d_reg = np.nansum(df_reg.mean_d.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
            mean_vn_err_reg = np.nansum(df_reg.mean_vn_err.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
            mean_Hn_err_reg = np.nansum(df_reg.mean_Hn.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
            mean_d_err_reg = np.nansum(df_reg.mean_d_err.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)

            # Define the average ice discharge correlation function in the region accordingly
            def vgm_func_gla(h):
                v = product_vgms(h, mean_vn_err_reg, mean_Hn_err_reg, mean_d_reg, mean_vn_reg, mean_Hn_reg)
                return v

            # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
            coords = np.array((df_reg['mean_easting'].values, df_reg['mean_northing'].values)).T
            V_tot_err_corr_reg = double_sum_covar(coords=coords, areas=np.ones(len(coords)), errors=df_reg.V_tot_err_corr.values,
                                              vgm_func=vgm_func_gla) * len(coords)

            # Store results in a dataframe
            df_res_reg = pd.DataFrame()
            df_res_reg = df_res_reg.assign(region=[region], V_tot=[V_tot_reg],
                                   V_tot_err_uncorr=[V_tot_err_uncorr_reg], V_tot_err_corr=[V_tot_err_corr_reg],
                                   V_tot_err_fullycorr=[V_tot_err_fullycorr_reg],
                                   nb_flux_gates=[np.sum(df_reg.nb_flux_gates.values)],
                                   mean_vn=[mean_vn_reg], mean_Hn=[mean_Hn_reg], mean_d=[mean_d_reg],
                                   mean_vn_err=[mean_vn_err_reg], mean_Hn_err=[mean_Hn_err_reg], mean_d_err=[mean_d_err_reg])
            list_df_reg.append(df_res_reg)

        df_all_reg = pd.concat(list_df_reg)
        if period=='early':
            df_all_reg.to_pickle('df_all_reg_early_GRE_'+regionsornot+date_string+'.pkl')
        if period=='late':
            df_all_reg.to_pickle('df_all_reg_late_GRE_'+regionsornot+date_string+'.pkl')
    print('done')

-----------------------All Greenland-----------------------
Running for period: early
Correcting ice thickness bias per ice thickness range...
skipping  BedMachine_
Milan_ ----------------------------
Median bias before:-105.1649940741057
Median bias after:-10.628409660056457
early __ 407.1369480394679
working on individual glaciers


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


Working on region: Greenland
Running for period: late
Correcting ice thickness bias per ice thickness range...
skipping  BedMachine_
Milan_ ----------------------------
Median bias before:-105.157552588064
Median bias after:-28.95018680590379
late __ 436.6690341609872
working on individual glaciers


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


Working on region: Greenland
done
-----------------------Greenland ice sheet and periphery-----------------------
Running for period: early
Correcting ice thickness bias per ice thickness range...
skipping  BedMachine_
Milan_ ----------------------------
Median bias before:-105.1649940741057
Median bias after:-10.628409660056457
early __ 407.1369480394679
working on individual glaciers


...

Working on region: ice sheet
Working on region: mountain
done
-----------------------Greenland regions-----------------------
Running for period: early
Correcting ice thickness bias per ice thickness range...
skipping  BedMachine_
Milan_ ----------------------------
Median bias before:-105.1649940741057
Median bias after:-10.628409660056453
early __ 407.1369480394678
working on individual glaciers


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


Working on region: E
Working on region: N
Working on region: SE
Working on region: SW
Working on region: W
Running for period: late
Correcting ice thickness bias per ice thickness range...
skipping  BedMachine_
Milan_ ----------------------------
Median bias before:-105.157552588064
Median bias after:-28.950186805903783
late __ 436.66903416098717


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


working on individual glaciers
Working on region: E
Working on region: N
Working on region: SE
Working on region: SW
Working on region: W
done


In [24]:
pwd

'/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge'

In [21]:
#make it a pandas data frame so we can groupby
Arctic_discharge_df=pd.DataFrame(Arctic_discharge)
print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
#calculate stats
Arctic_discharge_df_sum = Arctic_discharge_df.groupby(['name']).sum().reset_index()
Arctic_discharge_df_min = Arctic_discharge_df.groupby(['name']).min().reset_index()
Arctic_discharge_df_max = Arctic_discharge_df.groupby(['name']).max().reset_index()
Arctic_discharge_df_mean = Arctic_discharge_df.groupby(['name']).mean().reset_index()
Arctic_discharge_df_count = Arctic_discharge_df.groupby(['name']).count().reset_index()
print(Arctic_discharge_df_sum['2010_2020_mean_flux_gt'].sum())

# Clean the flux data
Arctic_discharge_fluxes_sum = Arctic_discharge_df_sum[['name','2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt', '2010_2020_mean_flux_gt', '2010_2020_mean_flux_err_gt']]
Arctic_discharge_fluxes_min = Arctic_discharge_df_min[['name','bed_machine']]
Arctic_discharge_fluxes_max = Arctic_discharge_df_max[['name','bed_machine','do_we_have_an_observation_in_middle_fifth']]
Arctic_discharge_fluxes_mean = Arctic_discharge_df_mean[['name','easting','northing','bed_machine','bed_machine_err','best_thickness','best_thickness_early','best_thickness_late','thickness_uncertainty','length_early_avg','length_early_avg_err','length_late_avg','length_late_avg_err','2000_2010_mean','2000_2010_mean_err','2010_2018_mean','2010_2018_mean_err']]
Arctic_discharge_fluxes_count = Arctic_discharge_df_count[['name','THICK','northing']]
Arctic_discharge_fluxes_count = Arctic_discharge_fluxes_count.rename(columns={"northing": "num_pts"})
Arctic_discharge_fluxes_count.loc[:,'gate_length_km']=Arctic_discharge_fluxes_count['num_pts']*25/1000 #25 is distance between points you specificed above
data_frames_for_merge = [Arctic_discharge_fluxes_sum, Arctic_discharge_fluxes_min, Arctic_discharge_fluxes_max, Arctic_discharge_fluxes_mean, Arctic_discharge_fluxes_count]

Arctic_discharge_fluxes = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
                                            how='outer'), data_frames_for_merge)
Arctic_discharge_fluxes = Arctic_discharge_fluxes.rename(columns={"bed_machine_x": "bed_machine_min",'bed_machine_y':'bed_machine_max','bed_machine':'bed_machine_mean','THICK':'GlaThiDa_observations','best_thickness_early':'best_thickness_early_mean','best_thickness_late':'best_thickness_late_mean','thickness_uncertainty':'thickness_mean_uncertainty'})  

Arctic_discharge_fluxes = gpd.GeoDataFrame(Arctic_discharge_fluxes, geometry=gpd.points_from_xy(Arctic_discharge_fluxes.easting, Arctic_discharge_fluxes.northing),crs='EPSG:3995')

df_all_early=df_all_early.sort_values('rgiid').reset_index(drop=True)
df_all_late=df_all_late.sort_values('rgiid').reset_index(drop=True)
Arctic_discharge_fluxes=Arctic_discharge_fluxes.sort_values('name')

print(Arctic_discharge_fluxes['2010_2020_mean_flux_gt'].sum())

Arctic_discharge_fluxes.loc[:,'2000_2010_mean_flux_gt']=df_all_early['V_tot']
# Arctic_discharge_fluxes['2000_2010_mean_flux_gt_V_tot']=np.nan
# Arctic_discharge_fluxes.loc[:,'2000_2010_mean_flux_gt_V_tot']=df_all_early['V_tot']
Arctic_discharge_fluxes.loc[:,'2000_2010_mean_flux_err_gt']=df_all_early['V_tot_err_corr']
Arctic_discharge_fluxes.loc[:,'2000_2010_mean']=df_all_early['mean_vn']
Arctic_discharge_fluxes.loc[:,'2000_2010_mean_err']=df_all_early['mean_vn_err']
Arctic_discharge_fluxes.loc[:,'best_thickness_early']=df_all_early['mean_Hn']
Arctic_discharge_fluxes.loc[:,'best_thickness_early_err']=df_all_early['mean_Hn_err']
Arctic_discharge_fluxes.loc[:,'length_early_avg']=df_all_early['mean_d']
Arctic_discharge_fluxes.loc[:,'length_early_avg_err']=df_all_early['mean_d_err']
print(Arctic_discharge_fluxes['2010_2020_mean_flux_gt'].sum())

#do it for late
Arctic_discharge_fluxes.loc[:,'2010_2020_mean_flux_gt']=df_all_late['V_tot']
Arctic_discharge_fluxes.loc[:,'2010_2020_mean_flux_err_gt']=df_all_late['V_tot_err_corr']
Arctic_discharge_fluxes.loc[:,'2010_2018_mean']=df_all_late['mean_vn']
Arctic_discharge_fluxes.loc[:,'2010_2018_mean_err']=df_all_late['mean_vn_err']
Arctic_discharge_fluxes.loc[:,'best_thickness_late']=df_all_late['mean_Hn']
Arctic_discharge_fluxes.loc[:,'best_thickness_late_err']=df_all_late['mean_Hn_err']
Arctic_discharge_fluxes.loc[:,'length_late_avg']=df_all_late['mean_d']
Arctic_discharge_fluxes.loc[:,'length_late_avg_err']=df_all_late['mean_d_err']
print(Arctic_discharge_fluxes['2010_2020_mean_flux_gt'].sum())

Arctic_discharge_fluxes.loc[Arctic_discharge_fluxes['best_thickness_early']<0,'best_thickness_early']=0
Arctic_discharge_fluxes.loc[Arctic_discharge_fluxes['best_thickness_early_err']<0,'best_thickness_early_err']=0
Arctic_discharge_fluxes.loc[Arctic_discharge_fluxes['best_thickness_late']<0,'best_thickness_late']=0
Arctic_discharge_fluxes.loc[Arctic_discharge_fluxes['best_thickness_late_err']<0,'best_thickness_late_err']=0
Arctic_discharge_fluxes.to_file('Greenland/Greenland_discharge_fluxes'+date_string+'.gpkg', driver="GPKG")


441.28019985108756


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  import sys


441.28019985108756
441.28019985108756
441.28019985108756
436.66903416098717


In [22]:
Arctic_discharge_fluxes['best_thickness_late'][Arctic_discharge_fluxes['best_thickness_late']<0]

Series([], Name: best_thickness_late, dtype: float64)

In [23]:
#calculate the area change of every glacier from 2000 to 2020
#read in data files
termini_2000_periperhy = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2000.geojson')
termini_2010_periperhy = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2010.geojson')
termini_2020_periperhy = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2020.geojson')

termini_2000_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2000_outlines.gpkg')
termini_2010_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2010_outlines.gpkg')
termini_2020_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2020_outlines.gpkg')

termini_2000 = termini_2000_periperhy.append(termini_2000_ice_sheet)
termini_2010 = termini_2010_periperhy.append(termini_2010_ice_sheet)
termini_2020 = termini_2020_periperhy.append(termini_2020_ice_sheet)

termini_2000=termini_2000[termini_2000.geometry.type=='Polygon']
termini_2010=termini_2010[termini_2010.geometry.type=='Polygon']
termini_2020=termini_2020[termini_2020.geometry.type=='Polygon']

termini_2000.loc[:,'geometry'] = termini_2000['geometry'].buffer(0)
termini_2010.loc[:,'geometry'] = termini_2010['geometry'].buffer(0)
termini_2020.loc[:,'geometry'] = termini_2020['geometry'].buffer(0)

termini_2000.loc[:,'time_period']='2000'
termini_2010.loc[:,'time_period']='2010'
termini_2020.loc[:,'time_period']='2020'

termini1=pd.concat([termini_2000,termini_2010,termini_2020]).reset_index().drop(columns=['index','level_0'])
Greenland_AOI= gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/Greenland/Greenland_RGI_AOI.gpkg')
termini = termini1[termini1.within(Greenland_AOI.loc[0, 'geometry'])]

boxes_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/valid_terminus_change_area.gpkg')
GRE_outlines = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Mouginot_GIS_basins/Kochtitzky_edits_Mouginot_GIS_basins_3995_v3.gpkg') 
boxes_attributed_ice_sheet = gpd.sjoin(boxes_ice_sheet, GRE_outlines, how='inner')[['NAME','geometry']]
boxes_attributed_ice_sheet['Glacier_type'] = 'Ice sheet'
boxes_attributed_ice_sheet.to_file('Greenland/boxes_attributed_ice_sheet_'+date_string+'.gpkg', driver='GPKG')

boxes_periphery = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')   
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['NAME']=RGI['RGIid_2']
boxes_attributed_periphery = gpd.sjoin(boxes_periphery, RGI, how='inner')[['NAME','geometry']]
boxes_attributed_periphery['Glacier_type'] = 'Periphery'

boxes_attributed1=boxes_attributed_ice_sheet.append(boxes_attributed_periphery,sort=False)
boxes_attributed = boxes_attributed1[boxes_attributed1.within(Greenland_AOI.loc[0, 'geometry'])]
boxes_attributed['geometry'] = boxes_attributed.geometry.buffer(-0.1)
boxes_attributed.to_file('Greenland/boxes_attributed1_'+date_string+'.gpkg', driver='GPKG')

#add data to the boxes
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2000, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Early_satellite_id'})
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2010, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Middle_satellite_id'})
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2020, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Late_satellite_id'})
boxes_attributed.loc[:,'Early_satellite_id'] = boxes_attributed['Early_satellite_id'].astype(str)
boxes_attributed.loc[:,'Middle_satellite_id'] = boxes_attributed['Middle_satellite_id'].astype(str)
boxes_attributed.loc[:,'Late_satellite_id'] = boxes_attributed['Late_satellite_id'].astype(str)
boxes_attributed = boxes_attributed.reset_index()
boxes_attributed.to_file('Greenland/boxes_attributed2_'+date_string+'.gpkg', driver='GPKG')
#get dates for the satellites images for the box attributes
#2000

boxes_attributed['Early_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Early_satellite_id'][i][0]=='L':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Early_satellite_id'][i][0:4]=='AST_':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Early_satellite_id'][i][0:4]=='AST1':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][12:20],format='%m%d%Y')
    if boxes_attributed['Early_satellite_id'][i][0:1]=='R':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][18:27],format='%d%m%Y')

#2010
boxes_attributed['Mid_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Middle_satellite_id'][i][0]=='L':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Middle_satellite_id'][i][0:4]=='AST_':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Middle_satellite_id'][i][0:4]=='AST1':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][12:20],format='%m%d%Y')
    if boxes_attributed['Middle_satellite_id'][i][0:3]=='IMG':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][26:34],format='%d%m%Y')

#2020
boxes_attributed['Late_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Late_satellite_id'][i][0]=='L':
        boxes_attributed.loc[i,'Late_date']=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Late_satellite_id'][i][0:4]=='AST_':
        boxes_attributed.loc[i,'Late_date']=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Late_satellite_id'][i][0:4]=='AST1':
        boxes_attributed.loc[i,'Late_date']=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][12:20],format='%m%d%Y')

boxes_attributed.drop(columns=['Early_date','Mid_date','Late_date']).to_file('Greenland/boxes_attributed_'+date_string+'.gpkg', driver='GPKG')
#get only the glacier terminus area within the box
termini_intersect = gpd.overlay(termini,boxes_attributed,how='intersection')#,keep_geom_type=True)
# termini_intersect['area'] = termini_intersect['geometry'].area/1000000

#get the date from the satellite ID - difference for Landsat and ASTER
termini_intersect['Date']=np.nan
for i in range(0,len(termini_intersect)):
    if termini_intersect['Landsat_ID'][i][0]=='L':
        termini_intersect.loc[i,'Date']=pd.to_datetime(termini_intersect['Landsat_ID'][i][17:25],format='%Y%m%d')
    if termini_intersect['Landsat_ID'][i][0:4]=='AST_':
        termini_intersect.loc[i,'Date']=pd.to_datetime(termini_intersect['Landsat_ID'][i][11:19],format='%m%d%Y')
    if termini_intersect['Landsat_ID'][i][0:4]=='AST1':
        termini_intersect.loc[i,'Date']=pd.to_datetime(termini_intersect['Landsat_ID'][i][12:20],format='%m%d%Y')

#sort by terminus date for differencing in next step
early_termini_intersect=termini_intersect[termini_intersect['time_period']=='2000']
middle_termini_intersect=termini_intersect[termini_intersect['time_period']=='2010']
late_termini_intersect=termini_intersect[termini_intersect['time_period']=='2020']

            #2000 to 2020
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_total = gpd.overlay(early_termini_intersect,late_termini_intersect,how='difference')
marine_based_glacier_area_lost_total=marine_based_glacier_area_lost_total[['NAME','Glacier_type','geometry']].dissolve(by='NAME')
marine_based_glacier_area_lost_total = marine_based_glacier_area_lost_total.reset_index()
marine_based_glacier_area_lost_total['geometry_3995']=marine_based_glacier_area_lost_total['geometry']
marine_based_glacier_area_lost_total.loc[:,'latlon'] = marine_based_glacier_area_lost_total.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_lost_total)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_lost_total.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_lost_total.loc[i]['latlon'].y)
    marine_based_glacier_area_lost_total = marine_based_glacier_area_lost_total.to_crs(projection)
    marine_based_glacier_area_lost_total.loc[[i],'geometry']=marine_based_glacier_area_lost_total.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_lost_total.loc[i,'area_km2'] = marine_based_glacier_area_lost_total.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_lost_total.loc[i,'perimeter'] = marine_based_glacier_area_lost_total.loc[i,'geometry'].length
    marine_based_glacier_area_lost_total.loc[i,'perimeter_error'] = marine_based_glacier_area_lost_total.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_total['perimeter_error'] = marine_based_glacier_area_lost_total['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_total=marine_based_glacier_area_lost_total[marine_based_glacier_area_lost_total['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2020
marine_based_glacier_area_gained_total = gpd.overlay(late_termini_intersect,early_termini_intersect,how='difference')
marine_based_glacier_area_gained_total=marine_based_glacier_area_gained_total[['NAME','Glacier_type','geometry']].dissolve(by='NAME')
marine_based_glacier_area_gained_total = marine_based_glacier_area_gained_total.reset_index()
marine_based_glacier_area_gained_total['geometry_3995']=marine_based_glacier_area_gained_total['geometry']
marine_based_glacier_area_gained_total.loc[:,'latlon'] = marine_based_glacier_area_gained_total.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_gained_total)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_gained_total.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_gained_total.loc[i]['latlon'].y)
    marine_based_glacier_area_gained_total = marine_based_glacier_area_gained_total.to_crs(projection)
    marine_based_glacier_area_gained_total.loc[[i],'geometry']=marine_based_glacier_area_gained_total.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_gained_total.loc[i,'area_km2'] = marine_based_glacier_area_gained_total.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_gained_total.loc[i,'perimeter'] = marine_based_glacier_area_gained_total.loc[i,'geometry'].length
    marine_based_glacier_area_gained_total.loc[i,'perimeter_error'] = marine_based_glacier_area_gained_total.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_total['perimeter_error'] = marine_based_glacier_area_gained_total['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_total=marine_based_glacier_area_gained_total[marine_based_glacier_area_gained_total['area_km2']>0.000225] #throw out tiny areas that are a mistakes


marine_based_glacier_area_lost_total.loc[:,'geometry']=marine_based_glacier_area_lost_total['geometry_3995']
marine_based_glacier_area_gained_total.loc[:,'geometry']=marine_based_glacier_area_gained_total['geometry_3995']
marine_based_glacier_area_lost_total.drop(columns=['geometry_3995','latlon'],inplace=True)
marine_based_glacier_area_gained_total.drop(columns=['geometry_3995','latlon'],inplace=True)

#export data
marine_based_glacier_area_lost_total.to_file('Greenland/marine_based_glacier_area_lost_total_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_total.to_file('Greenland/marine_based_glacier_area_gained_total_'+date_string+'.geojson', driver='GeoJSON')

            #2000 to 2010
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_early = gpd.overlay(early_termini_intersect,middle_termini_intersect,how='difference')
marine_based_glacier_area_lost_early=marine_based_glacier_area_lost_early[['NAME','Glacier_type','geometry']].dissolve(by='NAME')
marine_based_glacier_area_lost_early = marine_based_glacier_area_lost_early.reset_index()
marine_based_glacier_area_lost_early['geometry_3995']=marine_based_glacier_area_lost_early['geometry']
marine_based_glacier_area_lost_early.loc[:,'latlon'] = marine_based_glacier_area_lost_early.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_lost_early)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_lost_early.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_lost_early.loc[i]['latlon'].y)
    marine_based_glacier_area_lost_early = marine_based_glacier_area_lost_early.to_crs(projection)
    marine_based_glacier_area_lost_early.loc[[i],'geometry']=marine_based_glacier_area_lost_early.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_lost_early.loc[i,'area_km2'] = marine_based_glacier_area_lost_early.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_lost_early.loc[i,'perimeter'] = marine_based_glacier_area_lost_early.loc[i,'geometry'].length
    marine_based_glacier_area_lost_early.loc[i,'perimeter_error'] = marine_based_glacier_area_lost_early.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_early['perimeter_error'] = marine_based_glacier_area_lost_early['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_early=marine_based_glacier_area_lost_early[marine_based_glacier_area_lost_early['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2010
marine_based_glacier_area_gained_early = gpd.overlay(middle_termini_intersect,early_termini_intersect,how='difference')
marine_based_glacier_area_gained_early=marine_based_glacier_area_gained_early[['NAME','Glacier_type','geometry']].dissolve(by='NAME')
marine_based_glacier_area_gained_early = marine_based_glacier_area_gained_early.reset_index()
marine_based_glacier_area_gained_early['geometry_3995']=marine_based_glacier_area_gained_early['geometry']
marine_based_glacier_area_gained_early.loc[:,'latlon'] = marine_based_glacier_area_gained_early.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_gained_early)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_gained_early.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_gained_early.loc[i]['latlon'].y)
    marine_based_glacier_area_gained_early = marine_based_glacier_area_gained_early.to_crs(projection)
    marine_based_glacier_area_gained_early.loc[[i],'geometry']=marine_based_glacier_area_gained_early.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_gained_early.loc[i,'area_km2'] = marine_based_glacier_area_gained_early.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_gained_early.loc[i,'perimeter'] = marine_based_glacier_area_gained_early.loc[i,'geometry'].length
    marine_based_glacier_area_gained_early.loc[i,'perimeter_error'] = marine_based_glacier_area_gained_early.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_early['perimeter_error'] = marine_based_glacier_area_gained_early['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_early=marine_based_glacier_area_gained_early[marine_based_glacier_area_gained_early['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_early.loc[:,'geometry']=marine_based_glacier_area_lost_early['geometry_3995']
marine_based_glacier_area_gained_early.loc[:,'geometry']=marine_based_glacier_area_gained_early['geometry_3995']
marine_based_glacier_area_lost_early.drop(columns=['geometry_3995','latlon'],inplace=True)
marine_based_glacier_area_gained_early.drop(columns=['geometry_3995','latlon'],inplace=True)
#export data
marine_based_glacier_area_lost_early.to_file('Greenland/marine_based_glacier_area_lost_early_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_early.to_file('Greenland/marine_based_glacier_area_gained_early_'+date_string+'.geojson', driver='GeoJSON')


            #2010 to 2020
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_late = gpd.overlay(middle_termini_intersect,late_termini_intersect,how='difference')
marine_based_glacier_area_lost_late=marine_based_glacier_area_lost_late[['NAME','Glacier_type','geometry']].dissolve(by='NAME')
marine_based_glacier_area_lost_late = marine_based_glacier_area_lost_late.reset_index()
marine_based_glacier_area_lost_late['geometry_3995']=marine_based_glacier_area_lost_late['geometry']
marine_based_glacier_area_lost_late.loc[:,'latlon'] = marine_based_glacier_area_lost_late.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_lost_late)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_lost_late.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_lost_late.loc[i]['latlon'].y)
    marine_based_glacier_area_lost_late = marine_based_glacier_area_lost_late.to_crs(projection)
    marine_based_glacier_area_lost_late.loc[[i],'geometry']=marine_based_glacier_area_lost_late.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_lost_late.loc[i,'area_km2'] = marine_based_glacier_area_lost_late.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_lost_late.loc[i,'perimeter'] = marine_based_glacier_area_lost_late.loc[i,'geometry'].length
    marine_based_glacier_area_lost_late.loc[i,'perimeter_error'] = marine_based_glacier_area_lost_late.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_late['perimeter_error'] = marine_based_glacier_area_lost_late['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_late=marine_based_glacier_area_lost_late[marine_based_glacier_area_lost_late['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2010
marine_based_glacier_area_gained_late = gpd.overlay(late_termini_intersect,middle_termini_intersect,how='difference')
marine_based_glacier_area_gained_late=marine_based_glacier_area_gained_late[['NAME','Glacier_type','geometry']].dissolve(by='NAME')
marine_based_glacier_area_gained_late = marine_based_glacier_area_gained_late.reset_index()
marine_based_glacier_area_gained_late['geometry_3995']=marine_based_glacier_area_gained_late['geometry']
marine_based_glacier_area_gained_late.loc[:,'latlon'] = marine_based_glacier_area_gained_late.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_gained_late)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_gained_late.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_gained_late.loc[i]['latlon'].y)
    marine_based_glacier_area_gained_late = marine_based_glacier_area_gained_late.to_crs(projection)
    marine_based_glacier_area_gained_late.loc[[i],'geometry']=marine_based_glacier_area_gained_late.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_gained_late.loc[i,'area_km2'] = marine_based_glacier_area_gained_late.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_gained_late.loc[i,'perimeter'] = marine_based_glacier_area_gained_late.loc[i,'geometry'].length
    marine_based_glacier_area_gained_late.loc[i,'perimeter_error'] = marine_based_glacier_area_gained_late.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_late['perimeter_error'] = marine_based_glacier_area_gained_late['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_late=marine_based_glacier_area_gained_late[marine_based_glacier_area_gained_late['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_late.loc[:,'geometry']=marine_based_glacier_area_lost_late['geometry_3995']
marine_based_glacier_area_gained_late.loc[:,'geometry']=marine_based_glacier_area_gained_late['geometry_3995']
marine_based_glacier_area_lost_late.drop(columns=['geometry_3995','latlon'],inplace=True)
marine_based_glacier_area_gained_late.drop(columns=['geometry_3995','latlon'],inplace=True)

#export data
marine_based_glacier_area_lost_late.to_file('Greenland/marine_based_glacier_area_lost_late_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_late.to_file('Greenland/marine_based_glacier_area_gained_late_'+date_string+'.geojson', driver='GeoJSON')

# #create one list of all the glaciers we have terminus area change for
all_glacier_list=marine_based_glacier_area_lost_early.append([marine_based_glacier_area_gained_early, marine_based_glacier_area_lost_late, marine_based_glacier_area_gained_late])
all_glacier_list=all_glacier_list.groupby('NAME').min().reset_index()[['NAME']]

#merge datasets to calculate net area change
all_glacier_list=pd.merge(all_glacier_list,marine_based_glacier_area_gained_total,on=['NAME'],how='left')
net_area_change1 = pd.merge(all_glacier_list,marine_based_glacier_area_lost_total,on=['NAME'],how='left').rename(columns={"geometry_x": "geometry_gained_total", "area_km2_x": "area_km2_gained_total","geometry_y": "geometry_lost_total", "area_km2_y": "area_km2_lost_total",'perimeter_x':'perimeter_gained_total','perimeter_y':'perimeter_lost_total','perimeter_error_x':'perimeter_error_gained_total','perimeter_error_y':'perimeter_error_lost_total'})
net_area_change2 = pd.merge(net_area_change1,marine_based_glacier_area_lost_early,on=['NAME'],how='left').rename(columns={"geometry": "geometry_lost_early", "area_km2": "area_km2_lost_early",'perimeter':'perimeter_lost_early','perimeter_error':'perimeter_error_lost_early'})
net_area_change3 = pd.merge(net_area_change2,marine_based_glacier_area_gained_early,on=['NAME'],how='left').rename(columns={"geometry": "geometry_gained_early", "area_km2": "area_km2_gained_early",'perimeter':'perimeter_gained_early','perimeter_error':'perimeter_error_gained_early'})
net_area_change4 = pd.merge(net_area_change3,marine_based_glacier_area_lost_late,on=['NAME'],how='left').rename(columns={"geometry": "geometry_lost_late", "area_km2": "area_km2_lost_late",'perimeter':'perimeter_lost_late','perimeter_error':'perimeter_error_lost_late'})
net_area_change5 = pd.merge(net_area_change4,marine_based_glacier_area_gained_late,on=['NAME'],how='left').rename(columns={"geometry": "geometry_gained_late", "area_km2": "area_km2_gained_late",'perimeter':'perimeter_gained_late','perimeter_error':'perimeter_error_gained_late'})
net_area_change = pd.merge(net_area_change5,boxes_attributed[['NAME','Early_satellite_id','Middle_satellite_id', 'Late_satellite_id', 'Early_date','Mid_date','Late_date']],on=['NAME'],how='left')

#Set areas to 0 where there is no gain (but a loss) or no loss (but a gain) so that you still get results for net change 
net_area_change=net_area_change.reset_index()
for i in range(0,len(net_area_change)):
    if net_area_change['area_km2_gained_early'].isnull()[i] and (net_area_change['area_km2_lost_early'][i]>0):
        net_area_change.loc[i,'area_km2_gained_early']=0
    if net_area_change['area_km2_lost_early'].isnull()[i] and (net_area_change['area_km2_gained_early'][i]>0):
        net_area_change.loc[i,'area_km2_lost_early']=0
        
    if net_area_change['area_km2_gained_late'].isnull()[i] and (net_area_change['area_km2_lost_late'][i]>0):
        net_area_change.loc[i,'area_km2_gained_late']=0
    if net_area_change['area_km2_lost_late'].isnull()[i] and (net_area_change['area_km2_gained_late'][i]>0):
        net_area_change.loc[i,'area_km2_lost_late']=0
        
    if net_area_change['area_km2_gained_total'].isnull()[i] and (net_area_change['area_km2_lost_total'][i]>0):
        net_area_change.loc[i,'area_km2_gained_total']=0
    if net_area_change['area_km2_lost_total'].isnull()[i] and (net_area_change['area_km2_gained_total'][i]>0):
        net_area_change.loc[i,'area_km2_lost_total']=0

#calculate time elapsed between two images
net_area_change['Early_date_decimal_year']=np.nan
net_area_change['Middle_date_decimal_year']=np.nan
net_area_change['Late_date_decimal_year']=np.nan
for i in range(0,len(net_area_change)):
    if pd.isna(net_area_change['Early_date'][i])==False:#this way it skips nan values
        net_area_change.loc[i,'Early_date_decimal_year'] = net_area_change['Early_date'][i].year+net_area_change['Early_date'][i].dayofyear/365
    if pd.isna(net_area_change['Mid_date'][i])==False:#this way it skips nan values
        net_area_change.loc[i,'Middle_date_decimal_year'] = net_area_change['Mid_date'][i].year+net_area_change['Mid_date'][i].dayofyear/365
    if pd.isna(net_area_change['Late_date'][i])==False:#this way it skips nan values
        net_area_change.loc[i,'Late_date_decimal_year'] = net_area_change['Late_date'][i].year+net_area_change['Late_date'][i].dayofyear/365
net_area_change.loc[:,'total_years_elapsed']=net_area_change['Late_date_decimal_year']-net_area_change['Early_date_decimal_year']
net_area_change.loc[:,'early_to_middle_years_elapsed']=net_area_change['Middle_date_decimal_year']-net_area_change['Early_date_decimal_year']
net_area_change.loc[:,'middle_to_late_years_elapsed']=net_area_change['Late_date_decimal_year']-net_area_change['Middle_date_decimal_year']

net_area_change['total_years_elapsed'].fillna(20,inplace=True)
net_area_change['early_to_middle_years_elapsed'].fillna(10,inplace=True)
net_area_change['middle_to_late_years_elapsed'].fillna(10,inplace=True)

#clean up the dataset by replacing nans with 0 (since you have no area if nan) and then find net change
net_area_change=net_area_change.fillna(value={'area_km2_gained_early':0,'area_km2_gained_late':0,'area_km2_lost_early':0,'area_km2_lost_late':0,
                                              'perimeter_error_gained_early':0,'perimeter_error_lost_early':0,'perimeter_error_gained_late':0,'perimeter_error_lost_late':0,
                                             'area_km2_gained_total':0,'area_km2_lost_total':0,'perimeter_error_gained_total':0,'perimeter_error_lost_total':0})
net_area_change=net_area_change.drop_duplicates(subset='NAME')
net_area_change.loc[:,'net_area_change_total']=net_area_change['area_km2_gained_total']-net_area_change['area_km2_lost_total']
net_area_change.loc[:,'net_area_change_total_error']=net_area_change['perimeter_error_gained_total']+net_area_change['perimeter_error_lost_total']
net_area_change.loc[:,'net_area_change_per_year_total']=net_area_change['net_area_change_total']/net_area_change['total_years_elapsed']
net_area_change.loc[:,'net_area_change_per_year_total_error']=net_area_change['net_area_change_total_error']/net_area_change['total_years_elapsed']
net_area_change.loc[:,'net_area_change_early']=net_area_change['area_km2_gained_early']-net_area_change['area_km2_lost_early']
net_area_change.loc[:,'net_area_change_early_error']=net_area_change['perimeter_error_gained_early']+net_area_change['perimeter_error_lost_early']
net_area_change.loc[:,'net_area_change_per_year_early']=net_area_change['net_area_change_early']/net_area_change['early_to_middle_years_elapsed']
net_area_change.loc[:,'net_area_change_per_year_early_error']=net_area_change['net_area_change_early_error']/net_area_change['early_to_middle_years_elapsed']
net_area_change.loc[:,'net_area_change_late']=net_area_change['area_km2_gained_late']-net_area_change['area_km2_lost_late']
net_area_change.loc[:,'net_area_change_late_error']=net_area_change['perimeter_error_gained_late']+net_area_change['perimeter_error_lost_late']
net_area_change.loc[:,'net_area_change_per_year_late']=net_area_change['net_area_change_late']/net_area_change['middle_to_late_years_elapsed']
net_area_change.loc[:,'net_area_change_per_year_late_error']=net_area_change['net_area_change_late_error']/net_area_change['middle_to_late_years_elapsed']

#make pretty dates
net_area_change.loc[:,'Early_date'] = pd.to_datetime(net_area_change['Early_date']).dt.strftime('%Y/%m/%d')
net_area_change.loc[:,'Mid_date'] = pd.to_datetime(net_area_change['Mid_date']).dt.strftime('%Y/%m/%d')
net_area_change.loc[:,'Late_date'] = pd.to_datetime(net_area_change['Late_date']).dt.strftime('%Y/%m/%d')

net_area_change.to_pickle('Greenland/net_area_change_preliminary'+date_string+'.pkl')#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:254: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:260: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Glacier_type_x'} in the result is deprecated and will raise a MergeError in a

In [24]:
net_area_change = pd.read_pickle('Greenland/net_area_change_preliminary'+date_string+'.pkl')
#Fill zeros in area change data so that errors dont come out null
net_area_change[['index','NAME', #'geometry_gained_total',
       'area_km2_gained_total', 'perimeter_gained_total',
       'perimeter_error_gained_total', #'geometry_lost_total',
       'area_km2_lost_total', 'perimeter_lost_total',
       'perimeter_error_lost_total', #'geometry_lost_early',
       'area_km2_lost_early', 'perimeter_lost_early',
       'perimeter_error_lost_early', #'geometry_gained_early',
       'area_km2_gained_early', 'perimeter_gained_early',
       'perimeter_error_gained_early', #'geometry_lost_late',
       'area_km2_lost_late', 'perimeter_lost_late',
       'perimeter_error_lost_late', #'geometry_gained_late',
       'area_km2_gained_late', 'perimeter_gained_late',
       'perimeter_error_gained_late', 'Early_date', 'Mid_date',
       'Late_date', 'Early_date_decimal_year', 'Middle_date_decimal_year',
       'Late_date_decimal_year', 'total_years_elapsed',
       'early_to_middle_years_elapsed', 'middle_to_late_years_elapsed',
       'net_area_change_total', 'net_area_change_total_error',
       'net_area_change_per_year_total',
       'net_area_change_per_year_total_error', 'net_area_change_early',
       'net_area_change_early_error', 'net_area_change_per_year_early',
       'net_area_change_per_year_early_error', 'net_area_change_late',
       'net_area_change_late_error', 'net_area_change_per_year_late',
       'net_area_change_per_year_late_error']] = net_area_change[['index','NAME', #'geometry_gained_total',
       'area_km2_gained_total', 'perimeter_gained_total',
       'perimeter_error_gained_total', #'geometry_lost_total',
       'area_km2_lost_total', 'perimeter_lost_total',
       'perimeter_error_lost_total', #'geometry_lost_early',
       'area_km2_lost_early', 'perimeter_lost_early',
       'perimeter_error_lost_early', #'geometry_gained_early',
       'area_km2_gained_early', 'perimeter_gained_early',
       'perimeter_error_gained_early',# 'geometry_lost_late',
       'area_km2_lost_late', 'perimeter_lost_late',
       'perimeter_error_lost_late',# 'geometry_gained_late',
       'area_km2_gained_late', 'perimeter_gained_late',
       'perimeter_error_gained_late', 'Early_date', 'Mid_date',
       'Late_date', 'Early_date_decimal_year', 'Middle_date_decimal_year',
       'Late_date_decimal_year', 'total_years_elapsed',
       'early_to_middle_years_elapsed', 'middle_to_late_years_elapsed',
       'net_area_change_total', 'net_area_change_total_error',
       'net_area_change_per_year_total',
       'net_area_change_per_year_total_error', 'net_area_change_early',
       'net_area_change_early_error', 'net_area_change_per_year_early',
       'net_area_change_per_year_early_error', 'net_area_change_late',
       'net_area_change_late_error', 'net_area_change_per_year_late',
       'net_area_change_per_year_late_error']].fillna(value=0)

In [25]:
net_area_change = net_area_change.reset_index()
column_names = ['geometry_gained_total','geometry_lost_total','geometry_lost_early','geometry_gained_early','geometry_lost_late','geometry_gained_late']
for c in column_names:
    for i in range(0,len(net_area_change)):
        try:
            if net_area_change[c][i] == None:
                net_area_change.loc[i,c] = Polygon([(0, 0),(0, 0),(0, 0)])
        except:
              print("An exception occurred")

In [26]:
#Figure out thickness of lost/gained glacier area
OIB= gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Thickness_Data/Greenland_OIB_within_10km_of_fluxgates_or_near_termini_3995.gpkg')

net_area_change['average_OIB_thickness_lost_early'] = np.nan
net_area_change['average_OIB_thickness_gained_early'] = np.nan
net_area_change['average_OIB_thickness_lost_late'] = np.nan
net_area_change['average_OIB_thickness_gained_late'] = np.nan
net_area_change['average_OIB_thickness_lost_total'] = np.nan
net_area_change['average_OIB_thickness_gained_total'] = np.nan
# net_area_change['average_OIB_thickness_lost_early_err'] = np.nan
# net_area_change['average_OIB_thickness_gained_early_err'] = np.nan
# net_area_change['average_OIB_thickness_lost_late_err'] = np.nan
# net_area_change['average_OIB_thickness_gained_late_err'] = np.nan

for i in range(0,len(net_area_change)):
    if net_area_change['geometry_lost_early'][i]!=0:
        net_area_change.loc[i,'average_OIB_thickness_lost_early'] = OIB[OIB.within(net_area_change['geometry_lost_early'][i])]['THICK'].mean()
    if net_area_change['geometry_gained_early'][i]!=0:
        net_area_change.loc[i,'average_OIB_thickness_gained_early'] = OIB[OIB.within(net_area_change['geometry_gained_early'][i])]['THICK'].mean()
    if net_area_change['geometry_lost_late'][i]!=0:
        net_area_change.loc[i,'average_OIB_thickness_lost_late'] = OIB[OIB.within(net_area_change['geometry_lost_late'][i])]['THICK'].mean()
    if net_area_change['geometry_gained_late'][i]!=0:
        net_area_change.loc[i,'average_OIB_thickness_gained_late'] = OIB[OIB.within(net_area_change['geometry_gained_late'][i])]['THICK'].mean()
    if net_area_change['geometry_lost_total'][i]!=0:
        net_area_change.loc[i,'average_OIB_thickness_lost_total'] = OIB[OIB.within(net_area_change['geometry_lost_total'][i])]['THICK'].mean()
    if net_area_change['geometry_gained_total'][i]!=0:
        net_area_change.loc[i,'average_OIB_thickness_gained_total'] = OIB[OIB.within(net_area_change['geometry_gained_total'][i])]['THICK'].mean()
        
#Find the Glacier thickness for the areas lost/gained from the Farinotti and other raster datasets
net_area_change['average_bedmachine_thickness_lost_early'] = np.nan
net_area_change['average_bedmachine_thickness_gained_early'] = np.nan
net_area_change['average_bedmachine_thickness_lost_late'] = np.nan
net_area_change['average_bedmachine_thickness_gained_late'] = np.nan
net_area_change['average_bedmachine_thickness_lost_total'] = np.nan
net_area_change['average_bedmachine_thickness_gained_total'] = np.nan

net_area_change['average_bedmachine_thickness_lost_early_err'] = np.nan
net_area_change['average_bedmachine_thickness_gained_early_err'] = np.nan
net_area_change['average_bedmachine_thickness_lost_late_err'] = np.nan
net_area_change['average_bedmachine_thickness_gained_late_err'] = np.nan
net_area_change['average_bedmachine_thickness_lost_total_err'] = np.nan
net_area_change['average_bedmachine_thickness_gained_total_err'] = np.nan

net_area_change['average_Milan_thickness_lost_early'] = np.nan
net_area_change['average_Milan_thickness_gained_early'] = np.nan
net_area_change['average_Milan_thickness_lost_late'] = np.nan
net_area_change['average_Milan_thickness_gained_late'] = np.nan

net_area_change['average_Milan_thickness_lost_early_err'] = np.nan
net_area_change['average_Milan_thickness_gained_early_err'] = np.nan
net_area_change['average_Milan_thickness_lost_late_err'] = np.nan
net_area_change['average_Milan_thickness_gained_late_err'] = np.nan

net_area_change['average_Milan_thickness_lost_total'] = np.nan
net_area_change['average_Milan_thickness_gained_total'] = np.nan
net_area_change['average_Milan_thickness_lost_total_err'] = np.nan
net_area_change['average_Milan_thickness_gained_total_err'] = np.nan

bedmachine_raster = '/Users/willkochtitzky/Projects/Greenland_Calving/Thickness_Data/BedMachine_v4_Morlighem/BedMachineGreenland-2021-01-26_thickness_polar.tif'
bedmachine_err_raster = '/Users/willkochtitzky/Projects/Greenland_Calving/Thickness_Data/BedMachine_v4_Morlighem/BedMachineGreenland-2021-01-26_err_polar.tif'

for i in range(0,len(net_area_change)):
    if net_area_change['geometry_lost_early'][i].area!=0:
        net_area_change.loc[i,'average_bedmachine_thickness_lost_early']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],bedmachine_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_bedmachine_thickness_lost_early_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],bedmachine_err_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_early']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_early_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
    if net_area_change['geometry_gained_early'][i].area!=0:
        net_area_change.loc[i,'average_bedmachine_thickness_gained_early']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],bedmachine_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_bedmachine_thickness_gained_early_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],bedmachine_err_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_early']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_early_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
    if net_area_change['geometry_lost_late'][i].area!=0:
        net_area_change.loc[i,'average_bedmachine_thickness_lost_late']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],bedmachine_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_bedmachine_thickness_lost_late_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],bedmachine_err_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_late']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_late_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
    if net_area_change['geometry_gained_late'][i].area!=0:
        net_area_change.loc[i,'average_bedmachine_thickness_gained_late']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i].buffer(0),bedmachine_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_bedmachine_thickness_gained_late_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i].buffer(0),bedmachine_err_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_late']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i].buffer(0),'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_late_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i].buffer(0),'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
    if net_area_change['geometry_lost_total'][i].area!=0:
        net_area_change.loc[i,'average_bedmachine_thickness_lost_total']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],bedmachine_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_bedmachine_thickness_lost_total_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],bedmachine_err_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_total']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_total_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
    if net_area_change['geometry_gained_total'][i].area!=0:
        net_area_change.loc[i,'average_bedmachine_thickness_gained_total']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],bedmachine_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_bedmachine_thickness_gained_total_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],bedmachine_err_raster,stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_total']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_total_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        
net_area_change.to_pickle('Greenland/net_area_change'+date_string+'.pkl')#change file name based on distance

In [27]:
#calculate the melt below the flux gate
flux_lines= gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Kochtitzky_fluxgates.gpkg')
flux_lines_ice_sheet = flux_lines[flux_lines['Glacier_type']=='Ice sheet']  
#Read in RGI and fluxgate points (on and off glacier) data
# boxes = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/valid_terminus_change_area.gpkg')
boxes = gpd.read_file('Greenland/boxes_attributed_'+date_string+'.gpkg')
boxes['geometry'] = boxes['geometry'].buffer(-0.1)

termini_2000_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2000_outlines.gpkg')
split_output_accumulated = gpd.GeoDataFrame()
split_output_accumulated2 = gpd.GeoDataFrame()
split_output_accumulated3 = gpd.GeoDataFrame()

stranded_polys = gpd.GeoDataFrame()
gdf = gpd.GeoDataFrame()
ii=0
iii=0
iiii=0
counter=0
#produce stranded portion of polygons for each glacier
for g in range(0,len(termini_2000_ice_sheet)):
#     print(termini_2000_ice_sheet['fid_perm'][g])
    for i in range(0,len(flux_lines_ice_sheet)):
        if flux_lines_ice_sheet.geometry.iloc[i].intersects(termini_2000_ice_sheet.geometry.iloc[g]):
            split_output = split(termini_2000_ice_sheet.geometry.iloc[g],flux_lines_ice_sheet.geometry.iloc[i])
            
            split_output_accumulated=split_output_accumulated.append(gpd.GeoDataFrame(split_output))
            
            for ii in range(i+1,len(flux_lines_ice_sheet)):
                if flux_lines_ice_sheet.geometry.iloc[ii].intersects(split_output):
                    gdf = gpd.GeoDataFrame()
                    for k in range(0,len(split_output)):
                        gdf.loc[k, 'geometry'] = split_output.geoms[k]
                        gdf.crs = from_epsg(3995)
                        gdf.loc[k, 'area'] = split_output[k].area/1000000
                    gdf = gpd.sjoin(gdf,boxes,how='inner')
                    split_output = split(gdf.geometry.iloc[0],flux_lines_ice_sheet.geometry.iloc[ii])
                    split_output_accumulated2=split_output_accumulated.append(gpd.GeoDataFrame(split_output))
            for iii in range(ii+1,len(flux_lines_ice_sheet)):
                if flux_lines_ice_sheet.geometry.iloc[iii].intersects(split_output):
                    gdf = gpd.GeoDataFrame()
                    for k in range(0,len(split_output)):
                        gdf.loc[k, 'geometry'] = split_output.geoms[k]
                        gdf.crs = from_epsg(3995)
                        gdf.loc[k, 'area'] = split_output[k].area/1000000
                    gdf = gpd.sjoin(gdf,boxes,how='inner')
                    split_output = split(gdf.geometry.iloc[0],flux_lines_ice_sheet.geometry.iloc[iii])
            for iiii in range(iii+1,len(flux_lines_ice_sheet)):
                if flux_lines_ice_sheet.geometry.iloc[iiii].intersects(split_output):
                    gdf = gpd.GeoDataFrame()
                    for k in range(0,len(split_output)):
                        gdf.loc[k, 'geometry'] = split_output.geoms[k]
                        gdf.crs = from_epsg(3995)
                        gdf.loc[k, 'area'] = split_output[k].area/1000000
                    gdf = gpd.sjoin(gdf,boxes,how='inner')
                    split_output = split(gdf.geometry.iloc[0],flux_lines_ice_sheet.geometry.iloc[iiii])
            for iiiii in range(iiii+1,len(flux_lines_ice_sheet)):
                if flux_lines_ice_sheet.geometry.iloc[iiii].intersects(split_output):
                    gdf = gpd.GeoDataFrame()
                    for k in range(0,len(split_output)):
                        gdf.loc[k, 'geometry'] = split_output.geoms[k]
                        gdf.crs = from_epsg(3995)
                        gdf.loc[k, 'area'] = split_output[k].area/1000000
                    gdf = gpd.sjoin(gdf,boxes,how='inner')
                    split_output = split(gdf.geometry.iloc[0],flux_lines_ice_sheet.geometry.iloc[iiiii])
            split_output_accumulated3=split_output_accumulated.append(gpd.GeoDataFrame(split_output))
            gdf = gpd.GeoDataFrame()
            for k in range(0,len(split_output)):
                gdf.loc[k, 'geometry'] = split_output.geoms[k]
                gdf.crs = from_epsg(3995)
                gdf.loc[k, 'area'] = split_output[k].area/1000000
                counter=len(gdf)+counter
            try:
                gdf = gpd.sjoin(gdf,boxes,how='inner')
                stranded_polys = stranded_polys.append(gdf)#.sort_values('area',ascending=False)[0:1])
            except:
                print('------------------------------error--------------------------------')
                print(termini_2000_ice_sheet['fid_perm'][g])
                print('------------------------------error--------------------------------')
                break
            break

stranded_polys = stranded_polys.dissolve(by='NAME').reset_index()
stranded_polys['geometry'] = stranded_polys['geometry'].buffer(0.05)
stranded_polys['geometry'] = stranded_polys['geometry'].buffer(-0.1)
stranded_polys.to_file('Greenland/stranded_polygons_early_'+date_string+'.gpkg',driver='GPKG')

termini_2000_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2000_outlines.gpkg')
termini_2010_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2010_outlines.gpkg')
termini_2020_ice_sheet = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Terminus_outlines/Greenland_2020_outlines.gpkg')
termini_2000_ice_sheet = gpd.overlay(termini_2000_ice_sheet,boxes,how='intersection')
termini_2010_ice_sheet = gpd.overlay(termini_2010_ice_sheet,boxes,how='intersection')
termini_2020_ice_sheet = gpd.overlay(termini_2020_ice_sheet,boxes,how='intersection')
termini_2000_ice_sheet['geometry'] = termini_2000_ice_sheet['geometry'].buffer(0)
termini_2010_ice_sheet['geometry'] = termini_2010_ice_sheet['geometry'].buffer(0)
termini_2020_ice_sheet['geometry'] = termini_2020_ice_sheet['geometry'].buffer(0)

print('START SECOND HALF')
marine_based_glacier_area_lost_early = gpd.overlay(termini_2000_ice_sheet,termini_2010_ice_sheet,how='difference')
marine_based_glacier_area_gained_early = gpd.overlay(termini_2010_ice_sheet,termini_2000_ice_sheet,how='difference')
standed_poly_middle = gpd.overlay(marine_based_glacier_area_gained_early,stranded_polys,how='union')
marine_based_glacier_area_lost_early.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in marine_based_glacier_area_lost_early.geometry]
standed_poly_middle.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in standed_poly_middle.geometry]
marine_based_glacier_area_lost_early = marine_based_glacier_area_lost_early.explode()
marine_based_glacier_area_lost_early.to_file('Greenland/marine_based_glacier_area_lost_early'+date_string,driver='GPKG')
marine_based_glacier_area_lost_early['geometry']=marine_based_glacier_area_lost_early['geometry'].buffer(0)
standed_poly_middle = gpd.overlay(standed_poly_middle,marine_based_glacier_area_lost_early,how='difference')
standed_poly_middle.to_file('Greenland/stranded1_polygons_middle_'+date_string,driver='GPKG')
standed_poly_middle = standed_poly_middle[(standed_poly_middle['geometry'].type=='Polygon')|(standed_poly_middle['geometry'].type=='MultiPolygon')]
standed_poly_middle.rename(columns={"NAME_1": "NAME",'Glacier_type_1':'Glacier_type','Early_satellite_id_1':'Early_satellite_id','Middle_satellite_id_1':'Middle_satellite_id','Late_satellite_id_1':'Late_satellite_id'},inplace=True)
standed_poly_middle['NAME'].fillna(standed_poly_middle['NAME_2'],inplace=True)
standed_poly_middle['Early_satellite_id'].fillna(standed_poly_middle['Early_satellite_id_2'],inplace=True)
standed_poly_middle['Middle_satellite_id'].fillna(standed_poly_middle['Middle_satellite_id_2'],inplace=True)
standed_poly_middle['Late_satellite_id'].fillna(standed_poly_middle['Late_satellite_id_2'],inplace=True)
standed_poly_middle['Glacier_type'].fillna(standed_poly_middle['Late_satellite_id_2'],inplace=True)
standed_poly_middle.drop(columns=['NAME_2','Landsat_ID','Late_satellite_id_2','index_1', 'index_right', 'index_2','Glacier_type_2', 'fid_perm_1','fid_perm_2', 'area','Early_satellite_id_2','Middle_satellite_id_2','Late_satellite_id_2'],inplace=True)
standed_poly_middle['geometry'] = standed_poly_middle['geometry'].buffer(0.05)
standed_poly_middle = standed_poly_middle.dissolve(by='NAME').reset_index()
standed_poly_middle['geometry'] = standed_poly_middle['geometry'].buffer(-0.1)
standed_poly_middle.to_file('Greenland/stranded_polygons_middle_'+date_string+'.gpkg',driver='GPKG')

marine_based_glacier_area_lost_late = gpd.overlay(termini_2010_ice_sheet,termini_2020_ice_sheet,how='difference')
marine_based_glacier_area_gained_late = gpd.overlay(termini_2020_ice_sheet,termini_2010_ice_sheet,how='difference')
standed_poly_late = gpd.overlay(marine_based_glacier_area_gained_late,standed_poly_middle,how='union')
marine_based_glacier_area_lost_late.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in marine_based_glacier_area_lost_late.geometry]
standed_poly_late.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in standed_poly_late.geometry]
marine_based_glacier_area_lost_late.to_file('Greenland/marine_based_glacier_area_lost_late'+date_string,driver='GPKG')
marine_based_glacier_area_lost_late['geometry']=marine_based_glacier_area_lost_late['geometry'].buffer(0)
standed_poly_late = gpd.overlay(standed_poly_late,marine_based_glacier_area_lost_late.explode(),how='difference')
standed_poly_late = standed_poly_late[(standed_poly_late['geometry'].type=='Polygon')|(standed_poly_late['geometry'].type=='MultiPolygon')]
standed_poly_late.rename(columns={"NAME_1": "NAME",'Glacier_type_1':'Glacier_type','Early_satellite_id_1':'Early_satellite_id','Middle_satellite_id_1':'Middle_satellite_id','Late_satellite_id_1':'Late_satellite_id','Landsat_ID_1':'Landsat_ID'},inplace=True)
standed_poly_late['NAME'].fillna(standed_poly_late['NAME_2'],inplace=True)
standed_poly_late['Early_satellite_id'].fillna(standed_poly_late['Early_satellite_id_2'],inplace=True)
standed_poly_late['Middle_satellite_id'].fillna(standed_poly_late['Middle_satellite_id_2'],inplace=True)
standed_poly_late['Late_satellite_id'].fillna(standed_poly_late['Late_satellite_id_2'],inplace=True)
standed_poly_late['Glacier_type'].fillna(standed_poly_late['Late_satellite_id_2'],inplace=True)
standed_poly_late.drop(columns=['NAME_2','Landsat_ID','Late_satellite_id_2','Glacier_type_2','Early_satellite_id_2','Middle_satellite_id_2','Late_satellite_id_2'],inplace=True)
standed_poly_late = standed_poly_late.dissolve(by='NAME').reset_index()
standed_poly_late['geometry'] = standed_poly_late['geometry'].buffer(0.05)
standed_poly_late['geometry'] = standed_poly_late['geometry'].buffer(-0.1)
standed_poly_late.to_file('Greenland/stranded_polygons_late_'+date_string+'.gpkg',driver='GPKG')

print('END END END')

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:74: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:74: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

...

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

START SECOND HALF


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:101: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3995
Right CRS: None

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:101: UserWarning: `keep_geom_type=True` in overlay resulted in 7 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:124: UserWarning: `keep_geom_type=True` in overlay resulted in 12 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries


END END END


In [28]:
#Read in terminus change for SMB extraction
marine_based_glacier_area_gained_early=gpd.read_file('Greenland/marine_based_glacier_area_gained_early_'+date_string+'.geojson', driver='GeoJSON')
for i in range(0,len(marine_based_glacier_area_gained_early)):
    marine_based_glacier_area_gained_early.loc[i,'NAME_first3']=marine_based_glacier_area_gained_early['NAME'][i][0:3]    
marine_based_glacier_area_gained_early_GIC = marine_based_glacier_area_gained_early[marine_based_glacier_area_gained_early['NAME_first3']=='RGI'].reset_index(drop=True)
marine_based_glacier_area_gained_early_GrIS = marine_based_glacier_area_gained_early[marine_based_glacier_area_gained_early['NAME_first3']!='RGI'].reset_index(drop=True)

marine_based_glacier_area_lost_early=gpd.read_file('Greenland/marine_based_glacier_area_lost_early_'+date_string+'.geojson', driver='GeoJSON')
for i in range(0,len(marine_based_glacier_area_lost_early)):
    marine_based_glacier_area_lost_early.loc[i,'NAME_first3']=marine_based_glacier_area_lost_early['NAME'][i][0:3]    
marine_based_glacier_area_lost_early_GIC = marine_based_glacier_area_lost_early[marine_based_glacier_area_lost_early['NAME_first3']=='RGI'].reset_index(drop=True)
marine_based_glacier_area_lost_early_GrIS = marine_based_glacier_area_lost_early[marine_based_glacier_area_lost_early['NAME_first3']!='RGI'].reset_index(drop=True)

marine_based_glacier_area_gained_late=gpd.read_file('Greenland/marine_based_glacier_area_gained_late_'+date_string+'.geojson', driver='GeoJSON')
for i in range(0,len(marine_based_glacier_area_gained_late)):
    marine_based_glacier_area_gained_late.loc[i,'NAME_first3']=marine_based_glacier_area_gained_late['NAME'][i][0:3]    
marine_based_glacier_area_gained_late_GIC = marine_based_glacier_area_gained_late[marine_based_glacier_area_gained_late['NAME_first3']=='RGI'].reset_index(drop=True)
marine_based_glacier_area_gained_late_GrIS = marine_based_glacier_area_gained_late[marine_based_glacier_area_gained_late['NAME_first3']!='RGI'].reset_index(drop=True)

marine_based_glacier_area_lost_late=gpd.read_file('Greenland/marine_based_glacier_area_lost_late_'+date_string+'.geojson', driver='GeoJSON')
for i in range(0,len(marine_based_glacier_area_lost_late)):
    marine_based_glacier_area_lost_late.loc[i,'NAME_first3']=marine_based_glacier_area_lost_late['NAME'][i][0:3]    
marine_based_glacier_area_lost_late_GIC = marine_based_glacier_area_lost_late[marine_based_glacier_area_lost_late['NAME_first3']=='RGI'].reset_index(drop=True)
marine_based_glacier_area_lost_late_GrIS = marine_based_glacier_area_lost_late[marine_based_glacier_area_lost_late['NAME_first3']!='RGI'].reset_index(drop=True)

marine_based_glacier_area_gained_early_GIC.drop(columns='NAME_first3',inplace=True)
marine_based_glacier_area_lost_early_GIC.drop(columns='NAME_first3',inplace=True)
marine_based_glacier_area_gained_late_GIC.drop(columns='NAME_first3',inplace=True)
marine_based_glacier_area_lost_late_GIC.drop(columns='NAME_first3',inplace=True)

marine_based_glacier_area_gained_early_GrIS.drop(columns='NAME_first3',inplace=True)
marine_based_glacier_area_lost_early_GrIS.drop(columns='NAME_first3',inplace=True)
marine_based_glacier_area_gained_late_GrIS.drop(columns='NAME_first3',inplace=True)
marine_based_glacier_area_lost_late_GrIS.drop(columns='NAME_first3',inplace=True)

In [29]:
stranded_poly_late = gpd.read_file('Greenland/stranded_polygons_early_'+date_string+'.gpkg')
for i in range(0,len(stranded_poly_late)):
    if stranded_poly_late['NAME'][i][0:3]=='RGI':
        print(stranded_poly_late['NAME'][i])

In [30]:
#SMB for ICE SHEET
stranded_poly_early = gpd.read_file('Greenland/stranded_polygons_early_'+date_string+'.gpkg',driver='GPKG')
stranded_poly_middle = gpd.read_file('Greenland/stranded_polygons_middle_'+date_string+'.gpkg',driver='GPKG')
stranded_poly_late = gpd.read_file('Greenland/stranded_polygons_late_'+date_string+'.gpkg',driver='GPKG')

# smb_datasets_GIC = sorted(glob.glob('/Users/willkochtitzky/Projects/Greenland_Calving/SMB/GIC/SMB_rec.20*_3995polar.tif'))
smb_datasets_GrIS = sorted(glob.glob('/Users/willkochtitzky/Projects/Greenland_Calving/SMB/GrIS/smb_rec.20*_3995polar.tif'))

for i in range(0,len(smb_datasets_GrIS)):
# for i in range(0,1):
    data_name= re.search('rec.(.*).BN_RACMO2.3p2_ERA5_3h_FGRN055.1km.MM_smb_3413polar_3995polar.tif',smb_datasets_GrIS[i]).group(1)
    data_name = data_name+'_SMB'
    month_accum=[]
    print(data_name)
    for month in range(1,13,1):
        data_name_month=data_name+str(month)
        month_accum.append(data_name_month)
        #Stranded polygons
        for j in range(0,len(stranded_poly_late)):
            try:
                stranded_poly_early.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_early.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
                stranded_poly_middle.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_middle.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                stranded_poly_late.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_late.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
            except:
                stranded_poly_early.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_early.geometry[j].buffer(500),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
                stranded_poly_middle.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_middle.geometry[j].buffer(500),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                stranded_poly_late.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_late.geometry[j].buffer(500),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
            distance_counter=1000
            while stranded_poly_early[data_name_month][j]==0:
                stranded_poly_early.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_early.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
            distance_counter=1000
            while stranded_poly_middle[data_name_month][j]==0:
                stranded_poly_middle.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_middle.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
            distance_counter=1000
            while stranded_poly_late[data_name_month][j]==0:
                stranded_poly_late.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_late.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        #Terminus change
        for j in range(0,len(marine_based_glacier_area_gained_early_GrIS)):
            try:
                marine_based_glacier_area_gained_early_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_early_GrIS.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_gained_early_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_early_GrIS.geometry[j].buffer(1000),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_gained_early_GrIS[data_name_month][j]==0:
                marine_based_glacier_area_gained_early_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_early_GrIS.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        for j in range(0,len(marine_based_glacier_area_lost_early_GrIS)):
            try:
                marine_based_glacier_area_lost_early_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_early_GrIS.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_lost_early_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_early_GrIS.geometry[j].buffer(1000),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_lost_early_GrIS[data_name_month][j]==0:
                marine_based_glacier_area_lost_early_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_early_GrIS.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        for j in range(0,len(marine_based_glacier_area_gained_late_GrIS)):
            try:
                marine_based_glacier_area_gained_late_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_late_GrIS.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_gained_late_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_late_GrIS.geometry[j].buffer(1000),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_gained_late_GrIS[data_name_month][j]==0:
                marine_based_glacier_area_gained_late_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_late_GrIS.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        for j in range(0,len(marine_based_glacier_area_lost_late_GrIS)):
            try:
                marine_based_glacier_area_lost_late_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_late_GrIS.geometry[j],smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_lost_late_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_late_GrIS.geometry[j].buffer(1000),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_lost_late_GrIS[data_name_month][j]==0:
                marine_based_glacier_area_lost_late_GrIS.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_late_GrIS.geometry[j].buffer(distance_counter),smb_datasets_GrIS[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
    
    
    stranded_poly_early[data_name] = stranded_poly_early[month_accum].sum(axis=1)
    stranded_poly_early.drop(columns=month_accum,inplace=True)
    stranded_poly_middle[data_name] = stranded_poly_middle[month_accum].sum(axis=1)
    stranded_poly_middle.drop(columns=month_accum,inplace=True)
    stranded_poly_late[data_name] = stranded_poly_late[month_accum].sum(axis=1)
    stranded_poly_late.drop(columns=month_accum,inplace=True)
    
    marine_based_glacier_area_gained_early_GrIS[data_name] = marine_based_glacier_area_gained_early_GrIS[month_accum].sum(axis=1)
    marine_based_glacier_area_gained_early_GrIS.drop(columns=month_accum,inplace=True)
    marine_based_glacier_area_lost_early_GrIS[data_name] = marine_based_glacier_area_lost_early_GrIS[month_accum].sum(axis=1)
    marine_based_glacier_area_lost_early_GrIS.drop(columns=month_accum,inplace=True)
    marine_based_glacier_area_gained_late_GrIS[data_name] = marine_based_glacier_area_gained_late_GrIS[month_accum].sum(axis=1)
    marine_based_glacier_area_gained_late_GrIS.drop(columns=month_accum,inplace=True)
    marine_based_glacier_area_lost_late_GrIS[data_name] = marine_based_glacier_area_lost_late_GrIS[month_accum].sum(axis=1)
    marine_based_glacier_area_lost_late_GrIS.drop(columns=month_accum,inplace=True)

stranded_poly_early.to_file('Greenland/stranded_polygons_GrIS_early_with_SMB_'+date_string+'.gpkg',driver='GPKG')
stranded_poly_middle.to_file('Greenland/stranded_polygons_GrIS_middle_with_SMB_'+date_string+'.gpkg',driver='GPKG')
stranded_poly_late.to_file('Greenland/stranded_polygons_GrIS_late_with_SMB_'+date_string+'.gpkg',driver='GPKG')

marine_based_glacier_area_gained_early_GrIS.to_file('Greenland/marine_based_glacier_area_gained_early_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_early_GrIS.to_file('Greenland/marine_based_glacier_area_lost_early_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_gained_late_GrIS.to_file('Greenland/marine_based_glacier_area_gained_late_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_late_GrIS.to_file('Greenland/marine_based_glacier_area_lost_late_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')

2000_SMB
2001_SMB
2002_SMB
2003_SMB
2004_SMB
2005_SMB
2006_SMB
2007_SMB
2008_SMB
2009_SMB
2010_SMB
2011_SMB
2012_SMB
2013_SMB
2014_SMB
2015_SMB
2016_SMB
2017_SMB
2018_SMB
2019_SMB


In [31]:
# Get SMB for GIC
date_string2='Spring2022_v12'#this needs to be for other paper
stranded_poly_early = gpd.read_file('stranded_polygons_early_v2_'+date_string2+'.gpkg',driver='GPKG')
stranded_poly_middle = gpd.read_file('stranded_polygons_middle_v2_'+date_string2+'.gpkg',driver='GPKG')
stranded_poly_late = gpd.read_file('stranded_polygons_late_v2_'+date_string2+'.gpkg',driver='GPKG')

for i in range(0,len(stranded_poly_early)):
    stranded_poly_early.loc[i,'Region']=stranded_poly_early['RGIid_2_1'][i][7]
stranded_poly_early = stranded_poly_early[stranded_poly_early['Region']=='5'].reset_index()

for i in range(0,len(stranded_poly_middle)):
    stranded_poly_middle.loc[i,'Region']=stranded_poly_middle['RGIid_2_1'][i][7]
stranded_poly_middle = stranded_poly_middle[stranded_poly_middle['Region']=='5'].reset_index()

for i in range(0,len(stranded_poly_late)):
    stranded_poly_late.loc[i,'Region']=stranded_poly_late['RGIid_2_1'][i][7]
stranded_poly_late = stranded_poly_late[stranded_poly_late['Region']=='5'].reset_index()

smb_datasets_GIC = sorted(glob.glob('/Users/willkochtitzky/Projects/Greenland_Calving/SMB/GIC/SMB_rec.20*_3995polar.tif'))
# smb_datasets_GrIS = sorted(glob.glob('/Users/willkochtitzky/Projects/Greenland_Calving/SMB/GIC/smb_rec.20*_3995polar.tif'))

for i in range(0,len(smb_datasets_GIC)):
# for i in range(0,1):
    data_name= re.search('rec.(.*).BN_1958_2013_1km.MM_smb_3413polar_3995polar.tif',smb_datasets_GIC[i]).group(1)
    data_name = data_name+'_SMB'
    month_accum=[]
    print(data_name)
    for month in range(1,13,1):
        data_name_month=data_name+str(month)
        month_accum.append(data_name_month)
        for j in range(0,len(stranded_poly_late)):
            try:
                stranded_poly_early.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_early.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
                stranded_poly_middle.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_middle.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                stranded_poly_late.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_late.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
            except:
                stranded_poly_early.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_early.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
                stranded_poly_middle.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_middle.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                stranded_poly_late.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_late.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
            distance_counter=1000
            while stranded_poly_early[data_name_month][j]==0:
                stranded_poly_early.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_early.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
            distance_counter=1000
            while stranded_poly_middle[data_name_month][j]==0:
                stranded_poly_middle.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_middle.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
            distance_counter=1000
            while stranded_poly_late[data_name_month][j]==0:
                stranded_poly_late.loc[j,data_name_month]=rasterstats.zonal_stats(stranded_poly_late.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        #Terminus change
        for j in range(0,len(marine_based_glacier_area_gained_early_GIC)):
            try:
                marine_based_glacier_area_gained_early_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_early_GIC.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_gained_early_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_early_GIC.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_gained_early_GIC[data_name_month][j]==0:
                marine_based_glacier_area_gained_early_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_early_GIC.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        for j in range(0,len(marine_based_glacier_area_lost_early_GIC)):
            try:
                marine_based_glacier_area_lost_early_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_early_GIC.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_lost_early_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_early_GIC.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_lost_early_GIC[data_name_month][j]==0:
                marine_based_glacier_area_lost_early_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_early_GIC.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        for j in range(0,len(marine_based_glacier_area_gained_late_GIC)):
            try:
                marine_based_glacier_area_gained_late_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_late_GIC.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_gained_late_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_late_GIC.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_gained_late_GIC[data_name_month][j]==0:
                marine_based_glacier_area_gained_late_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_gained_late_GIC.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
        for j in range(0,len(marine_based_glacier_area_lost_late_GIC)):
            try:
                marine_based_glacier_area_lost_late_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_late_GIC.geometry[j],smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000#divide by 1000 to convert from mm w.e. to m w.e.
            except:
                marine_based_glacier_area_lost_late_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_late_GIC.geometry[j].buffer(1000),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
            while marine_based_glacier_area_lost_late_GIC[data_name_month][j]==0:
                marine_based_glacier_area_lost_late_GIC.loc[j,data_name_month]=rasterstats.zonal_stats(marine_based_glacier_area_lost_late_GIC.geometry[j].buffer(distance_counter),smb_datasets_GIC[i], stats=['mean'],band=month)[0]['mean']/1000
                distance_counter=distance_counter+1000
                
    stranded_poly_early[data_name] = stranded_poly_early[month_accum].sum(axis=1)
    stranded_poly_early.drop(columns=month_accum,inplace=True)
    stranded_poly_middle[data_name] = stranded_poly_middle[month_accum].sum(axis=1)
    stranded_poly_middle.drop(columns=month_accum,inplace=True)
    stranded_poly_late[data_name] = stranded_poly_late[month_accum].sum(axis=1)
    stranded_poly_late.drop(columns=month_accum,inplace=True)
    
    marine_based_glacier_area_gained_early_GIC[data_name] = marine_based_glacier_area_gained_early_GIC[month_accum].sum(axis=1)
    marine_based_glacier_area_gained_early_GIC.drop(columns=month_accum,inplace=True)
    marine_based_glacier_area_lost_early_GIC[data_name] = marine_based_glacier_area_lost_early_GIC[month_accum].sum(axis=1)
    marine_based_glacier_area_lost_early_GIC.drop(columns=month_accum,inplace=True)
    marine_based_glacier_area_gained_late_GIC[data_name] = marine_based_glacier_area_gained_late_GIC[month_accum].sum(axis=1)
    marine_based_glacier_area_gained_late_GIC.drop(columns=month_accum,inplace=True)
    marine_based_glacier_area_lost_late_GIC[data_name] = marine_based_glacier_area_lost_late_GIC[month_accum].sum(axis=1)
    marine_based_glacier_area_lost_late_GIC.drop(columns=month_accum,inplace=True)

stranded_poly_early.to_file('Greenland/stranded_polygons_GIC_early_with_SMB_'+date_string+'.gpkg',driver='GPKG')
stranded_poly_middle.to_file('Greenland/stranded_polygons_GIC_middle_with_SMB_'+date_string+'.gpkg',driver='GPKG')
stranded_poly_late.to_file('Greenland/stranded_polygons_GIC_late_with_SMB_'+date_string+'.gpkg',driver='GPKG')

marine_based_glacier_area_gained_early_GIC.to_file('Greenland/marine_based_glacier_area_gained_early_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_early_GIC.to_file('Greenland/marine_based_glacier_area_lost_early_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_gained_late_GIC.to_file('Greenland/marine_based_glacier_area_gained_late_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_late_GIC.to_file('Greenland/marine_based_glacier_area_lost_late_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')

2000_SMB
2001_SMB
2002_SMB
2003_SMB
2004_SMB
2005_SMB
2006_SMB
2007_SMB
2008_SMB
2009_SMB
2010_SMB
2011_SMB
2012_SMB
2013_SMB
2014_SMB
2015_SMB
2016_SMB
2017_SMB
2018_SMB
2019_SMB


In [32]:
#calculations for terminus change melt
marine_based_glacier_area_gained_early_GIC = gpd.read_file('Greenland/marine_based_glacier_area_gained_early_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_early_GIC = gpd.read_file('Greenland/marine_based_glacier_area_lost_early_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_gained_late_GIC = gpd.read_file('Greenland/marine_based_glacier_area_gained_late_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_late_GIC = gpd.read_file('Greenland/marine_based_glacier_area_lost_late_GIC_with_SMB_'+date_string+'.gpkg',driver='GPKG')

marine_based_glacier_area_gained_early_GrIS = gpd.read_file('Greenland/marine_based_glacier_area_gained_early_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_early_GrIS = gpd.read_file('Greenland/marine_based_glacier_area_lost_early_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_gained_late_GrIS = gpd.read_file('Greenland/marine_based_glacier_area_gained_late_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')
marine_based_glacier_area_lost_late_GrIS = gpd.read_file('Greenland/marine_based_glacier_area_lost_late_GrIS_with_SMB_'+date_string+'.gpkg',driver='GPKG')

marine_based_glacier_area_gained_early_GIC['terminus_melt_early'] = marine_based_glacier_area_gained_early_GIC[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
marine_based_glacier_area_lost_early_GIC['terminus_melt_early'] = marine_based_glacier_area_lost_early_GIC[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
marine_based_glacier_area_gained_late_GIC['terminus_melt_late'] = marine_based_glacier_area_gained_late_GIC[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)
marine_based_glacier_area_lost_late_GIC['terminus_melt_late'] = marine_based_glacier_area_lost_late_GIC[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)

marine_based_glacier_area_gained_early_GrIS['terminus_melt_early'] = marine_based_glacier_area_gained_early_GrIS[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
marine_based_glacier_area_lost_early_GrIS['terminus_melt_early'] = marine_based_glacier_area_lost_early_GrIS[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
marine_based_glacier_area_gained_late_GrIS['terminus_melt_late'] = marine_based_glacier_area_gained_late_GrIS[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)
marine_based_glacier_area_lost_late_GrIS['terminus_melt_late'] = marine_based_glacier_area_lost_late_GrIS[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)

terminus_change_melt_early_GIC = marine_based_glacier_area_lost_early_GIC.append(marine_based_glacier_area_gained_early_GIC).dissolve('NAME',aggfunc='mean').reset_index()
terminus_change_melt_late_GIC = marine_based_glacier_area_lost_late_GIC.append(marine_based_glacier_area_gained_late_GIC).dissolve('NAME',aggfunc='mean').reset_index()

terminus_change_melt_early_GrIS = marine_based_glacier_area_lost_early_GrIS.append(marine_based_glacier_area_gained_early_GrIS).dissolve('NAME',aggfunc='mean').reset_index()
terminus_change_melt_late_GrIS = marine_based_glacier_area_lost_late_GrIS.append(marine_based_glacier_area_gained_late_GrIS).dissolve('NAME',aggfunc='mean').reset_index()

terminus_change_melt_early = terminus_change_melt_early_GrIS.append(terminus_change_melt_early_GIC).reset_index(drop=True)
terminus_change_melt_late = terminus_change_melt_late_GrIS.append(terminus_change_melt_late_GIC).reset_index(drop=True)

terminus_change_melt = terminus_change_melt_early.merge(terminus_change_melt_late,on='NAME',how='left')[['NAME','terminus_melt_early','terminus_melt_late']]

terminus_change_melt['terminus_melt_early_err']=terminus_change_melt['terminus_melt_early']*0.2
terminus_change_melt['terminus_melt_late_err']=terminus_change_melt['terminus_melt_late']*0.2

terminus_change_melt['terminus_melt_total'] = np.mean(terminus_change_melt['terminus_melt_early']+terminus_change_melt['terminus_melt_late'])
terminus_change_melt['terminus_melt_total_err'] = terminus_change_melt['terminus_melt_total']*0.2

In [33]:
#calculations for stranded polygons and export combined dataset
stranded_poly_early_GIC = gpd.read_file('Greenland/stranded_polygons_GIC_early_with_SMB_'+date_string+'.gpkg')
stranded_poly_middle_GIC = gpd.read_file('Greenland/stranded_polygons_GIC_middle_with_SMB_'+date_string+'.gpkg')
stranded_poly_late_GIC = gpd.read_file('Greenland/stranded_polygons_GIC_late_with_SMB_'+date_string+'.gpkg')

stranded_poly_early_GrIS = gpd.read_file('Greenland/stranded_polygons_GrIS_early_with_SMB_'+date_string+'.gpkg')
stranded_poly_middle_GrIS = gpd.read_file('Greenland/stranded_polygons_GrIS_middle_with_SMB_'+date_string+'.gpkg')
stranded_poly_late_GrIS = gpd.read_file('Greenland/stranded_polygons_GrIS_late_with_SMB_'+date_string+'.gpkg')

stranded_poly_early_GIC = stranded_poly_early_GIC.set_crs('EPSG:3995',allow_override=True)
stranded_poly_middle_GIC = stranded_poly_middle_GIC.set_crs('EPSG:3995',allow_override=True)
stranded_poly_late_GIC = stranded_poly_late_GIC.set_crs('EPSG:3995',allow_override=True)

stranded_poly_early_GrIS = stranded_poly_early_GrIS.set_crs('EPSG:3995',allow_override=True)
stranded_poly_middle_GrIS = stranded_poly_middle_GrIS.set_crs('EPSG:3995',allow_override=True)
stranded_poly_late_GrIS = stranded_poly_late_GrIS.set_crs('EPSG:3995',allow_override=True)

# stranded_poly_early_GIC['area_below_fluxgate_early'] = stranded_poly_early_GIC.area/1000000
# stranded_poly_middle_GIC['area_below_fluxgate_middle'] = stranded_poly_middle_GIC.area/1000000
# stranded_poly_late_GIC['area_below_fluxgate_late'] = stranded_poly_late_GIC.area/1000000
# stranded_poly_early_GrIS['area_below_fluxgate_early'] = stranded_poly_early_GrIS.area/1000000
# stranded_poly_middle_GrIS['area_below_fluxgate_middle'] = stranded_poly_middle_GrIS.area/1000000
# stranded_poly_late_GrIS['area_below_fluxgate_late'] = stranded_poly_late_GrIS.area/1000000

stranded_poly_early_GIC['geometry_3995'] = stranded_poly_early_GIC['geometry']
stranded_poly_early_GIC.loc[:,'latlon'] = stranded_poly_early_GIC.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(stranded_poly_early_GIC)):
    projection = ccrs.Orthographic(central_longitude=stranded_poly_early_GIC.loc[i]['latlon'].centroid.x, central_latitude=stranded_poly_early_GIC.loc[i]['latlon'].centroid.y)
    stranded_poly_early_GIC = stranded_poly_early_GIC.to_crs(projection)
    stranded_poly_early_GIC.loc[i,'area_below_fluxgate_early']=stranded_poly_early_GIC.loc[i].geometry.area/1000000
stranded_poly_early_GIC.loc[:,'geometry'] = stranded_poly_early_GIC['geometry_3995']

stranded_poly_middle_GIC['geometry_3995'] = stranded_poly_middle_GIC['geometry']
stranded_poly_middle_GIC.loc[:,'latlon'] = stranded_poly_middle_GIC.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(stranded_poly_middle_GIC)):
    projection = ccrs.Orthographic(central_longitude=stranded_poly_middle_GIC.loc[i]['latlon'].centroid.x, central_latitude=stranded_poly_middle_GIC.loc[i]['latlon'].centroid.y)
    stranded_poly_middle_GIC = stranded_poly_middle_GIC.to_crs(projection)
    stranded_poly_middle_GIC.loc[i,'area_below_fluxgate_middle']=stranded_poly_middle_GIC.loc[i].geometry.area/1000000
stranded_poly_middle_GIC.loc[:,'geometry'] = stranded_poly_middle_GIC['geometry_3995']

stranded_poly_late_GIC['geometry_3995'] = stranded_poly_late_GIC['geometry']
stranded_poly_late_GIC.loc[:,'latlon'] = stranded_poly_late_GIC.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(stranded_poly_early_GIC)):
    projection = ccrs.Orthographic(central_longitude=stranded_poly_late_GIC.loc[i]['latlon'].centroid.x, central_latitude=stranded_poly_late_GIC.loc[i]['latlon'].centroid.y)
    stranded_poly_late_GIC = stranded_poly_late_GIC.to_crs(projection)
    stranded_poly_late_GIC.loc[i,'area_below_fluxgate_late']=stranded_poly_late_GIC.loc[i].geometry.area/1000000
stranded_poly_late_GIC.loc[:,'geometry'] = stranded_poly_late_GIC['geometry_3995']

stranded_poly_early_GrIS['geometry_3995'] = stranded_poly_early_GrIS['geometry']
stranded_poly_early_GrIS.loc[:,'latlon'] = stranded_poly_early_GrIS.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(stranded_poly_early_GrIS)):
    projection = ccrs.Orthographic(central_longitude=stranded_poly_early_GrIS.loc[i]['latlon'].centroid.x, central_latitude=stranded_poly_early_GrIS.loc[i]['latlon'].centroid.y)
    stranded_poly_early_GrIS = stranded_poly_early_GrIS.to_crs(projection)
    stranded_poly_early_GrIS.loc[i,'area_below_fluxgate_early']=stranded_poly_early_GrIS.loc[i].geometry.area/1000000
stranded_poly_early_GrIS.loc[:,'geometry'] = stranded_poly_early_GrIS['geometry_3995']

stranded_poly_middle_GrIS['geometry_3995'] = stranded_poly_middle_GrIS['geometry']
stranded_poly_middle_GrIS.loc[:,'latlon'] = stranded_poly_middle_GrIS.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(stranded_poly_middle_GrIS)):
    projection = ccrs.Orthographic(central_longitude=stranded_poly_middle_GrIS.loc[i]['latlon'].centroid.x, central_latitude=stranded_poly_middle_GrIS.loc[i]['latlon'].centroid.y)
    stranded_poly_middle_GrIS = stranded_poly_middle_GrIS.to_crs(projection)
    stranded_poly_middle_GrIS.loc[i,'area_below_fluxgate_middle']=stranded_poly_middle_GrIS.loc[i].geometry.area/1000000
stranded_poly_middle_GrIS.loc[:,'geometry'] = stranded_poly_middle_GrIS['geometry_3995']

stranded_poly_late_GrIS['geometry_3995'] = stranded_poly_late_GrIS['geometry']
stranded_poly_late_GrIS.loc[:,'latlon'] = stranded_poly_late_GrIS.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(stranded_poly_early_GrIS)):
    projection = ccrs.Orthographic(central_longitude=stranded_poly_late_GrIS.loc[i]['latlon'].centroid.x, central_latitude=stranded_poly_late_GrIS.loc[i]['latlon'].centroid.y)
    stranded_poly_late_GrIS = stranded_poly_late_GrIS.to_crs(projection)
    stranded_poly_late_GrIS.loc[i,'area_below_fluxgate_late']=stranded_poly_late_GrIS.loc[i].geometry.area/1000000
stranded_poly_late_GrIS.loc[:,'geometry'] = stranded_poly_late_GrIS['geometry_3995']

stranded_poly_early_GIC['early_poly_early_melt']=stranded_poly_early_GIC[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
stranded_poly_middle_GIC['middle_poly_early_melt']=stranded_poly_middle_GIC[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
stranded_poly_middle_GIC['middle_poly_late_melt']=stranded_poly_middle_GIC[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)
stranded_poly_late_GIC['late_poly_late_melt']=stranded_poly_late_GIC[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)

stranded_poly_early_GrIS['early_poly_early_melt']=stranded_poly_early_GrIS[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
stranded_poly_middle_GrIS['middle_poly_early_melt']=stranded_poly_middle_GrIS[['2000_SMB', '2001_SMB', '2002_SMB', '2003_SMB','2004_SMB', '2005_SMB', '2006_SMB', '2007_SMB', '2008_SMB', '2009_SMB']].mean(axis=1)
stranded_poly_middle_GrIS['middle_poly_late_melt']=stranded_poly_middle_GrIS[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)
stranded_poly_late_GrIS['late_poly_late_melt']=stranded_poly_late_GrIS[['2010_SMB', '2011_SMB', '2012_SMB', '2013_SMB', '2014_SMB', '2015_SMB','2016_SMB', '2017_SMB', '2018_SMB', '2019_SMB']].mean(axis=1)

stranded_poly_early_GIC.rename(columns={"RGIid_2_1":'NAME'},inplace=True)
stranded_poly_middle_GIC.rename(columns={"RGIid_2_1":'NAME'},inplace=True)
stranded_poly_late_GIC.rename(columns={"RGIid_2_1":'NAME'},inplace=True)
print(stranded_poly_late_GIC.duplicated(subset='NAME').value_counts())

stranded_poly_early = stranded_poly_early_GIC.append(stranded_poly_early_GrIS)[['NAME','early_poly_early_melt','area_below_fluxgate_early']]
stranded_poly_middle = stranded_poly_middle_GIC.append(stranded_poly_middle_GrIS)[['NAME','middle_poly_early_melt','middle_poly_late_melt','area_below_fluxgate_middle']]
stranded_poly_late = stranded_poly_late_GIC.append(stranded_poly_late_GrIS).reset_index()
print(stranded_poly_late.duplicated(subset='NAME').value_counts())

stranded_poly_late['easting']=stranded_poly_late.centroid.x
stranded_poly_late['northing']=stranded_poly_late.centroid.y
easting=[]
northing=[]
for i in range(0,len(stranded_poly_late)):
    easting.append(stranded_poly_late['easting'][i])
    northing.append(stranded_poly_late['northing'][i])

#add lat lon in decimal degrees
inProj = Proj(init='epsg:3995')
outProj = Proj(init='epsg:4326')    
stranded_poly_late.loc[:,'lon'],stranded_poly_late.loc[:,'lat'] = transform(inProj,outProj,easting,northing)

stranded_poly_late=stranded_poly_late[['NAME','late_poly_late_melt','area_below_fluxgate_late','lat','lon']]
print(stranded_poly_late.duplicated(subset='NAME').value_counts())

stranded_poly = stranded_poly_early.merge(stranded_poly_middle).merge(stranded_poly_late)#[['NAME','early_poly_early_melt','middle_poly_early_melt','middle_poly_late_melt','late_poly_late_melt']]

#assign errors, Brice says it should be 20% for ablation area
stranded_poly['early_poly_early_melt_err']=stranded_poly['early_poly_early_melt']*0.2
stranded_poly['middle_poly_early_melt_err']=stranded_poly['middle_poly_early_melt']*0.2
stranded_poly['middle_poly_late_melt_err']=stranded_poly['middle_poly_late_melt']*0.2
stranded_poly['late_poly_late_melt_err']=stranded_poly['late_poly_late_melt']*0.2

combined_melt_df = gpd.GeoDataFrame(stranded_poly, geometry=gpd.points_from_xy(stranded_poly.lon, stranded_poly.lat),crs='EPSG:4326')
combined_melt_df=combined_melt_df.merge(terminus_change_melt,on='NAME')
combined_melt_df.to_file('Greenland/Combined_pt_melt_data.gpkg', driver='GPKG')
combined_melt_df.head(2)

False    537
dtype: int64
False    750
dtype: int64
False    750
dtype: int64


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:104: DeprecationWarning: This function is deprecated. See: htt

,NAME,early_poly_early_melt,area_below_fluxgate_early,middle_poly_early_melt,middle_poly_late_melt,area_below_fluxgate_middle,late_poly_late_melt,area_below_fluxgate_late,lat,lon,early_poly_early_melt_err,middle_poly_early_melt_err,middle_poly_late_melt_err,late_poly_late_melt_err,geometry,terminus_melt_early,terminus_melt_late,terminus_melt_early_err,terminus_melt_late_err,terminus_melt_total,terminus_melt_total_err
0,RGI60-05.00268,-0.186827,0.153594,-0.200171,-0.202764,0.151663,-0.202764,0.150968,65.877302,-52.516417,-0.037365,-0.040034,-0.040553,-0.040553,POINT (-52.51642 65.87730),-0.107014,-0.207201,-0.021403,-0.041440,-1.932303,-0.386461
1,RGI60-05.00310,-0.054344,0.174731,-0.054344,-0.098851,0.172717,-0.098851,0.163484,65.971130,-52.081720,-0.010869,-0.010869,-0.019770,-0.019770,POINT (-52.08172 65.97113),0.022804,-0.217307,0.004561,-0.043461,-1.932303,-0.386461


In [34]:
Arctic_discharge_fluxes = gpd.read_file('Greenland/Greenland_discharge_fluxes'+date_string+'.gpkg')
combined_melt_df = gpd.read_file('Greenland/Combined_pt_melt_data.gpkg')

print(len(Arctic_discharge_fluxes))
print(len(combined_melt_df))

set(Arctic_discharge_fluxes['name']).difference(combined_melt_df['NAME'])
# set(combined_melt_df['NAME']).difference(Arctic_discharge_fluxes['name'])

750
750


set()

In [35]:
#define functions for Romain
# Define the NMAD as a robust measure of dispersion
def nmad(data: np.ndarray, nfact: float = 1.4826) -> float:
    """
    Calculate the normalized median absolute deviation (NMAD) of an array.

    :param data: input data
    :param nfact: normalization factor for the data; default is 1.4826

    :returns nmad: (normalized) median absolute deviation of data.
    """
    data_arr = np.asarray(data)
    return nfact * np.nanmedian(np.abs(data_arr - np.nanmedian(data_arr)))
# Function for a sum of spherical variogram models
def sum_vgm(h, params, error1, error2):
    """Sum of variograms models"""
    v = 0
    for k in range(len(params)):
        v += skgstat.models.spherical(h, params[k][0], params[k][1] * error1 * error2)

    return v

# Function to integrate the variogram from a spatial support to another: a double sum of covariances
def double_sum_covar(coords: np.ndarray, areas: np.ndarray, errors: np.ndarray,
                     vgm_func: Callable):
    """
    Double sum of covariance for euclidean coordinates
    :param coords: Spatial support (typically, pixel) coordinates
    :param areas:  Area of supports
    :param errors: Standard errors of supports
    :param vgm_func: Variogram function
    :return:
    """
    n = len(coords)
    pds = pdist(coords)
    var = 0
    for i in range(n):
        for j in range(n):
            # For index calculation of the pairwise distance, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
            if i == j:
                d = 0
            elif i < j:
                ind = n * i + j - ((i + 2) * (i + 1)) // 2
                d = pds[ind]
            else:
                ind = n * j + i - ((j + 2) * (j + 1)) // 2
                d = pds[ind]

            var += errors[i] * errors[j] * (1 - vgm_func(d)) * areas[i] * areas[j]

    total_area = sum(areas)
    se_dsc = np.sqrt(var / total_area ** 2)

    return se_dsc

# Standardized variogram function (percentage of correlation)
def vgm_func_gla(h):
    v = sum_vgm(h, params=params_icethickness, error1=1, error2=1)
    # Account for percentage partial sills
    return v/100



In [36]:
#finish frontal ablation calculations
net_area_change = pd.read_pickle('Greenland/net_area_change'+date_string+'.pkl')#read area change data so that you dont need to rerun code above
Arctic_discharge_fluxes = gpd.read_file('Greenland/Greenland_discharge_fluxes'+date_string+'.gpkg', driver="GPKG")
Arctic_discharge_fluxes_with_area = pd.merge(Arctic_discharge_fluxes,net_area_change,left_on='name',right_on='NAME',how='left')#combine datasets
combined_melt_df = gpd.read_file('Greenland/Combined_pt_melt_data.gpkg')
Arctic_discharge_fluxes_with_area = pd.merge(Arctic_discharge_fluxes_with_area,combined_melt_df,how='left',left_on='name',right_on='NAME')

for period in ['early','late','total']:
    print('Running for period: '+period)
    # 0/ Perform the bias correction for ice thickness
    print('Correcting ice thickness bias per ice thickness range...')

    for area_loc in ['lost','gained']:
        if (area_loc=='gained')&(period=='early'):
            Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased']=Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period]
            Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period+'_debiased']=Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period]
            print('SKIP early gained')
        else:
            print('----',area_loc)

            oib = Arctic_discharge_fluxes_with_area['average_OIB_thickness_'+area_loc+'_'+period].values
            best_meas_icethickness = []
            for i in range(len(Arctic_discharge_fluxes_with_area)):
                if np.isfinite(oib[i]):
                    best_meas_icethickness.append(oib[i])
                else:
                    best_meas_icethickness.append(np.nan)
            best_meas_icethickness = np.array(best_meas_icethickness)

    #Debias Millan
            Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period]>3000,'average_Milan_thickness_'+area_loc+'_'+period]=np.nan
            Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period]<0,'average_Milan_thickness_'+area_loc+'_'+period]=np.nan

            # Modelled
            best_model_icethickness = Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period].values

            it_diff = best_meas_icethickness - best_model_icethickness
            it_diff[np.abs(it_diff)>3000] = np.nan

            print('Median Millan bias before:' + str(np.nanmean(it_diff)))
            print('mean Millan values before:', np.nanmean(best_model_icethickness))
            df_bin = xdem.spatialstats.nd_binning(it_diff, list_var=[best_model_icethickness], list_var_names=['ice_thickness'], list_var_bins=20)

            df_bin = df_bin[np.isfinite(df_bin.nanmedian)]
            y = df_bin.nanmedian.values
            x = pd.IntervalIndex(df_bin.ice_thickness).mid.values
            p = np.polyfit(x[np.isfinite(y)], y[np.isfinite(y)], deg=1)

            # Derive the debiased modelled ice thickness
            Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased'] = Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period] \
                                                                             + np.polyval(p, Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period])

            print('Median Millan bias after:' +str(np.nanmean(best_meas_icethickness - Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased'])))
            print('Mean Millan values after:',Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased'].mean())
            
            #make negative values =0
            Arctic_discharge_fluxes_with_area[Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased']<0]['average_Milan_thickness_'+area_loc+'_'+period+'_debiased']=0
            
            
#     #Debias BedMachine
#             Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period]>5000,'average_bedmachine_thickness_'+area_loc+'_'+period]=np.nan
#             Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period]<0,'average_bedmachine_thickness_'+area_loc+'_'+period]=np.nan

#             # Modelled
#             best_model_icethickness = Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period].values

#             it_diff = best_meas_icethickness - best_model_icethickness
#             it_diff[np.abs(it_diff)>3000] = np.nan

#             print('Median BedMachine bias before:' + str(np.nanmean(it_diff)))
#             print('mean BedMachine values before:', np.nanmean(best_model_icethickness))
#             df_bin = xdem.spatialstats.nd_binning(it_diff, list_var=[best_model_icethickness], list_var_names=['ice_thickness'], list_var_bins=20)

#             df_bin = df_bin[np.isfinite(df_bin.nanmedian)]
#             y = df_bin.nanmedian.values
#             x = pd.IntervalIndex(df_bin.ice_thickness).mid.values
#             p = np.polyfit(x[np.isfinite(y)], y[np.isfinite(y)], deg=1)

#             # Derive the debiased modelled ice thickness
#             Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period+'_debiased'] = Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period] \
#                                                                              + np.polyval(p, Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period])

#             print('Median BedMachine bias after:' +str(np.nanmean(best_meas_icethickness - Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period+'_debiased'])))
#             print('Mean BedMachine values after:',Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_'+area_loc+'_'+period+'_debiased'].mean())

print('sub len',len(Arctic_discharge_fluxes_with_area[Arctic_discharge_fluxes_with_area.duplicated(subset='name')]))

Running for period: early
Correcting ice thickness bias per ice thickness range...
---- lost
Median Millan bias before:-101.45081242712602
mean Millan values before: 63.25104083941808
Median Millan bias after:1.5348871217068318
Mean Millan values after: 93.22327162085094
SKIP early gained
Running for period: late
Correcting ice thickness bias per ice thickness range...
---- lost
Median Millan bias before:-96.7014358443532
mean Millan values before: 85.23840632626737
Median Millan bias after:-3.7450385633398025
Mean Millan values after: 41.819471005652005
---- gained
Median Millan bias before:-142.98015573686374
mean Millan values before: 78.1546488937796
Median Millan bias after:1.8258005971260047
Mean Millan values after: 128.748366382053
Running for period: total
Correcting ice thickness bias per ice thickness range...
---- lost
Median Millan bias before:-72.24398804280445
mean Millan values before: 85.32076934589654
Median Millan bias after:4.2480203291362955
Mean Millan values afte

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [37]:
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_source']=np.nan
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_source']=np.nan
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_source']=np.nan
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_source']=np.nan

#Find the best thickness data for areas gained/lost by preference:
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_early'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_early']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_early_err'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_early']*0.1
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_early'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_early']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_early_err'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_early']*0.1
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_late'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_late']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_late_err'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_late']*0.1
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_late'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_late']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_late_err'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_late']*0.1
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_total'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_total']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_total_err'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_total']*0.1
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_total'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_total']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_total_err'] = Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_total']*0.1

print('OIB obs')
print('thickness_for_area_lost_early',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].count())
print('thickness_for_area_gained_early',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].count())
print('thickness_for_area_lost_late',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].count())
print('thickness_for_area_gained_late',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].count())
print('thickness_for_area_lost_late mean',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].mean())

# Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_early'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_early_err'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_early'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_early_err'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_late'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_late_err'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_late'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_late_err'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_total'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_total_err'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_total'], inplace=True)
# Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_total_err'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_lost_early'])
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_lost_early_err'])
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_gained_early'])
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_gained_early_err'])
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_lost_late'])
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_lost_late_err'])
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_gained_late'])
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_gained_late_err'])
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_lost_total'])
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_lost_total_err'])
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_gained_total'])
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_bedmachine_thickness_gained_total_err'])

print('bed machine')
print('thickness_for_area_lost_early',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].count())
print('thickness_for_area_gained_early',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].count())
print('thickness_for_area_lost_late',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].count())
print('thickness_for_area_gained_late',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].count())
print('thickness_for_area_lost_late mean',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].mean())

Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['best_thickness_early_mean']<0,'best_thickness_early_mean']=0
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['best_thickness_late_mean']<0,'best_thickness_late_mean']=0

#If we dont have any data, fill it with 60±30%
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early_mean']*0.60)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early_mean']*0.60)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late_mean']*0.60)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late_mean']*0.60)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early_mean']+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'])/2*0.60)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early_mean']+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'])/2*0.60)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early_mean']*0.30)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early_mean']*0.30)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late_mean']*0.30)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late_mean']*0.30)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early_mean']+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'])/2*0.30)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early_mean']+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'])/2*0.30)

print('filled')
print('thickness_for_area_lost_early',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].count())
print('thickness_for_area_gained_early',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].count())
print('thickness_for_area_lost_late',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].count())
print('thickness_for_area_gained_late',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].count())
print('thickness_for_area_lost_late mean',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].mean())

print(len(Arctic_discharge_fluxes_with_area))
bad_thickness_glaciers=0
for i in range(0, len(Arctic_discharge_fluxes_with_area)):
    for period in ['early','late']:
        for var in ['thickness_for_area_lost_'+period,'thickness_for_area_gained_'+period]:
            if (Arctic_discharge_fluxes_with_area[var][i]<10):
                Arctic_discharge_fluxes_with_area.loc[i,var]=Arctic_discharge_fluxes_with_area['best_thickness_'+period][i]*0.60
                Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=Arctic_discharge_fluxes_with_area['best_thickness_'+period][i]*0.30
                bad_thickness_glaciers = bad_thickness_glaciers+1
                Arctic_discharge_fluxes_with_area.loc[i,var+'_source']=1
            if Arctic_discharge_fluxes_with_area[var][i]>4000:
                print(Arctic_discharge_fluxes_with_area['Name_x'][i],' has a very large thickness for terminus change',var,Arctic_discharge_fluxes_with_area[var][i])
                Arctic_discharge_fluxes_with_area.loc[i,var]=Arctic_discharge_fluxes_with_area['best_thickness_'+period+'_mean'][i]*0.60
                Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=Arctic_discharge_fluxes_with_area['best_thickness_'+period+'_mean'][i]*0.30
                Arctic_discharge_fluxes_with_area.loc[i,var+'_source']=1
        if (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]+1>Arctic_discharge_fluxes_with_area['best_thickness_early'][i])&(Arctic_discharge_fluxes_with_area['net_area_change_early'][i]>0):
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.60
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.30
            print(Arctic_discharge_fluxes_with_area['name'][i],Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i],Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i])
            bad_thickness_glaciers = bad_thickness_glaciers+1
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_source']=1
        if (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]+1>Arctic_discharge_fluxes_with_area['best_thickness_late'][i])&(Arctic_discharge_fluxes_with_area['net_area_change_late'][i]>0):
#             print('thickness for area gained late and best thickness',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i],Arctic_discharge_fluxes_with_area['best_thickness_late'][i])
#             print(Arctic_discharge_fluxes_with_area['Late_satellite_id'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.60
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late_err']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.30
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late_source']=1
            bad_thickness_glaciers = bad_thickness_glaciers+1
print('filled2')
print('thickness_for_area_lost_early',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].count())
print('thickness_for_area_gained_early',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].count())
print('thickness_for_area_lost_late',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].count())
print('thickness_for_area_gained_late',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].count())
print('thickness_for_area_lost_late mean',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].mean())

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_source']=np.nan
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_source']=np.nan
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_source']=np.nan
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_source']=np.nan

#fill with 30 ± 20 if still no data
bad_thickness_glaciers2=0
for i in range(0, len(Arctic_discharge_fluxes_with_area)):
    for period in ['early','late']:
        for var in ['thickness_for_area_lost_'+period,'thickness_for_area_gained_'+period]:
            if Arctic_discharge_fluxes_with_area[var][i]<30:
                Arctic_discharge_fluxes_with_area.loc[i,var]=30
                Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=20
                Arctic_discharge_fluxes_with_area.loc[i,var+'_source']=2
                bad_thickness_glaciers2 = bad_thickness_glaciers2+1
print('glaciers that used 30 ± 20',bad_thickness_glaciers2)
            
for i in range(0, len(Arctic_discharge_fluxes_with_area)): 
    for var in ['thickness_for_area_lost_total','thickness_for_area_gained_total']:
        if Arctic_discharge_fluxes_with_area[var][i]>4000:
            print(Arctic_discharge_fluxes_with_area['name'][i],' has a very large thickness for TOTAL terminus change',var,Arctic_discharge_fluxes_with_area[var][i])
            Arctic_discharge_fluxes_with_area.loc[i,var]=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.60
            Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.30

print('glaciers that have their thickness replaced with 60±30% at the terminus:', bad_thickness_glaciers)

print('filled3')
print('thickness_for_area_lost_early',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].count())
print('thickness_for_area_gained_early',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].count())
print('thickness_for_area_lost_late',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].count())
print('thickness_for_area_gained_late',Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].count())
print('thickness_for_area_lost_late mean',Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].mean())

#If the thickness err is greater than the signal, make the error the signal for the terminus volume change calculations
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_total_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_total_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'][i]


# #Use the fluxgate mean thickness instead of BedMachine results if we have observations at the fluxgate but not in the changed area
# for i in range(0,len(Arctic_discharge_fluxes_with_area)):
#     if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i]==1)&(math.isnan(Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_early'][i])):
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early']=Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]*0.6
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early_err']=Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]*0.3
#     if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i]==1)&(math.isnan(Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_early'][i])):
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early']=Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]*0.6
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]*0.3
#     if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i]==1)&(math.isnan(Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_late'][i])):
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late']=Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i]*0.6
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late']=Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i]*0.3
#     if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i]==1)&(math.isnan(Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_late'][i])):
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late']=Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i]*0.6
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late']=Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i]*0.3
#     if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i]==1)&(math.isnan(Arctic_discharge_fluxes_with_area['average_OIB_thickness_lost_total'][i])):
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_total']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.6
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_total']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.3
#     if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i]==1)&(math.isnan(Arctic_discharge_fluxes_with_area['average_OIB_thickness_gained_total'][i])):
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_total']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.6
#         Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_total']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.3


OIB obs
thickness_for_area_lost_early 69
thickness_for_area_gained_early 15
thickness_for_area_lost_late 89
thickness_for_area_gained_late 21
thickness_for_area_lost_late mean 192.3669972296891
bed machine
thickness_for_area_lost_early 608
thickness_for_area_gained_early 92
thickness_for_area_lost_late 570
thickness_for_area_gained_late 112
thickness_for_area_lost_late mean 47.283046021298404
filled
thickness_for_area_lost_early 750
thickness_for_area_gained_early 750
thickness_for_area_lost_late 750
thickness_for_area_gained_late 750
thickness_for_area_lost_late mean 39.58296102502059
750
QAJUUTTAP_SERMIA 453.68834188814486 272.21300513288685
RGI60-05.00814 2.0525631928012995 1.2315379156807797
RGI60-05.00814 2.0525631928012995 1.2315379156807797
RGI60-05.04906 0.0 0.0
RGI60-05.04906 0.0 0.0
RGI60-05.04951 0.0 0.0
RGI60-05.04951 0.0 0.0
RGI60-05.05048 2.394027442569061 1.4364164655414364
RGI60-05.05048 2.394027442569061 1.4364164655414364
RGI60-05.06320 0.0 0.0
RGI60-05.06320 0.0 0.0


In [38]:
for period in ['early','late']:
    for var in ['thickness_for_area_lost_'+period,'thickness_for_area_gained_'+period]:
        print(Arctic_discharge_fluxes_with_area.groupby(var+'_source').count()['name'])
        print(var)

thickness_for_area_lost_early_source
2.0    465
Name: name, dtype: int64
thickness_for_area_lost_early
thickness_for_area_gained_early_source
2.0    455
Name: name, dtype: int64
thickness_for_area_gained_early
thickness_for_area_lost_late_source
2.0    515
Name: name, dtype: int64
thickness_for_area_lost_late
thickness_for_area_gained_late_source
2.0    516
Name: name, dtype: int64
thickness_for_area_gained_late


In [39]:
#differentiate between ice sheet and mountain glacier
Arctic_discharge_fluxes_with_area['Glacier_type']=np.nan
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['name'][i][0:3]=='RGI':
        Arctic_discharge_fluxes_with_area.loc[i,'Glacier_type']='mountain'
    else:
        Arctic_discharge_fluxes_with_area.loc[i,'Glacier_type']='ice sheet'

In [40]:
for section in [0,1,2]:
    if section == 0:
        print('-----------------------All Greenland-----------------------')
        #If you want all Greenland together
        Arctic_discharge_fluxes_with_area['region']='Greenland'
        regionsornot='all_Greenland_'

    if section == 1:
        print('-----------------------Greenland ice sheet and periphery-----------------------')
        #If you want periphery and ice sheet
        Arctic_discharge_fluxes_with_area['region']=Arctic_discharge_fluxes_with_area['Glacier_type']
        regionsornot='seperated_Greenland_'

    if section == 2:
        print('-----------------------Greenland regions-----------------------')
        #if you want regions of Greenland (NE, SW, etc.)
        greenland_aois = gpd.read_file('/Users/willkochtitzky/Projects/Greenland_Calving/Greenland_regions.gpkg')
        Arctic_discharge_fluxes_with_area['geometry']=Arctic_discharge_fluxes_with_area['geometry_x']
        Arctic_discharge_fluxes_with_area = gpd.sjoin(Arctic_discharge_fluxes_with_area,greenland_aois)
        Arctic_discharge_fluxes_with_area = Arctic_discharge_fluxes_with_area.drop(columns='index_right')
        regionsornot='regional_Greenland_'
        Arctic_discharge_fluxes_with_area['region']=Arctic_discharge_fluxes_with_area['Greenland_region_name']


    Arctic_discharge_fluxes_with_area.drop_duplicates(subset='name',keep='first',inplace=True)

    #calculate the spatial correlation of the terminus change volume error from Romain
    for period in ['early','late','total']:
        print('Running for period: '+period)
        # 1/ Derive uncertainty for pixel -> glacier based on the area of averaging (lost or gained)
        # Get variables of interest: ice thickness

        if period == 'early':
            dt = Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'].values
        else:
            dt = Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'].values

        # Lost
        meanHl = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period].values
        meanHl_err_fullycorr = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'].values
        dSldt = Arctic_discharge_fluxes_with_area['area_km2_lost_' + period].values / dt
        dSldt = np.where(dSldt==np.inf,np.nan,dSldt)
        dSldt = np.where(np.isnan(dSldt),0,dSldt)

        # Remove aberrantly large uncertainty values for ice thickness uncertainty
        meanHl_err_fullycorr[np.abs(meanHl_err_fullycorr - np.nanmedian(meanHl_err_fullycorr)) > 5 * nmad(meanHl_err_fullycorr)] = np.nan
        # Replace no data that might exist in the uncertainty from the average uncertainty
        meanHl_err_fullycorr[~np.isfinite(meanHl_err_fullycorr)] = np.nanmean(meanHl_err_fullycorr)

        # Gained
        meanHg = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period].values
        meanHg_err_fullycorr = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'].values
        dSgdt = Arctic_discharge_fluxes_with_area['area_km2_gained_' + period].values / dt
        dSgdt = np.where(dSgdt==np.inf,np.nan,dSgdt)
        dSgdt = np.where(np.isnan(dSgdt),0,dSgdt)

        # Remove aberrantly large uncertainty values for ice thickness uncertainty
        meanHg_err_fullycorr[np.abs(meanHg_err_fullycorr - np.nanmedian(meanHg_err_fullycorr)) > 5 * nmad(meanHg_err_fullycorr)] = np.nan
        # Replace no data that might exist in the uncertainty from the average uncertainty
        meanHg_err_fullycorr[~np.isfinite(meanHg_err_fullycorr)] = np.nanmean(meanHg_err_fullycorr)

        # Parameters estimated for the ice thickness variogram (correlation range, partial sill):
        # range in meters, partial sill is in percentage of the variance
        params_icethickness = ((2100, 28), (47000, 50),
                               (1354000, 22))

        # Looping for every glacier
        unique_rgiid = list(np.unique(Arctic_discharge_fluxes_with_area['name'].values))

        list_Hl_err_corr, list_Hg_err_corr = ([] for i in range(2))
        for rgiid in unique_rgiid:

            # Select only a given RGIId = glacier
            index_glacier = (Arctic_discharge_fluxes_with_area['name'] == rgiid)

    #         print('Working on glacier: '+rgiid + ' with '+ str(np.count_nonzero(index_glacier))+ ' flux gates...')

            # Variograms parameters formatted for xdem
            list_vgm=[(params_icethickness[0][0], 'Sph', params_icethickness[0][1]),
                      (params_icethickness[1][0], 'Sph', params_icethickness[1][1]),
                      (params_icethickness[2][0], 'Sph', params_icethickness[2][1])]

            # a/ Terminus lost
            # Number of effective samples in the area from the variogram
            dSl = dSldt[index_glacier] * dt[index_glacier]
            neff_l = xdem.spatialstats.neff_circ(area=dSl*10**6, list_vgm=list_vgm)

            # Uncertainty value for the average ice thickness of the glacier considering spatial correlation
            if dSl != 0:
                meanHl_err_corr = meanHl_err_fullycorr[index_glacier][0] / np.sqrt(neff_l)[0]
            else:
                meanHl_err_corr = 0

    #         print('For terminus lost:')
    #         print('Replacing fully correlated averaged ice thickness error {:.1f} m into spatially correlated {:.1f} m'.format(meanHl_err_fullycorr[index_glacier][0], meanHl_err_corr))
            list_Hl_err_corr.append(meanHl_err_corr)

            # b/ Terminus gained
            # Number of effective samples in the area from the variogram
            dSg = dSgdt[index_glacier] * dt[index_glacier]
            neff_g = xdem.spatialstats.neff_circ(area=dSg*10**6, list_vgm=list_vgm)

            # Uncertainty value for the average ice thickness of the glacier considering spatial correlation
            if dSg != 0:
                meanHg_err_corr = meanHg_err_fullycorr[index_glacier][0] / np.sqrt(neff_g)[0]
            else:
                meanHg_err_corr = 0

    #         print('For terminus gained:')
    #         print('Replacing fully correlated averaged ice thickness error {:.1f} m into spatially correlated {:.1f} m'.format(meanHg_err_fullycorr[index_glacier][0],meanHg_err_corr))
            list_Hg_err_corr.append(meanHg_err_corr)

        # Save the old columns of error into a fullycorr column
        Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err_fullycorr'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'].values
        Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err_fullycorr'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'].values
        # Replace the normal error column with the new values
        Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'] = list_Hl_err_corr
        Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'] = list_Hg_err_corr
        print('checking for thickness errors')
        print('----------before-----------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]>0)]))
        print('----------before-----------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]>0)]))
        Arctic_discharge_fluxes_with_area.loc[(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]>0),'thickness_for_area_lost_' + period + '_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]*0.5
        Arctic_discharge_fluxes_with_area.loc[(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]>0),'thickness_for_area_gained_' + period + '_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]*0.5
        print('----------after-----------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]>0)]))
        print('----------after-----------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]>0)]))

        # Load under the names previously used
        meanHl_err = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'].values
        meanHg_err = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'].values
        meanHl_err = np.where(np.isnan(meanHl_err),0,meanHl_err)
        meanHg_err = np.where(np.isnan(meanHg_err),0,meanHg_err)

        # 2/ Propagation glacier -> region

        list_regions = list(np.unique(Arctic_discharge_fluxes_with_area['region'].values))

        list_df_reg = []
        for region in list_regions:

            print('Working on region: '+region)

            # Isolate region of interest
            ind_reg = Arctic_discharge_fluxes_with_area['region'].values == region

            # Terminus ice lost
            Tl_reg = np.nansum(meanHl[ind_reg] * dSldt[ind_reg]) * density / 1000
            mean_meanHl_reg = np.nanmean(meanHl[ind_reg])
            mean_dSldt_reg = np.nanmean(dSldt[ind_reg])
            mean_meanHl_err_reg = np.nanmean(meanHl_err[ind_reg])

            # Terminus ice gained
            Tg_reg = np.nansum(meanHg_err[ind_reg] * dSgdt[ind_reg]) * density / 1000
            mean_meanHg_reg = np.nanmean(meanHg[ind_reg])
            mean_dSgdt_reg = np.nanmean(dSgdt[ind_reg])
            mean_meanHg_err_reg = np.nanmean(meanHg_err[ind_reg])

            # Separating two sources of terminus ice loss: one based on area uncertainty, one based on ice thickness uncertainty
            # Here we only compute only the ice thickness source (second term of Equation 6 in the paper)

            # a/ Terminus mass lost
            # Considering all ice thicknesses uncertainties fully independent
            Tl_errH_uncorr_reg = np.sqrt(np.nansum((meanHl_err[ind_reg] * dSldt[ind_reg]) ** 2)) * density / 1000

            # Uncertainty on the sum, considering everything correlated
            Tl_errH_fulluycorr_reg = np.nansum(meanHl_err[ind_reg] * dSldt[ind_reg]) * density / 1000

            # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
            coords = np.array((Arctic_discharge_fluxes_with_area['easting'].values[ind_reg], Arctic_discharge_fluxes_with_area['northing'].values[ind_reg])).T
            Tl_errH_corr_reg = double_sum_covar(coords=coords, areas=dSldt[ind_reg], errors=meanHl_err[ind_reg],
                                              vgm_func=vgm_func_gla) * np.nansum(dSldt[ind_reg]) * density / 1000

            # b/ Terminus mass gained
            # Considering all ice thicknesses uncertainties fully independent
            Tg_errH_uncorr_reg = np.sqrt(np.nansum((meanHg_err[ind_reg] * dSgdt[ind_reg]) ** 2)) * density / 1000

            # Uncertainty on the sum, considering everything correlated
            Tg_errH_fulluycorr_reg = np.nansum(meanHg_err[ind_reg] * dSgdt[ind_reg]) * density / 1000

            # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
            coords = np.array((Arctic_discharge_fluxes_with_area['easting'].values[ind_reg], Arctic_discharge_fluxes_with_area['northing'].values[ind_reg])).T
            Tg_errH_corr_reg = double_sum_covar(coords=coords, areas=dSgdt[ind_reg], errors=meanHg_err[ind_reg],
                                              vgm_func=vgm_func_gla) * np.nansum(dSgdt[ind_reg]) * density / 1000

            # Store results in a dataframe
            df_res_reg = pd.DataFrame()
            df_res_reg = df_res_reg.assign(region=[region], Tl=[Tl_reg], Tg=[Tg_reg],
                                   Tl_errH_uncorr=[Tl_errH_uncorr_reg], Tl_errH_fullycor=[Tl_errH_fulluycorr_reg],
                                   Tl_errH_corr=[Tl_errH_corr_reg],
                                   Tg_errH_uncorr=[Tg_errH_uncorr_reg], Tg_errH_fullycor=[Tg_errH_fulluycorr_reg],
                                   Tg_errH_corr=[Tg_errH_corr_reg],
                                   mean_Hl=[mean_meanHl_reg], mean_meanHl_err=[mean_meanHl_err_reg] ,mean_dSl=[mean_dSldt_reg],
                                   mean_Hg=[mean_meanHg_reg], mean_meanHg_err=[mean_meanHg_err_reg] ,mean_dSg=[mean_dSgdt_reg])
            list_df_reg.append(df_res_reg)

        if period=='early':
            df_all_reg_early = pd.concat(list_df_reg)
            df_all_reg_early.to_pickle('Greenland/df_all_reg_early_TC_'+regionsornot+date_string+'.pkl')
        if period=='late':
            df_all_reg_late = pd.concat(list_df_reg)
            df_all_reg_late.to_pickle('Greenland/df_all_reg_late_TC_'+regionsornot+date_string+'.pkl')
    #     df_all_reg.to_csv('/home/atom/ongoing/work_frontal_ablation/df_terminus_region.csv')
    print('DONE')

-----------------------All Greenland-----------------------
Running for period: early


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/xdem/spatialstats.py:884: RuntimeWarning: invalid value encountered in true_divide
  std_err = np.sqrt(2*np.pi*full_int / area)


checking for thickness errors
----------before----------- early 55
----------before----------- early 616
----------after----------- early 0
----------after----------- early 0
Working on region: Greenland
Running for period: late


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/xdem/spatialstats.py:884: RuntimeWarning: invalid value encountered in true_divide
  std_err = np.sqrt(2*np.pi*full_int / area)


checking for thickness errors
----------before----------- late 96
----------before----------- late 594
----------after----------- late 0
----------after----------- late 0
Working on region: Greenland
Running for period: total
checking for thickness errors
----------before----------- total 65
----------before----------- total 640
----------after----------- total 0
----------after----------- total 0
Working on region: Greenland
DONE
-----------------------Greenland ice sheet and periphery-----------------------
Running for period: early


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


checking for thickness errors
----------before----------- early 55
----------before----------- early 616
----------after----------- early 0
----------after----------- early 0
Working on region: ice sheet
Working on region: mountain
Running for period: late
checking for thickness errors
----------before----------- late 96
----------before----------- late 594
----------after----------- late 0
----------after----------- late 0
Working on region: ice sheet
Working on region: mountain
Running for period: total
checking for thickness errors
----------before----------- total 65
----------before----------- total 640
----------after----------- total 0
----------after----------- total 0
Working on region: ice sheet
Working on region: mountain
DONE
-----------------------Greenland regions-----------------------
Running for period: early


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/xdem/spatialstats.py:884: RuntimeWarning: invalid value encountered in true_divide
  std_err = np.sqrt(2*np.pi*full_int / area)


checking for thickness errors
----------before----------- early 55
----------before----------- early 616
----------after----------- early 0
----------after----------- early 0
Working on region: E
Working on region: N
Working on region: SE
Working on region: SW
Working on region: W
Running for period: late
checking for thickness errors
----------before----------- late 96
----------before----------- late 594
----------after----------- late 0
----------after----------- late 0
Working on region: E
Working on region: N
Working on region: SE
Working on region: SW
Working on region: W
Running for period: total
checking for thickness errors
----------before----------- total 146
----------before----------- total 647
----------after----------- total 0
----------after----------- total 0
Working on region: E
Working on region: N
Working on region: SE
Working on region: SW
Working on region: W
DONE


In [41]:
#Merge area change data for every glacier with the discharge estimates
#calculate discharge due to terminus retreat/advance with uncertainties (change thickness uncertainty above)
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_early'] = Arctic_discharge_fluxes_with_area['area_km2_gained_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000*density
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_early_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_gained_early'] + Arctic_discharge_fluxes_with_area['perimeter_error_gained_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_gained_early'] - Arctic_discharge_fluxes_with_area['perimeter_error_gained_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err']/1000)))/2*density

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_early'] = Arctic_discharge_fluxes_with_area['area_km2_lost_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000*density
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_early_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_lost_early'] + Arctic_discharge_fluxes_with_area['perimeter_error_lost_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_lost_early'] - Arctic_discharge_fluxes_with_area['perimeter_error_lost_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err']/1000)))/2*density

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_late'] = Arctic_discharge_fluxes_with_area['area_km2_gained_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000*density
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_late_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_gained_late'] + Arctic_discharge_fluxes_with_area['perimeter_error_gained_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_gained_late'] - Arctic_discharge_fluxes_with_area['perimeter_error_gained_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err']/1000)))/2*density

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_late'] = Arctic_discharge_fluxes_with_area['area_km2_lost_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late']/1000*density
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_late_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_lost_late'] + Arctic_discharge_fluxes_with_area['perimeter_error_lost_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_lost_late'] - Arctic_discharge_fluxes_with_area['perimeter_error_lost_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000)))/2*density

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_total'] = Arctic_discharge_fluxes_with_area['area_km2_gained_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total']/1000*density
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_total_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_gained_total'] + Arctic_discharge_fluxes_with_area['perimeter_error_gained_total'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_gained_total'] - Arctic_discharge_fluxes_with_area['perimeter_error_gained_total'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err']/1000)))/2*density

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_total'] = Arctic_discharge_fluxes_with_area['area_km2_lost_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total']/1000*density
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_total_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_lost_total'] + Arctic_discharge_fluxes_with_area['perimeter_error_lost_total'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_lost_total'] - Arctic_discharge_fluxes_with_area['perimeter_error_lost_total'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err']/1000)))/2*density

Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early']=Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early'].fillna(0)
Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early_error']=Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early_error'].fillna(0) 

Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early']=Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early'].fillna(0)
Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early_error']=Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early_error'].fillna(0) 

Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late']=Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late'].fillna(0)
Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late_error']=Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late_error'].fillna(0)

Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late']=Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late'].fillna(0)
Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late_error']=Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late_error'].fillna(0)

Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total']=Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total'].fillna(0)
Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total_error']=Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total_error'].fillna(0)

Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total']=Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total'].fillna(0)
Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total_error']=Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total_error'].fillna(0)

Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_area_error_without_melt']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early_error']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early_error']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])**2)
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_area_error_without_melt']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late_error']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late_error']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])**2)
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_total_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_total_area_error_without_melt']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total_error']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total_error']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])**2)

#put zeros on glaciers that are no longer marine terminating
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed']==0,'terminus_gt_change_per_year_late']=0


#save discharge without correction for melt
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt']
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt']
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt']

#calculate the melt correction
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_early_terminus_change'] = Arctic_discharge_fluxes_with_area['net_area_change_early']/2*Arctic_discharge_fluxes_with_area['terminus_melt_early']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_late_terminus_change'] = Arctic_discharge_fluxes_with_area['net_area_change_late']/2*Arctic_discharge_fluxes_with_area['terminus_melt_late']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_early_err_terminus_change'] = np.abs(Arctic_discharge_fluxes_with_area['net_area_change_early']/2)*Arctic_discharge_fluxes_with_area['terminus_melt_early_err']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_late_err_terminus_change'] = np.abs(Arctic_discharge_fluxes_with_area['net_area_change_late']/2)*Arctic_discharge_fluxes_with_area['terminus_melt_late_err']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_total_terminus_change'] = Arctic_discharge_fluxes_with_area['net_area_change_total']/2*Arctic_discharge_fluxes_with_area['terminus_melt_total']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_total_err_terminus_change'] = np.abs(Arctic_discharge_fluxes_with_area['net_area_change_total']/2)*Arctic_discharge_fluxes_with_area['terminus_melt_total_err']/1000

#only multiply by end of time period for area because otherwise you are double counting it with terminus change, you want just the area unique to discarhge during that time period.
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['net_area_change_early'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_early'][i]*Arctic_discharge_fluxes_with_area['early_poly_early_melt'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_early'][i]*Arctic_discharge_fluxes_with_area['early_poly_early_melt_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_early'][i]<0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['middle_poly_early_melt'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['middle_poly_early_melt_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_early'][i]==0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_discharge'] = 0
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_err_discharge'] = 0
    if Arctic_discharge_fluxes_with_area['net_area_change_late'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['middle_poly_late_melt'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['middle_poly_late_melt_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_late'][i]<0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_late'][i]*Arctic_discharge_fluxes_with_area['late_poly_late_melt'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_late'][i]*Arctic_discharge_fluxes_with_area['late_poly_late_melt_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_late'][i]==0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_discharge'] = 0
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_err_discharge'] = 0

Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_area_error']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_area_error']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_total'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total_without_melt'] + Arctic_discharge_fluxes_with_area['melt_correct_gt_total_terminus_change']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_total_area_error'] = np.sqrt(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total_area_error_without_melt']**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_total_err_terminus_change']**2)

Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_gt']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_gt']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt']=np.nan

Arctic_discharge_fluxes_with_area.loc[:,'FA_not_calculated_in_normal_way_early'] = 0
Arctic_discharge_fluxes_with_area.loc[:,'FA_not_calculated_in_normal_way_late'] = 0

#if the glaciers are gone now, set them to 0
mask = Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'
Arctic_discharge_fluxes_with_area.loc[mask,'terminus_gt_change_per_year_late']= 0
Arctic_discharge_fluxes_with_area.loc[mask,'terminus_gt_change_per_year_late_area_error'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'2010_2020_mean_flux_gt'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'2010_2020_mean_flux_err_gt']= 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_discharge'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_err_discharge'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_terminus_change'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_err_terminus_change'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'2010_2020_mean_flux_gt_without_melt'] = 0

#add melt correction and deal with instances where the melt is greater than the total mass change
discharge_adjusted_early=0
discharge_adjusted_late=0
discharge_adjusted_fixed_early=0
discharge_adjusted_fixed_late=0

discharge_adjusted_fixed_early_total=0
discharge_adjusted_fixed_early_total_err=0
discharge_adjusted_early_total=0
discharge_adjusted_early_total_err=0

discharge_adjusted_fixed_late_total=0
discharge_adjusted_fixed_late_total_err=0
discharge_adjusted_late_total=0
discharge_adjusted_late_total_err=0

for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    for time_period in ['early','late']:
        if (Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i] > Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_terminus_change'][i])&(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i]<0):
            print(i,Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i],Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_terminus_change'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_'+time_period] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_area_error_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_terminus_change'][i]+Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_err_terminus_change'][i]
            if Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period][i] <0:
                Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_'+time_period]=Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+''][i]/2
                Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_'+time_period+'_area_error']=-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period][i]/2
                if time_period == 'early':
                    Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_early'] = 1
                if time_period == 'late':
                    Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_late'] = 1
            if Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period][i]>0:
                print(i,' still a problem terminus ', time_period)
        elif time_period == 'early':
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_terminus_change'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early_area_error'] = np.sqrt(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_err_terminus_change'][i]**2)
        elif time_period == 'late':
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_terminus_change'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late_area_error'] = np.sqrt(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_err_terminus_change'][i]**2)

    if np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i])>Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i]+(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_err_discharge'][i]*(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i]/np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i])))
        if Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]>0:
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_early'] = 1
        if Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]<0:
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt']=(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt'][i])/2
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_early'] = 1
            discharge_adjusted_fixed_early=discharge_adjusted_fixed_early+1
            discharge_adjusted_fixed_early_total=discharge_adjusted_fixed_early_total+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
            discharge_adjusted_fixed_early_total_err=discharge_adjusted_fixed_early_total_err+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'][i]
        discharge_adjusted_early=discharge_adjusted_early+1
        discharge_adjusted_early_total=discharge_adjusted_early_total+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
        discharge_adjusted_early_total_err=discharge_adjusted_early_total_err+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'][i]
    else:
        Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i]
        Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_err_discharge'][i]**2)
        
    if np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i])>Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i]+(Arctic_discharge_fluxes_with_area['melt_correct_gt_late_err_discharge'][i]*(-Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i]/np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i])))
        if Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]>0:
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_late'] = 1
        if Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]<0:
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt']=(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt'][i])/2
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_late'] = 1
            discharge_adjusted_fixed_late=discharge_adjusted_fixed_late+1
            discharge_adjusted_fixed_late_total=discharge_adjusted_fixed_late_total+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
            discharge_adjusted_fixed_late_total_err=discharge_adjusted_fixed_late_total_err+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'][i]
        discharge_adjusted_late=discharge_adjusted_late+1
        discharge_adjusted_late_total=discharge_adjusted_late_total+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
        discharge_adjusted_late_total_err=discharge_adjusted_late_total_err+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'][i]
    else:
        Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i]
        Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_err_discharge'][i]**2)
        
print('Glaciers that had melt bigger than discharge early:',discharge_adjusted_early, ' which was',discharge_adjusted_early_total,' ± ',discharge_adjusted_early_total_err)
print('Glaciers that had melt bigger than discharge early fixed:',discharge_adjusted_fixed_early, ' which was ',discharge_adjusted_fixed_early_total,' ± ',discharge_adjusted_fixed_early_total_err)
print()
print('Glaciers that had melt bigger than discharge late:',discharge_adjusted_late, ' which was',discharge_adjusted_late_total,' ± ',discharge_adjusted_late_total_err)
print('Glaciers that had melt bigger than discharge late fixed:',discharge_adjusted_fixed_late, ' which was',discharge_adjusted_fixed_late_total,' ± ',discharge_adjusted_fixed_late_total_err)
print()
print()
#deal with what happens when the terminus change is greater than the discharge
sum_discharge_early = 0
sum_terminus_early = 0
sum_discharge_late = 0
sum_terminus_late = 0
count_early = 0
count_late = 0
        
for i in range(0,len(Arctic_discharge_fluxes_with_area)): #make terminus change as small as possible to be less than discharge
    if (Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])&(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]>(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i])):
        mass_gap=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]-(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i])
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early']=Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i]+mass_gap/2
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early_area_error']=mass_gap/2
        count_early = count_early +1
    if (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])&(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]>(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i])):
        mass_gap=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]-(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i])
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late']=Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i]+mass_gap/2
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late_area_error']=mass_gap/2
        count_late = count_late +1  
print('Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) early: ',count_early)
print('Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) late: ',count_late)

#calculate frontal ablation wihtout any melt
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error_without_melt']**2)
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error_without_melt']**2)

print('---without melt---')
print('frontal abalation test',np.nansum(Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt']))
print('discharge test',np.nansum(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt']))
print('discharge min',min(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt']))
print('discharge max',max(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt']))
print('terminus change test',np.nansum(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt']))

print('---before')
print('discharge test',np.nansum(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('terminus change test',np.nansum(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']))

#calculate final numbers
#sqrt on uncertainties because of this link: http://lectureonline.cl.msu.edu/~mmp/labs/error/e2.htm
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error']**2)
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error']**2)

print('--------------early--------------')
print('frontal abalation test',np.nansum(Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean']))
print('discharge test',np.nansum(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt']))
print('discharge test >0',np.nansum(Arctic_discharge_fluxes_with_area[Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt']>0]['2000_2010_mean_flux_gt']))
print('discharge min',min(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt']))
print('discharge max',max(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt']))
print('terminus change test',np.nansum(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early']))
print('--------------late--------------')
print('frontal abalation test',np.nansum(Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean']))
print('discharge test',np.nansum(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('discharge test >0',np.nansum(Arctic_discharge_fluxes_with_area[Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']>0]['2010_2020_mean_flux_gt']))
print('discharge min',min(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('discharge max',max(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('terminus change test',np.nansum(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']))

print('---ZI---')
print('frontal abalation test',np.nansum(Arctic_discharge_fluxes_with_area.loc[749]['Frontal_ablation_2010_to_2020_gt_per_yr_mean']))
print('discharge test',np.nansum(Arctic_discharge_fluxes_with_area.loc[749]['2010_2020_mean_flux_gt']))
print('terminus change test',np.nansum(Arctic_discharge_fluxes_with_area.loc[749]['terminus_gt_change_per_year_late']))




Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_area_error_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error_without_melt']

Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_area_error_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error_without_melt']
print(len(Arctic_discharge_fluxes_with_area))
for z in ['','_without_melt']:
    counter_big_term_late = 0
    counter_big_term_early = 0
    count_tiny_early = 0
    count_tiny_late=0
    sum_FA_early =0
    for i in range(0,len(Arctic_discharge_fluxes_with_area)): #make terminus change as small as possible to be less than discharge
        if (Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'+z][i])&(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i]>0.04):
            print('==============================')
            print('discharge early',Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i],Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'+z][i])
            print('terminus early',Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'+z][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'+z][i])
            dis_term_max = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i]+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'+z][i]
            dis_term_min = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'+z][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'+z][i]
            print('max min',dis_term_max,dis_term_min)
            if dis_term_max<dis_term_min:
                print('PROBLEM PROBLEM PROBLEM')
            else:
                print('SUCCESS SUCCESS SUCCCESS')
            dis_term_avg = (dis_term_max+dis_term_min)/2
            dis_term_err = (dis_term_max-dis_term_min)/2
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt'+z] = dis_term_avg
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early'+z] = dis_term_avg-dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early_area_error'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean'+z] = -dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'+z] = dis_term_err
            print('still big early')
            print(Arctic_discharge_fluxes_with_area['RGIId'][i],Arctic_discharge_fluxes_with_area['Name_x'][i])
            print('FA',dis_term_err,Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'+z][i],Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'+z][i])
            counter_big_term_early = counter_big_term_early+1
        if (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i])&(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]>0.04):
            print('==============================')
            print('discharge late',Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i],Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'+z][i])
            print('terminus late',Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'+z][i])
            dis_term_max = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'+z][i]
            dis_term_min = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'+z][i]
            print('max min',dis_term_max,dis_term_min)
            if dis_term_max<dis_term_min:
                print('PROBLEM PROBLEM PROBLEM')
            else:
                print('SUCCESS SUCCESS SUCCCESS')
            dis_term_avg = (dis_term_max+dis_term_min)/2
            dis_term_err = (dis_term_max-dis_term_min)/2
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt'+z] = dis_term_avg
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late'+z] = dis_term_avg-dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late_area_error'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'+z] = -dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'+z] = np.abs(dis_term_err)
            print('still big late')
            print(Arctic_discharge_fluxes_with_area['name'][i])
            print('FA',dis_term_err,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'+z][i],Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'+z][i])
            counter_big_term_late = counter_big_term_late+1      
        #for glacires with small terminus changes, just make the discharge bigger
        if (Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])&(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]<0.05):
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
            print('+++++++++++++++++++++++++',Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=-Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=-Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]
            print(Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i])
            sum_FA_early = sum_FA_early+Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]
            count_tiny_early = count_tiny_early+1
        if (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])&(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]<0.05):
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
            print('+++++++++++++++++++++++++',Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=-Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=-Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]
            print(Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i])
            count_tiny_late = count_tiny_late+1

    print('Number of glaciers that terminus changes was bigger than discahrge that I altered within uncertainties --early ',z,counter_big_term_early)
    print('Glaciers with terminus change less than 0.05 gt a-1 that now have changed discharge to be greater',count_tiny_early, ' accounting for this much FA:  ', sum_FA_early)
    print('Number of glaciers that terminus changes was bigger than discahrge that I altered within uncertainties --late ',z,counter_big_term_late)
    print('Glaciers with terminus change less than 0.05 gt a-1 that now have changed discharge to be greater',count_tiny_late)

print(len(Arctic_discharge_fluxes_with_area))
#If the glacier was no longer marine terminating after 2010 - the later flux should be 0
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'])     
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err'])

Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'])     
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'])

Arctic_discharge_fluxes_with_area=Arctic_discharge_fluxes_with_area.drop(columns=['NAME_x','NAME_y','geometry_x','geometry_y', 'geometry_gained_total','geometry_lost_total', 'geometry_gained_early','geometry_lost_early', 'geometry_gained_late','geometry_lost_late'])
#export data with flux and area change estimates
easting=[]
northing=[]
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    easting.append(Arctic_discharge_fluxes_with_area['easting'][i])
    northing.append(Arctic_discharge_fluxes_with_area['northing'][i])

#add lat lon in decimal degrees
inProj = Proj(init='epsg:3995')
outProj = Proj(init='epsg:4326')    
Arctic_discharge_fluxes_with_area.loc[:,'lon'],Arctic_discharge_fluxes_with_area.loc[:,'lat'] = transform(inProj,outProj,easting,northing)
print(len(Arctic_discharge_fluxes_with_area))    
# Arctic_discharge_fluxes_with_area.to_csv('Arctic_discharge_fluxes_with_area_'+date_string+'.txt')

#export to geopackage
Arctic_discharge_fluxes_with_area_export = gpd.GeoDataFrame(Arctic_discharge_fluxes_with_area, geometry=gpd.points_from_xy(Arctic_discharge_fluxes_with_area.easting, Arctic_discharge_fluxes_with_area.northing),crs='EPSG:3995')
#Add regions as an attribute
# arctic_aois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_AOIs.gpkg')
# arctic_subaois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_subAOIs.gpkg')
# arctic_subaois2 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_subAOIs_v2.gpkg')
# Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_aois)
# Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')
# Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_subaois)
# Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')
# Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_subaois2)
# Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')

Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt']
Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt']

Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean']
Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean']

#differentiate between ice sheet and mountain glacier
Arctic_discharge_fluxes_with_area_export['Glacier_type']=np.nan
for i in range(0,len(Arctic_discharge_fluxes_with_area_export)):
    if Arctic_discharge_fluxes_with_area_export['name'][i][0:3]=='RGI':
        Arctic_discharge_fluxes_with_area_export.loc[i,'Glacier_type']='mountain'
    else:
        Arctic_discharge_fluxes_with_area_export.loc[i,'Glacier_type']='ice sheet'

Arctic_discharge_fluxes_with_area_export.to_file('Greenland/Greenland_frontal_ablation_'+date_string+'.gpkg', driver="GPKG")

Arctic_discharge_fluxes_with_area_export_important = Arctic_discharge_fluxes_with_area_export[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
    '2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt',
    'melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
    'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
    'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err','geometry']]

Arctic_discharge_fluxes_with_area_export_important.to_file('Greenland/Greenland_frontal_ablation_most_important_'+date_string+'.gpkg', driver="GPKG")#change file name based on distance

Glaciers that had melt bigger than discharge early: 232  which was 0.35225003916240405  ±  0.3040572944544334
Glaciers that had melt bigger than discharge early fixed: 140  which was  0.25586454974646283  ±  0.25586454974646283

Glaciers that had melt bigger than discharge late: 325  which was 0.43142559180173995  ±  0.4220143971018442
Glaciers that had melt bigger than discharge late fixed: 234  which was 0.4126032024019481  ±  0.4126032024019481


Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) early:  14
Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) late:  14
---without melt---
frontal abalation test -479.581909747032
discharge test 436.6656680447862
discharge min -0.08395639321695496
discharge max 40.15660416406239
terminus change test -42.91524388636551
---before
discharge test 424.7610619376743
terminus change test

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:395: DeprecationWarning: This function is deprecated. See: htt

In [42]:
print('frontal abalation test',np.nansum(Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean']))
print('discharge test',np.nansum(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('discharge test >0',np.nansum(Arctic_discharge_fluxes_with_area[Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']>0]['2010_2020_mean_flux_gt']))
print('discharge min',min(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('discharge max',max(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']))
print('terminus change test',np.nansum(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']))

Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']





frontal abalation test 466.0955079623968
discharge test 424.7645223893443
discharge test >0 424.7990641387005
discharge min -0.033050033876258496
discharge max 39.79348410851159
terminus change test -41.33326349592656


In [43]:
Arctic_discharge_fluxes_with_area[Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'] != -Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']][['name','2010_2020_mean_flux_gt','terminus_gt_change_per_year_late','thickness_for_area_lost_late','terminus_gt_change_per_year_late_without_melt','area_flux_gt_gained_late','middle_to_late_years_elapsed','area_flux_gt_lost_late','melt_correct_gt_late_terminus_change']]#[0:40]

,name,2010_2020_mean_flux_gt,terminus_gt_change_per_year_late,thickness_for_area_lost_late,terminus_gt_change_per_year_late_without_melt,area_flux_gt_gained_late,middle_to_late_years_elapsed,area_flux_gt_lost_late,melt_correct_gt_late_terminus_change
369,RGI60-05.09689_2,0.000000,NaN,30.000000,NaN,0.0,0.0,0.000000,0.000000e+00
370,RGI60-05.09689_3,0.000000,NaN,30.000000,NaN,0.0,0.0,0.000000,0.000000e+00
379,RGI60-05.09810,0.000000,NaN,30.000000,NaN,0.0,0.0,0.000000,0.000000e+00
414,RGI60-05.10352,0.000024,NaN,30.000000,NaN,0.0,0.0,0.000567,1.128043e-06
630,RGI60-05.15250,0.000006,NaN,30.000000,NaN,0.0,0.0,0.000931,-1.380901e-06
...,...,...,...,...,...,...,...,...,...
605,RGI60-05.13882,0.000000,NaN,30.000000,NaN,0.0,0.0,0.000000,0.000000e+00
620,RGI60-05.13960,0.000000,NaN,30.000000,NaN,0.0,0.0,0.000000,0.000000e+00
621,RGI60-05.13978,0.000012,NaN,30.000000,NaN,0.0,0.0,0.000044,4.657970e-07
703,SE_NONAME9,0.000000,NaN,67.964422,NaN,0.0,0.0,0.000000,0.000000e+00


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
# Arctic_discharge_fluxes_with_area.loc[735]['area_flux_gt_gained_late']
Arctic_discharge_fluxes_with_area.loc[735]['middle_to_late_years_elapsed']

0.0

In [ ]:
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])



In [44]:
Arctic_discharge_fluxes_with_area_export.sum()[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
'2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


terminus_gt_change_per_year_early                   -46.130166
terminus_gt_change_per_year_early_area_error               inf
terminus_gt_change_per_year_late                    -41.333263
terminus_gt_change_per_year_late_area_error            6.36041
2000_2010_mean_flux_gt                              392.426938
2000_2010_mean_flux_err_gt                            9.208844
2010_2020_mean_flux_gt                              424.764522
2010_2020_mean_flux_err_gt                            7.017316
melt_correct_gt_early_terminus_change                 2.363848
melt_correct_gt_early_err_terminus_change            -0.477185
melt_correct_gt_late_terminus_change                  1.606814
melt_correct_gt_late_err_terminus_change             -0.324994
melt_correct_gt_early_discharge                      -17.02993
melt_correct_gt_early_err_discharge                  -3.405986
melt_correct_gt_late_discharge                      -14.472199
melt_correct_gt_late_err_discharge                    -

In [7]:
Arctic_discharge_fluxes_with_area_export.sum()[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
'2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


terminus_gt_change_per_year_early                   -46.538236
terminus_gt_change_per_year_early_area_error          6.697728
terminus_gt_change_per_year_late                    -39.288475
terminus_gt_change_per_year_late_area_error           5.104328
2000_2010_mean_flux_gt                              400.114614
2000_2010_mean_flux_err_gt                            9.364888
2010_2020_mean_flux_gt                              432.985718
2010_2020_mean_flux_err_gt                              7.1004
melt_correct_gt_early_terminus_change                 2.363848
melt_correct_gt_early_err_terminus_change            -0.477185
melt_correct_gt_late_terminus_change                  1.606814
melt_correct_gt_late_err_terminus_change             -0.324994
melt_correct_gt_early_discharge                      -17.02993
melt_correct_gt_early_err_discharge                  -3.405986
melt_correct_gt_late_discharge                        -14.4722
melt_correct_gt_late_err_discharge                    -

In [46]:
Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early_area_error']>100]

,name,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,bed_machine_min,bed_machine_max,do_we_have_an_observation_in_middle_fifth,easting,northing,bed_machine_mean,bed_machine_err,best_thickness,best_thickness_early_mean,best_thickness_late_mean,thickness_mean_uncertainty,length_early_avg,length_early_avg_err,length_late_avg,length_late_avg_err,2000_2010_mean,2000_2010_mean_err,2010_2018_mean,2010_2018_mean_err,GlaThiDa_observations,num_pts,gate_length_km,best_thickness_early,best_thickness_early_err,best_thickness_late,best_thickness_late_err,level_0,index,Glacier_type_x,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,Glacier_type_y,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,Glacier_type_x,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,Glacier_type_y,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,Glacier_type_x,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,Glacier_type_y,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_OIB_thickness_lost_early,average_OIB_thickness_gained_early,average_OIB_thickness_lost_late,average_OIB_thickness_gained_late,average_OIB_thickness_lost_total,average_OIB_thickness_gained_total,average_bedmachine_thickness_lost_early,average_bedmachine_thickness_gained_early,average_bedmachine_thickness_lost_late,average_bedmachine_thickness_gained_late,average_bedmachine_thickness_lost_total,average_bedmachine_thickness_gained_total,average_bedmachine_thickness_lost_early_err,average_bedmachine_thickness_gained_early_err,average_bedmachine_thickness_lost_late_err,average_bedmachine_thickness_gained_late_err,average_bedmachine_thickness_lost_total_err,average_bedmachine_thickness_gained_total_err,average_Milan_thickness_lost_early,average_Milan_thickness_gained_early,average_Milan_thickness_lost_late,average_Milan_thickness_gained_late,average_Milan_thickness_lost_early_err,average_Milan_thickness_gained_early_err,average_Milan_thickness_lost_late_err,average_Milan_thickness_gained_late_err,average_Milan_thickness_lost_total,average_Milan_thickness_gained_total,average_Milan_thickness_lost_total_err,average_Milan_thickness_gained_total_err,early_poly_early_melt,area_below_fluxgate_early,middle_poly_early_melt,middle_poly_late_melt,area_below_fluxgate_middle,late_poly_late_melt,area_below_fluxgate_late,lat,lon,early_poly_early_melt_err,middle_poly_early_melt_err,middle_poly_late_melt_err,late_poly_late_melt_err,terminus_melt_early,terminus_melt_late,terminus_melt_early_err,terminus_melt_late_err,terminus_melt_total,terminus_melt_total_err,average_Milan_thickness_lost_early_debiased,average_Milan_thickness_gained_early_debiased,average_bedmachine_thickness_gained_early_debiased,average_Milan_thickness_lost_late_debiased,average_Milan_thickness_gained_late_debiased,average_Milan_thickness_lost_total_debiased,average_Milan_thickness_gained_total_debiased,thickness_for_area_lost_early_source,thickness_for_area_gained_early_source,thickness_for_area_lost_late_source,thickness_for_area_gained_late_source,thickness_for_area_lost_early,thickness_for_area_lost_early_err,thickness_for_area_gained_early,thickness_for_area_gained_early_err,thickness_for_area_lost_late,thickness_for_area_lost_late_err,thickness_for_area_gained_late,thickness_for_area_gai

In [47]:
Arctic_discharge_fluxes_with_area_export.to_csv('Greenland/Greenland_frontal_ablation_'+date_string+'.csv')


In [49]:
gdf = pd.read_csv('Greenland/Greenland_frontal_ablation_'+date_string+'.csv')
gdf=gdf.drop(columns=['geometry','Unnamed: 0','level_0','index'])
gdf = gpd.GeoDataFrame(gdf,geometry=gpd.points_from_xy(gdf.easting,gdf.northing))
gdf = gdf.set_crs('EPSG:3995')
# gdf = gdf.set_crs('EPSG:4326')
# gdf = gdf.to_crs('EPSG:3995')
gdf.to_file('Greenland/Greenland_frontal_ablation_GOOD_'+date_string+'.gpkg', driver="GPKG")

,name,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,bed_machine_min,bed_machine_max,do_we_have_an_observation_in_middle_fifth,easting,northing,bed_machine_mean,bed_machine_err,best_thickness,best_thickness_early_mean,best_thickness_late_mean,thickness_mean_uncertainty,length_early_avg,length_early_avg_err,length_late_avg,length_late_avg_err,2000_2010_mean,2000_2010_mean_err,2010_2018_mean,2010_2018_mean_err,GlaThiDa_observations,num_pts,gate_length_km,best_thickness_early,best_thickness_early_err,best_thickness_late,best_thickness_late_err,Glacier_type_x,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,Glacier_type_y,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,Glacier_type_x.1,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,Glacier_type_y.1,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,Glacier_type_x.2,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,Glacier_type_y.2,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_OIB_thickness_lost_early,average_OIB_thickness_gained_early,average_OIB_thickness_lost_late,average_OIB_thickness_gained_late,average_OIB_thickness_lost_total,average_OIB_thickness_gained_total,average_bedmachine_thickness_lost_early,average_bedmachine_thickness_gained_early,average_bedmachine_thickness_lost_late,average_bedmachine_thickness_gained_late,average_bedmachine_thickness_lost_total,average_bedmachine_thickness_gained_total,average_bedmachine_thickness_lost_early_err,average_bedmachine_thickness_gained_early_err,average_bedmachine_thickness_lost_late_err,average_bedmachine_thickness_gained_late_err,average_bedmachine_thickness_lost_total_err,average_bedmachine_thickness_gained_total_err,average_Milan_thickness_lost_early,average_Milan_thickness_gained_early,average_Milan_thickness_lost_late,average_Milan_thickness_gained_late,average_Milan_thickness_lost_early_err,average_Milan_thickness_gained_early_err,average_Milan_thickness_lost_late_err,average_Milan_thickness_gained_late_err,average_Milan_thickness_lost_total,average_Milan_thickness_gained_total,average_Milan_thickness_lost_total_err,average_Milan_thickness_gained_total_err,early_poly_early_melt,area_below_fluxgate_early,middle_poly_early_melt,middle_poly_late_melt,area_below_fluxgate_middle,late_poly_late_melt,area_below_fluxgate_late,lat,lon,early_poly_early_melt_err,middle_poly_early_melt_err,middle_poly_late_melt_err,late_poly_late_melt_err,terminus_melt_early,terminus_melt_late,terminus_melt_early_err,terminus_melt_late_err,terminus_melt_total,terminus_melt_total_err,average_Milan_thickness_lost_early_debiased,average_Milan_thickness_gained_early_debiased,average_bedmachine_thickness_gained_early_debiased,average_Milan_thickness_lost_late_debiased,average_Milan_thickness_gained_late_debiased,average_Milan_thickness_lost_total_debiased,average_Milan_thickness_gained_total_debiased,thickness_for_area_lost_early,thickness_for_area_lost_early_err,thickness_for_area_gained_early,thickness_for_area_gained_early_err,thickness_for_area_lost_late,thickness_for_area_lost_late_err,thickness_for_area_gained_late,thickness_for_area_gained_late_err,thickness_for_area_lost_total,thickness_for_area_lost_total_err,thickness_for_area_gained_total,thickness_for_area_gained_total_err,region,thic

In [105]:
pd.read_file('Greenland/Greenland_frontal_ablation_'+date_string+'.gpkg', driver="GPKG")

,name,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,bed_machine_min,bed_machine_max,do_we_have_an_observation_in_middle_fifth,easting,northing,bed_machine_mean,bed_machine_err,best_thickness,best_thickness_early_mean,best_thickness_late_mean,thickness_mean_uncertainty,length_early_avg,length_early_avg_err,length_late_avg,length_late_avg_err,2000_2010_mean,2000_2010_mean_err,2010_2018_mean,2010_2018_mean_err,GlaThiDa_observations,num_pts,gate_length_km,best_thickness_early,best_thickness_early_err,best_thickness_late,best_thickness_late_err,level_0,index,Glacier_type_x,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,Glacier_type_y,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_OIB_thickness_lost_early,average_OIB_thickness_gained_early,average_OIB_thickness_lost_late,average_OIB_thickness_gained_late,average_OIB_thickness_lost_total,average_OIB_thickness_gained_total,average_bedmachine_thickness_lost_early,average_bedmachine_thickness_gained_early,average_bedmachine_thickness_lost_late,average_bedmachine_thickness_gained_late,average_bedmachine_thickness_lost_total,average_bedmachine_thickness_gained_total,average_bedmachine_thickness_lost_early_err,average_bedmachine_thickness_gained_early_err,average_bedmachine_thickness_lost_late_err,average_bedmachine_thickness_gained_late_err,average_bedmachine_thickness_lost_total_err,average_bedmachine_thickness_gained_total_err,average_Milan_thickness_lost_early,average_Milan_thickness_gained_early,average_Milan_thickness_lost_late,average_Milan_thickness_gained_late,average_Milan_thickness_lost_early_err,average_Milan_thickness_gained_early_err,average_Milan_thickness_lost_late_err,average_Milan_thickness_gained_late_err,average_Milan_thickness_lost_total,average_Milan_thickness_gained_total,average_Milan_thickness_lost_total_err,average_Milan_thickness_gained_total_err,early_poly_early_melt,area_below_fluxgate_early,middle_poly_early_melt,middle_poly_late_melt,area_below_fluxgate_middle,late_poly_late_melt,area_below_fluxgate_late,lat,lon,early_poly_early_melt_err,middle_poly_early_melt_err,middle_poly_late_melt_err,late_poly_late_melt_err,terminus_melt_early,terminus_melt_late,terminus_melt_early_err,terminus_melt_late_err,terminus_melt_total,terminus_melt_total_err,average_Milan_thickness_lost_early_debiased,average_Milan_thickness_gained_early_debiased,average_bedmachine_thickness_gained_early_debiased,average_Milan_thickness_lost_late_debiased,average_Milan_thickness_gained_late_debiased,average_Milan_thickness_lost_total_debiased,average_Milan_thickness_gained_total_debiased,thickness_for_area_lost_early,thickness_for_area_lost_early_err,thickness_for_area_gained_early,thickness_for_area_gained_early_err,thickness_for_area_lost_late,thickness_for_area_lost_late_err,thickness_for_area_gained_late,thickness_for_area_gained_late_err,thickness_for_area_lost_total,thickness_for_area_lost_total_err,thickness_for_area_gained_total,thickness_for_area_gained_total_err,region,thickness_for_area_lost_early_err_fullycorr,thickness_for_

In [9]:
Arctic_discharge_fluxes_with_area_export.groupby('Glacier_type').sum()[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
'2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt',#'melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
#     'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

,terminus_gt_change_per_year_early,terminus_gt_change_per_year_early_area_error,terminus_gt_change_per_year_late,terminus_gt_change_per_year_late_area_error,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,Frontal_ablation_2000_to_2010_gt_per_yr_mean,Frontal_ablation_2000_to_2010_gt_per_yr_mean_err,Frontal_ablation_2010_to_2020_gt_per_yr_mean,Frontal_ablation_2010_to_2020_gt_per_yr_mean_err
Glacier_type,,,,,,,,,,,,
ice sheet,-43.509804,8.787714,-39.924398,5.597459,371.151209,40.047901,412.772300,40.721081,414.661013,43.055444,452.696664,42.384694
mountain,-5.046446,2.487751,-4.745862,1.741432,3.430388,1.904413,2.857107,1.299577,8.476834,3.708147,7.602919,2.584536


In [90]:
Arctic_discharge_fluxes_with_area_export.sort_values('2010_2020_mean_flux_gt',ascending=False).head(5)[['name',
'2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change',# 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
#     'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
'terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

,name,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,melt_correct_gt_early_terminus_change,terminus_gt_change_per_year_early,terminus_gt_change_per_year_early_area_error,terminus_gt_change_per_year_late,terminus_gt_change_per_year_late_area_error,Frontal_ablation_2000_to_2010_gt_per_yr_mean,Frontal_ablation_2000_to_2010_gt_per_yr_mean_err,Frontal_ablation_2010_to_2020_gt_per_yr_mean,Frontal_ablation_2010_to_2020_gt_per_yr_mean_err
74,JAKOBSHAVN_ISBRAE,30.909272,2.503443,37.835838,3.049797,0.226226,-3.176917,1.748463,-0.160956,0.148708,34.086189,3.053580,37.996794,3.053421
78,KANGERLUSSUAQ,23.297371,2.069581,24.360856,2.140396,0.021691,-0.420315,0.238439,-0.122243,0.013681,23.717686,2.083272,24.483100,2.140440
57,HELHEIMGLETSCHER,24.112033,1.507283,23.753559,1.439086,0.040048,-0.961066,0.526900,-0.287755,0.112320,25.073099,1.596723,24.041314,1.443463
85,KOGE_BUGT_C,15.224837,0.976681,16.169946,0.945058,0.005804,-0.091674,0.056688,-0.010702,0.007204,15.316511,0.978325,16.180648,0.945086
743,ZACHARIAE_ISSTROM,9.875074,0.628807,12.896091,0.606011,0.232423,-22.708889,11.933984,-8.859339,3.376105,32.583963,11.950538,21.755430,3.430063


In [12]:
Arctic_discharge_fluxes_with_area_export[['melt_correct_gt_early_discharge','melt_correct_gt_late_discharge']].sum()

melt_correct_gt_early_discharge   -16.932362
melt_correct_gt_late_discharge    -14.411122
dtype: float64